In [1]:
!pip install -q transformers accelerate torch timm scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency 

In [ ]:
# Run in a notebook cell (prefix with !)
!pip install -U "requests>=2.31.0" "transformers>=4.40.0" "huggingface_hub>=0.18.0"


In [ ]:
!gdown 1jEJ2nf5CDJknq80ogzU-Uyz7jbBi-1LZ --fuzzy
!unzip *.zip

In [ ]:
!gdown --folder 19yaav8ORSVj9DeJUaHKq1H3HtVnkClBw --remaining-ok

In [ ]:
import zipfile
import os

# Path to the zip file
zip_path = '/kaggle/working/Memotion 3/memotion3.zip'

# Directory where you want to extract (current working directory)
extract_to = '/kaggle/working/'

# Password for the encrypted zip file
password = b'memotion3taskaaai@22'

# Unzip the file with password
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to, pwd=password)
    print(f"Successfully extracted to: {extract_to}")
    
# List the extracted contents
print("\nExtracted files/folders:")
for item in os.listdir(extract_to):
    print(f"  - {item}")

In [ ]:
import pandas as pd
import os
import shutil
from tqdm import tqdm

# --- 1. DEFINE PATHS AND PARAMETERS ---
ORIGINAL_TRAIN_IMG_DIR = '/kaggle/working/trainImages/'
ORIGINAL_CSV_PATH = '/kaggle/working/memotion3/train.csv'

# Image filename column - the images are named by index (0.jpg, 1.jpg, etc.)
IMAGE_FILENAME_COLUMN = 'Unnamed: 0'

# Number of images for validation (last 1500 rows)
VALIDATION_SPLIT_SIZE = 1500

# --- 2. DEFINE OUTPUT DIRECTORIES ---
OUTPUT_BASE_DIR = '/kaggle/working/'
NEW_VAL_DIR = os.path.join(OUTPUT_BASE_DIR, 'validation_images/')
NEW_TRAIN_DIR = os.path.join(OUTPUT_BASE_DIR, 'new_train_images/')

# Create directories
os.makedirs(NEW_VAL_DIR, exist_ok=True)
os.makedirs(NEW_TRAIN_DIR, exist_ok=True)
print(f"Created directories:\n- {NEW_VAL_DIR}\n- {NEW_TRAIN_DIR}")

# --- 3. LOAD AND SPLIT THE CSV FILE ---
print("\nLoading CSV file...")
df = pd.read_csv(ORIGINAL_CSV_PATH)
print(f"CSV columns: {list(df.columns)}")
print(f"\nFirst few rows of the dataframe:")
print(df.head())

# Verify the image column exists
if IMAGE_FILENAME_COLUMN not in df.columns:
    print(f"\n⚠️ WARNING: Column '{IMAGE_FILENAME_COLUMN}' not found!")
    print(f"Available columns: {list(df.columns)}")
    print("\n🔍 Please update IMAGE_FILENAME_COLUMN to match one of the above columns.")
else:
    print(f"\n✅ Found image column: '{IMAGE_FILENAME_COLUMN}'")

# Split: Last 1500 rows for validation, rest for training
val_df = df.iloc[-VALIDATION_SPLIT_SIZE:]
train_df = df.iloc[:-VALIDATION_SPLIT_SIZE]

print(f"\n📊 Dataset Split:")
print(f"Original dataframe shape: {df.shape}")
print(f"New training dataframe shape: {train_df.shape}")
print(f"New validation dataframe shape: {val_df.shape}")

# Verify the split
print(f"\n🔍 Verification:")
print(f"Last training index: {train_df.index[-1]}")
print(f"First validation index: {val_df.index[0]}")
print(f"Last validation index: {val_df.index[-1]}")

# --- 4. COPY IMAGE FILES ---
print(f"\n📁 Copying {len(val_df)} validation images...")
copied_val = 0
missing_val = 0
for idx in tqdm(val_df[IMAGE_FILENAME_COLUMN]):
    filename = f"{idx}.jpg"  # Convert index to filename format
    source_path = os.path.join(ORIGINAL_TRAIN_IMG_DIR, filename)
    if os.path.exists(source_path):
        destination_path = os.path.join(NEW_VAL_DIR, filename)
        shutil.copy(source_path, destination_path)
        copied_val += 1
    else:
        missing_val += 1
        if missing_val <= 5:  # Only print first 5 missing files
            print(f"Missing: {source_path}")

print(f"\n📁 Copying {len(train_df)} training images...")
copied_train = 0
missing_train = 0
for idx in tqdm(train_df[IMAGE_FILENAME_COLUMN]):
    filename = f"{idx}.jpg"  # Convert index to filename format
    source_path = os.path.join(ORIGINAL_TRAIN_IMG_DIR, filename)
    if os.path.exists(source_path):
        destination_path = os.path.join(NEW_TRAIN_DIR, filename)
        shutil.copy(source_path, destination_path)
        copied_train += 1
    else:
        missing_train += 1
        if missing_train <= 5:  # Only print first 5 missing files
            print(f"Missing: {source_path}")

# --- 5. SAVE THE NEW CSV FILES ---
new_train_csv_path = os.path.join(OUTPUT_BASE_DIR, 'train_split.csv')
new_val_csv_path = os.path.join(OUTPUT_BASE_DIR, 'validation_split.csv')

train_df.to_csv(new_train_csv_path, index=False)
val_df.to_csv(new_val_csv_path, index=False)

print(f"\n✅ Successfully created split datasets!")
print(f"New training CSV saved to: {new_train_csv_path}")
print(f"New validation CSV saved to: {new_val_csv_path}")

# --- 6. VERIFICATION ---
print("\n📊 Final Verification:")
print(f"Training CSV rows: {len(train_df)}")
print(f"Validation CSV rows: {len(val_df)}")
print(f"Images copied to training directory: {copied_train} (missing: {missing_train})")
print(f"Images copied to validation directory: {copied_val} (missing: {missing_val})")
print(f"Images in new training directory: {len(os.listdir(NEW_TRAIN_DIR))}")
print(f"Images in new validation directory: {len(os.listdir(NEW_VAL_DIR))}")

# Verify last 1500 split
print(f"\n✅ Confirmed: Validation set contains the LAST {VALIDATION_SPLIT_SIZE} rows from the original CSV")

In [ ]:
# --- 7. CREATE FINAL ZIP FILES WITH IMAGES + CSV ---

import shutil

# Temporary folders for packaging
TRAIN_PACKAGE_DIR = os.path.join(OUTPUT_BASE_DIR, 'training_dataset/')
VAL_PACKAGE_DIR = os.path.join(OUTPUT_BASE_DIR, 'validation_dataset/')

# Create directories
os.makedirs(TRAIN_PACKAGE_DIR, exist_ok=True)
os.makedirs(VAL_PACKAGE_DIR, exist_ok=True)

# Copy CSVs and image folders into these directories
# Training
shutil.copy(new_train_csv_path, os.path.join(TRAIN_PACKAGE_DIR, 'train_split.csv'))
shutil.copytree(NEW_TRAIN_DIR, os.path.join(TRAIN_PACKAGE_DIR, 'new_train_images'), dirs_exist_ok=True)

# Validation
shutil.copy(new_val_csv_path, os.path.join(VAL_PACKAGE_DIR, 'validation_split.csv'))
shutil.copytree(NEW_VAL_DIR, os.path.join(VAL_PACKAGE_DIR, 'validation_images'), dirs_exist_ok=True)

# Create ZIP archives
train_zip_path = shutil.make_archive(os.path.join(OUTPUT_BASE_DIR, 'training_dataset'), 'zip', TRAIN_PACKAGE_DIR)
val_zip_path = shutil.make_archive(os.path.join(OUTPUT_BASE_DIR, 'validation_dataset'), 'zip', VAL_PACKAGE_DIR)

print("\n✅ Created ZIP files:")
print(f"📦 Training dataset ZIP: {train_zip_path}")
print(f"📦 Validation dataset ZIP: {val_zip_path}")

# Optional: Verify contents
print("\n📂 Contents of training dataset folder:")
print(os.listdir(TRAIN_PACKAGE_DIR))

print("\n📂 Contents of validation dataset folder:")
print(os.listdir(VAL_PACKAGE_DIR))


In [ ]:
"""
Part 1: Configuration, Imports, and Utility Functions
Run this cell first to set up the environment
"""

import os
import yaml
import random
import zipfile
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast

from transformers import AutoModel, AutoTokenizer, CLIPModel, CLIPProcessor
import torchvision.transforms as transforms
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support, 
                            mean_squared_error, mean_absolute_error, confusion_matrix, 
                            classification_report, average_precision_score, f1_score)
from scipy.stats import spearmanr

print("✓ All imports successful")

# ==================== Configuration ====================

CONFIG = """
# Multi-modal Fusion Configuration - ENHANCED
TEXT_MODEL: "google/muril-base-cased"
IMAGE_MODEL: "openai/clip-vit-base-patch32"
TEXT_DIM: 768
IMAGE_DIM: 768
FUSION_DIM: 512
FUSION_OUT_DIM: 512
MAX_LEN: 128
IMG_SIZE: 224
BATCH_SIZE: 16
GRADIENT_ACCUMULATION_STEPS: 2
LR_HEADS: 0.001
LR_BACKBONE: 0.00001
WEIGHT_DECAY: 0.01
EPOCHS: 5
LOSS_WEIGHTS:
  sentiment: 1.0
  emotion: 1.0
  intensity: 0.5
SEED: 42
DEVICE: "cuda"
CHECKPOINT_PATH: "/kaggle/working/checkpoints"
NUM_SENTIMENT_CLASSES: 5
NUM_EMOTION_CLASSES: 4
POOLING: "mean"
USE_AMP: true
GRADIENT_CLIP: 1.0
SCHEDULER: "cosine"
WARMUP_EPOCHS: 1
UNFREEZE_BACKBONE_EPOCH: 3
LABEL_SMOOTHING: 0.1
CROSS_ATTN_HEADS: 8
CROSS_ATTN_DROPOUT: 0.1
EMO_THRESHOLDS: [0.5, 0.5, 0.5, 0.5]
"""

# ==================== Utility Functions ====================

def set_seed(seed: int):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"✓ Random seed set to {seed}")

def load_config(config_str: str) -> dict:
    """Load configuration from YAML string"""
    cfg = yaml.safe_load(config_str)
    print("✓ Configuration loaded")
    return cfg

def save_checkpoint(state: dict, path: str):
    """Save model checkpoint"""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(state, path)
    print(f"✓ Checkpoint saved to {path}")

def load_checkpoint(model, optimizer, path: str, device: str):
    """Load model checkpoint"""
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print(f"✓ Checkpoint loaded from {path}")
    return checkpoint['epoch'], checkpoint.get('best_metric', 0)

def get_tokenizer(cfg: dict):
    """Get tokenizer for text model"""
    tokenizer = AutoTokenizer.from_pretrained(cfg['TEXT_MODEL'])
    print(f"✓ Tokenizer loaded: {cfg['TEXT_MODEL']}")
    return tokenizer

def get_image_transform(cfg: dict, is_train: bool = True):
    """Get image transforms - CLIP-specific normalization"""
    if is_train:
        transform = transforms.Compose([
            transforms.Resize((cfg['IMG_SIZE'], cfg['IMG_SIZE'])),
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], 
                               std=[0.26862954, 0.26130258, 0.27577711])
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((cfg['IMG_SIZE'], cfg['IMG_SIZE'])),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], 
                               std=[0.26862954, 0.26130258, 0.27577711])
        ])
    
    print(f"✓ Image transform created (train={is_train})")
    return transform

def explore_dataset(df, name="Dataset"):
    """Explore dataset structure and label distributions"""
    print(f"\n{'='*70}")
    print(f"{name} Exploration")
    print(f"{'='*70}")
    
    print(f"\nShape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    
    # Check for label columns
    label_cols = ['humour', 'humor', 'sarcastic', 'sarcasm', 'offensive', 'motivational', 'overall', 'sentiment']
    
    for col in label_cols:
        if col in df.columns:
            print(f"\n{col} distribution:")
            print(df[col].value_counts())
    
    print(f"\nMissing values:")
    print(df.isnull().sum())
    
    print(f"\nSample rows:")
    print(df.head(3))
    
    print(f"{'='*70}\n")

# Initialize configuration
cfg = load_config(CONFIG)
set_seed(cfg['SEED'])

# Set device
device = torch.device(cfg['DEVICE'] if torch.cuda.is_available() else 'cpu')
print(f"✓ Using device: {device}")

if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  CUDA Version: {torch.version.cuda}")

print("\n" + "="*70)
print("Part 1 Complete - Configuration and utilities loaded")
print("="*70)

In [ ]:
# ==================== Part 2: Data Preparation and Dataset Class ====================

def prepare_data(cfg):
    """Extract and prepare dataset"""
    print("Preparing datasets...")
    
    train_dir = '/kaggle/working/train'
    val_dir = '/kaggle/working/val'
    
    # Extract if needed
    if not os.path.exists(train_dir):
        print("Extracting training dataset...")
        with zipfile.ZipFile('/kaggle/working/training_dataset.zip', 'r') as zip_ref:
            zip_ref.extractall('/kaggle/working/train')
    else:
        print("✓ Training dataset already extracted")
    
    if not os.path.exists(val_dir):
        print("Extracting validation dataset...")
        with zipfile.ZipFile('/kaggle/working/validation_dataset.zip', 'r') as zip_ref:
            zip_ref.extractall('/kaggle/working/val')
    else:
        print("✓ Validation dataset already extracted")
    
    # Find CSV files
    train_csv = None
    val_csv = None
    
    for root, dirs, files in os.walk(train_dir):
        for file in files:
            if file.endswith('.csv'):
                train_csv = os.path.join(root, file)
                break
        if train_csv:
            break
    
    for root, dirs, files in os.walk(val_dir):
        for file in files:
            if file.endswith('.csv'):
                val_csv = os.path.join(root, file)
                break
        if val_csv:
            break
    
    if train_csv is None or val_csv is None:
        raise FileNotFoundError("Could not find CSV files")
    
    print(f"✓ Loading training data from {train_csv}")
    train_df = pd.read_csv(train_csv)
    
    print(f"✓ Loading validation data from {val_csv}")
    val_df = pd.read_csv(val_csv)
    
    # Find image directories
    train_img_dir = None
    val_img_dir = None
    
    for root, dirs, files in os.walk(train_dir):
        img_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png'))]
        if img_files:
            train_img_dir = root
            break
        if 'images' in dirs:
            train_img_dir = os.path.join(root, 'images')
            break
    
    for root, dirs, files in os.walk(val_dir):
        img_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png'))]
        if img_files:
            val_img_dir = root
            break
        if 'images' in dirs:
            val_img_dir = os.path.join(root, 'images')
            break
    
    if train_img_dir is None or val_img_dir is None:
        raise FileNotFoundError("Could not find image directories")
    
    print(f"\n{'='*70}")
    print(f"Dataset Information:")
    print(f"  Train CSV: {train_csv}")
    print(f"  Train Images: {train_img_dir}")
    print(f"  Train Samples: {len(train_df)}")
    print(f"  Val CSV: {val_csv}")
    print(f"  Val Images: {val_img_dir}")
    print(f"  Val Samples: {len(val_df)}")
    print(f"{'='*70}\n")
    
    return train_df, val_df, train_img_dir, val_img_dir

class MemeDataset(Dataset):
    """Dataset for meme sentiment and emotion analysis"""
    
    def __init__(self, df: pd.DataFrame, tokenizer, image_transform, image_dir: str, cfg: dict):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.image_transform = image_transform
        self.image_dir = image_dir
        self.cfg = cfg
        
        self._detect_columns()
        print(f"✓ Dataset initialized with {len(self.df)} samples")
        
    def _detect_columns(self):
        """Detect column names in the dataframe"""
        cols = self.df.columns.tolist()
        
        # Image column
        possible_image_cols = ['image_name', 'image', 'img_name', 'filename', 'image_id', 'Unnamed: 0']
        self.image_col = next((col for col in possible_image_cols if col in cols), cols[0])
        
        # Text column
        possible_text_cols = ['text', 'ocr_text', 'caption', 'extracted_text', 'OCR_extracted_text', 'ocr']
        self.text_col = next((col for col in possible_text_cols if col in cols), None)
        
        # Sentiment column
        possible_sentiment_cols = ['sentiment', 'overall_sentiment', 'sentiment_label', 'overall']
        self.sentiment_col = next((col for col in possible_sentiment_cols if col in cols), None)
        
        print(f"  Detected columns - Image: {self.image_col}, Text: {self.text_col}, Sentiment: {self.sentiment_col}")
        
        self._create_label_mappings()
    
    def _create_label_mappings(self):
        """Create mappings for string labels to numeric values"""
        # Sentiment mapping (5 classes)
        self.sentiment_map = {
            'very_positive': 0,
            'positive': 1,
            'neutral': 2,
            'negative': 3,
            'very_negative': 4
        }
        
        # Humor mapping (binary)
        self.humor_map = {
            'not_funny': 0,
            'funny': 1,
            'very_funny': 1,
            'hilarious': 1
        }
        
        # Sarcasm mapping (binary)
        self.sarcasm_map = {
            'not_sarcastic': 0,
            'general': 1,
            'twisted_meaning': 1,
            'very_twisted': 1
        }
        
        # Offensive mapping (binary)
        self.offensive_map = {
            'not_offensive': 0,
            'slight': 1,
            'very_offensive': 1,
            'hateful_offensive': 1
        }
        
        # Motivational mapping (binary)
        self.motivational_map = {
            'not_motivational': 0,
            'motivational': 1
        }
    
    def _map_label(self, value, mapping, default=0):
        """Map string label to numeric value"""
        if pd.isna(value):
            return default
        if isinstance(value, str):
            return mapping.get(value.lower().strip(), default)
        try:
            return int(value)
        except:
            return default
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int) -> Dict:
        row = self.df.iloc[idx]
        
        # Load image
        image_name = row[self.image_col] if self.image_col else f"{idx}.jpg"
        
        if isinstance(image_name, (int, float)):
            image_name = f"{int(image_name)}.jpg"
        else:
            image_name = str(image_name)
            if not any(image_name.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
                image_name = f"{image_name}.jpg"
        
        image_path = os.path.join(self.image_dir, image_name)
        
        try:
            image = Image.open(image_path).convert('RGB')
            image = self.image_transform(image)
        except Exception as e:
            # Use blank image on error
            image = torch.zeros(3, self.cfg['IMG_SIZE'], self.cfg['IMG_SIZE'])
        
        # Tokenize text
        text = str(row.get(self.text_col, '')) if self.text_col else ''
        if text in ['nan', '', 'None'] or text is None:
            text = 'No text available'
            
        encoding = self.tokenizer(
            text,
            max_length=self.cfg['MAX_LEN'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Get sentiment label (5 classes)
        sentiment_label = 2  # default neutral
        if self.sentiment_col and self.sentiment_col in row:
            sent_val = row[self.sentiment_col]
            sentiment_label = self._map_label(sent_val, self.sentiment_map, default=2)
        
        sentiment_label = torch.tensor(sentiment_label, dtype=torch.long)
        
        # Get emotion labels (multi-label binary)
        humor_val = row.get('humour', row.get('humor', 0))
        sarcasm_val = row.get('sarcastic', row.get('sarcasm', 0))
        offensive_val = row.get('offensive', 0)
        motivational_val = row.get('motivational', 0)
        
        emotion_labels = torch.tensor([
            float(self._map_label(humor_val, self.humor_map, default=0)),
            float(self._map_label(sarcasm_val, self.sarcasm_map, default=0)),
            float(self._map_label(offensive_val, self.offensive_map, default=0)),
            float(self._map_label(motivational_val, self.motivational_map, default=0))
        ], dtype=torch.float)
        
        # Get intensity score (normalized 0-1)
        intensity_val = row.get('overall_sentiment', row.get('overall', 0.0))
        if isinstance(intensity_val, str):
            intensity_score_map = {
                'very_positive': 1.0,
                'positive': 0.75,
                'neutral': 0.5,
                'negative': 0.25,
                'very_negative': 0.0
            }
            intensity_val = intensity_score_map.get(intensity_val.lower().strip(), 0.5)
        
        intensity = torch.tensor(float(intensity_val) if not pd.isna(intensity_val) else 0.5, dtype=torch.float)
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'image': image,
            'sentiment_label': sentiment_label,
            'emotion_labels': emotion_labels,
            'intensity': intensity
        }

def collate_fn(batch: List[Dict]):
    """Collate function for batching"""
    return {
        'input_ids': torch.stack([item['input_ids'] for item in batch]),
        'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
        'image': torch.stack([item['image'] for item in batch]),
        'sentiment_label': torch.stack([item['sentiment_label'] for item in batch]),
        'emotion_labels': torch.stack([item['emotion_labels'] for item in batch]),
        'intensity': torch.stack([item['intensity'] for item in batch])
    }

# ==================== Run Data Preparation ====================

try:
    train_df, val_df, train_img_dir, val_img_dir = prepare_data(cfg)
    explore_dataset(train_df, "Training Set")
    explore_dataset(val_df, "Validation Set")
    
    print("\n" + "="*70)
    print("Part 2 Complete - Data preparation successful")
    print("="*70)
    
except Exception as e:
    print(f"\n✗ Error in data preparation: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# ==================== Part 3: Model Architecture ====================

class CrossAttentionFusion(nn.Module):
    """Bidirectional cross-attention between text and image modalities"""
    def __init__(self, dim, num_heads=8, dropout=0.1):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        assert self.head_dim * num_heads == dim, "dim must be divisible by num_heads"
        
        # Text queries attend to Image keys/values
        self.text_to_image_attn = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        # Image queries attend to Text keys/values
        self.image_to_text_attn = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        # Layer norms
        self.text_norm = nn.LayerNorm(dim)
        self.image_norm = nn.LayerNorm(dim)
        
        # Feed-forward networks
        self.text_ffn = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 4, dim),
            nn.Dropout(dropout)
        )
        
        self.image_ffn = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 4, dim),
            nn.Dropout(dropout)
        )
        
        self.ffn_norm_text = nn.LayerNorm(dim)
        self.ffn_norm_image = nn.LayerNorm(dim)
        
    def forward(self, text_emb, image_emb):
        # Add sequence dimension for attention (batch, 1, dim)
        text_seq = text_emb.unsqueeze(1)
        image_seq = image_emb.unsqueeze(1)
        
        # Text attends to Image
        text_attended, _ = self.text_to_image_attn(
            query=text_seq,
            key=image_seq,
            value=image_seq
        )
        text_attended = text_attended.squeeze(1)
        text_out = self.text_norm(text_emb + text_attended)
        
        # Image attends to Text
        image_attended, _ = self.image_to_text_attn(
            query=image_seq,
            key=text_seq,
            value=text_seq
        )
        image_attended = image_attended.squeeze(1)
        image_out = self.image_norm(image_emb + image_attended)
        
        # Feed-forward with residual
        text_ffn_out = self.text_ffn(text_out)
        text_final = self.ffn_norm_text(text_out + text_ffn_out)
        
        image_ffn_out = self.image_ffn(image_out)
        image_final = self.ffn_norm_image(image_out + image_ffn_out)
        
        return text_final, image_final


class EnhancedFusionModel(nn.Module):
    """Multi-modal fusion model with Cross-Attention"""
    
    def __init__(self, cfg: dict):
        super().__init__()
        self.cfg = cfg
        
        print(f"Loading text model: {cfg['TEXT_MODEL']}")
        self.text_model = AutoModel.from_pretrained(cfg['TEXT_MODEL'])
        
        print(f"Loading image model: {cfg['IMAGE_MODEL']}")
        clip_model = CLIPModel.from_pretrained(cfg['IMAGE_MODEL'])
        self.image_model = clip_model.vision_model
        
        # Freeze encoders initially
        self._freeze_encoders()
        
        # Projection layers to common dimension
        self.text_proj = nn.Linear(cfg['TEXT_DIM'], cfg['FUSION_DIM'])
        self.image_proj = nn.Linear(cfg['IMAGE_DIM'], cfg['FUSION_DIM'])
        
        # Initialize projections
        nn.init.xavier_uniform_(self.text_proj.weight)
        nn.init.zeros_(self.text_proj.bias)
        nn.init.xavier_uniform_(self.image_proj.weight)
        nn.init.zeros_(self.image_proj.bias)
        
        # Cross-Attention Fusion
        self.cross_attention = CrossAttentionFusion(
            dim=cfg['FUSION_DIM'],
            num_heads=8,
            dropout=0.1
        )
        
        # Final fusion MLP
        fusion_input_dim = cfg['FUSION_DIM'] * 2
        self.fusion_norm = nn.LayerNorm(fusion_input_dim)
        self.fusion_mlp = nn.Sequential(
            nn.Linear(fusion_input_dim, 512),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(512, cfg['FUSION_OUT_DIM']),
            nn.LayerNorm(cfg['FUSION_OUT_DIM'])
        )
        
        # Task-specific heads
        self.sentiment_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, cfg['NUM_SENTIMENT_CLASSES'])
        )
        
        self.emotion_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, cfg['NUM_EMOTION_CLASSES'])
        )
        
        self.intensity_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )
        
        print("✓ Enhanced model with Cross-Attention initialized")
        
    def _freeze_encoders(self):
        """Freeze encoder parameters"""
        for param in self.text_model.parameters():
            param.requires_grad = False
        for param in self.image_model.parameters():
            param.requires_grad = False
        print("✓ Encoders frozen")
            
    def unfreeze_backbone(self, layers_to_unfreeze: int = 2):
        """Unfreeze top layers of encoders"""
        if hasattr(self.text_model, 'encoder') and hasattr(self.text_model.encoder, 'layer'):
            text_layers = list(self.text_model.encoder.layer[-layers_to_unfreeze:])
            for layer in text_layers:
                for param in layer.parameters():
                    param.requires_grad = True
            print(f"✓ Unfroze {layers_to_unfreeze} text encoder layers")
        
        if hasattr(self.image_model, 'encoder') and hasattr(self.image_model.encoder, 'layers'):
            image_layers = list(self.image_model.encoder.layers[-layers_to_unfreeze:])
            for layer in image_layers:
                for param in layer.parameters():
                    param.requires_grad = True
            print(f"✓ Unfroze {layers_to_unfreeze} image encoder layers")
    
    def pool_text(self, model_output, attention_mask):
        """Pool text embeddings"""
        last_hidden = model_output.last_hidden_state
        
        if self.cfg['POOLING'] == 'cls':
            return last_hidden[:, 0]
        elif self.cfg['POOLING'] == 'mean':
            mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
            sum_embeddings = torch.sum(last_hidden * mask_expanded, 1)
            sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
            return sum_embeddings / sum_mask
        else:
            return last_hidden[:, 0]
    
    def forward(self, input_ids, attention_mask, image):
        """Forward pass with cross-attention fusion"""
        # Text encoding
        text_output = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_emb = self.pool_text(text_output, attention_mask)
        
        # Image encoding
        image_output = self.image_model(pixel_values=image)
        image_emb = image_output.pooler_output
        
        # Project to common dimension
        text_proj = self.text_proj(text_emb)
        image_proj = self.image_proj(image_emb)
        
        # Cross-Attention Fusion
        text_cross, image_cross = self.cross_attention(text_proj, image_proj)
        
        # Concatenate and fuse
        fused = torch.cat([text_cross, image_cross], dim=1)
        fused = self.fusion_norm(fused)
        fused = self.fusion_mlp(fused)
        
        # Task predictions
        sentiment_logits = self.sentiment_head(fused)
        emotion_logits = self.emotion_head(fused)
        intensity = self.intensity_head(fused).squeeze(-1)
        
        return {
            'sentiment_logits': sentiment_logits,
            'emotion_logits': emotion_logits,
            'intensity': intensity,
            'text_features': text_cross,
            'image_features': image_cross
        }


# ==================== Loss Functions ====================

def sentiment_loss_fn(logits, labels, cfg):
    """Sentiment classification loss with label smoothing"""
    label_smoothing = cfg.get('LABEL_SMOOTHING', 0.0)
    return F.cross_entropy(logits, labels, label_smoothing=label_smoothing)

def emotion_loss_fn(logits, labels, cfg):
    """Emotion classification loss (multi-label)"""
    return F.binary_cross_entropy_with_logits(logits, labels)

def intensity_loss_fn(preds, targets, cfg):
    """Intensity regression loss"""
    return F.smooth_l1_loss(preds, targets)

def combined_loss(outputs, batch, cfg):
    """Combined multi-task loss"""
    loss_sent = sentiment_loss_fn(outputs['sentiment_logits'], batch['sentiment_label'], cfg)
    loss_emotion = emotion_loss_fn(outputs['emotion_logits'], batch['emotion_labels'], cfg)
    loss_intensity = intensity_loss_fn(outputs['intensity'], batch['intensity'], cfg)
    
    total_loss = (cfg['LOSS_WEIGHTS']['sentiment'] * loss_sent +
                  cfg['LOSS_WEIGHTS']['emotion'] * loss_emotion +
                  cfg['LOSS_WEIGHTS']['intensity'] * loss_intensity)
    
    return total_loss, loss_sent, loss_emotion, loss_intensity


# ==================== Initialize and Test Model ====================

try:
    print("\nInitializing Enhanced Model with Cross-Attention...")
    model = EnhancedFusionModel(cfg).to(device)
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\n{'='*70}")
    print("Model Summary:")
    print(f"  Total parameters: {total_params:,}")
    print(f"  Trainable parameters: {trainable_params:,}")
    print(f"  Frozen parameters: {total_params - trainable_params:,}")
    print(f"{'='*70}")
    
    print("\nTesting forward pass with dummy data...")
    dummy_input_ids = torch.randint(0, 1000, (2, cfg['MAX_LEN'])).to(device)
    dummy_attention_mask = torch.ones(2, cfg['MAX_LEN']).to(device)
    dummy_image = torch.randn(2, 3, cfg['IMG_SIZE'], cfg['IMG_SIZE']).to(device)
    
    with torch.no_grad():
        test_output = model(dummy_input_ids, dummy_attention_mask, dummy_image)
    
    print(f"✓ Forward pass successful!")
    print(f"  Sentiment logits shape: {test_output['sentiment_logits'].shape}")
    print(f"  Emotion logits shape: {test_output['emotion_logits'].shape}")
    print(f"  Intensity shape: {test_output['intensity'].shape}")
    print(f"  Text features shape: {test_output['text_features'].shape}")
    print(f"  Image features shape: {test_output['image_features'].shape}")
    
    print("\n" + "="*70)
    print("Part 3 Complete - Enhanced Model initialized and tested")
    print("="*70)
    
except Exception as e:
    print(f"\n✗ Error in model initialization: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# ==================== Part 4: DataLoader Creation and Testing ====================

LABELS = ["humor", "sarcasm", "offensive", "motivational"]

try:
    print("Creating tokenizer and transforms...")
    tokenizer = get_tokenizer(cfg)
    train_transform = get_image_transform(cfg, is_train=True)
    val_transform = get_image_transform(cfg, is_train=False)
    
    print("\nCreating datasets...")
    train_dataset = MemeDataset(train_df, tokenizer, train_transform, train_img_dir, cfg)
    val_dataset = MemeDataset(val_df, tokenizer, val_transform, val_img_dir, cfg)
    
    print("\nCreating dataloaders...")
    num_workers = 0 if device.type == 'cpu' else 2
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg['BATCH_SIZE'],
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=collate_fn,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=cfg['BATCH_SIZE'],
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=collate_fn
    )
    
    print(f"✓ Train batches: {len(train_loader)}")
    print(f"✓ Val batches: {len(val_loader)}")
    
    # ==================== Test DataLoader ====================
    
    print("\nTesting dataloader with real batch...")
    test_batch = next(iter(train_loader))
    
    print(f"  Input IDs shape: {test_batch['input_ids'].shape}")
    print(f"  Attention mask shape: {test_batch['attention_mask'].shape}")
    print(f"  Image shape: {test_batch['image'].shape}")
    print(f"  Sentiment labels shape: {test_batch['sentiment_label'].shape}")
    print(f"  Emotion labels shape: {test_batch['emotion_labels'].shape}")
    print(f"  Intensity shape: {test_batch['intensity'].shape}")
    
    print("\nSample sentiment labels:", test_batch['sentiment_label'][:5])
    print("Sample emotion labels:", test_batch['emotion_labels'][:2])
    print("Sample intensity values:", test_batch['intensity'][:5])
    
    # ==================== Test Model with Real Batch ====================
    
    print("\nTesting model forward pass with real batch...")
    model.eval()
    
    with torch.no_grad():
        batch_input_ids = test_batch['input_ids'][:2].to(device)
        batch_attention_mask = test_batch['attention_mask'][:2].to(device)
        batch_image = test_batch['image'][:2].to(device)
        
        real_output = model(batch_input_ids, batch_attention_mask, batch_image)
    
    print(f"✓ Real batch forward pass successful!")
    print(f"  Sentiment logits: {real_output['sentiment_logits'].shape}")
    print(f"  Emotion logits: {real_output['emotion_logits'].shape}")
    print(f"  Intensity: {real_output['intensity'].shape}")
    
    # Test loss computation
    print("\nTesting loss computation...")
    batch_device = {
        'sentiment_label': test_batch['sentiment_label'][:2].to(device),
        'emotion_labels': test_batch['emotion_labels'][:2].to(device),
        'intensity': test_batch['intensity'][:2].to(device)
    }
    
    total_loss, loss_sent, loss_emotion, loss_int = combined_loss(real_output, batch_device, cfg)
    
    print(f"✓ Loss computation successful!")
    print(f"  Total loss: {total_loss.item():.4f}")
    print(f"  Sentiment loss: {loss_sent.item():.4f}")
    print(f"  Emotion loss: {loss_emotion.item():.4f}")
    print(f"  Intensity loss: {loss_int.item():.4f}")
    
    # ==================== DIAGNOSTIC 4: Dataset Imbalance Stats ====================
    
    print("\n" + "="*70)
    print("DIAGNOSTIC: Dataset Imbalance Statistics")
    print("="*70)
    
    def label_stats(loader):
        cnt = np.zeros(len(LABELS), dtype=np.int64)
        total = 0
        for batch in loader:
            y = batch['emotion_labels'].numpy()
            cnt += y.sum(axis=0).astype(int)
            total += y.shape[0]
        print("Total samples:", total)
        for i, n in enumerate(LABELS):
            pos = cnt[i]
            print(f"{n}: pos={pos}, pos_rate={pos/total:.4f}, neg={total-pos}")
        return cnt, total
    
    pos_counts, N = label_stats(train_loader)
    neg_counts = N - pos_counts
    pos_weight_np = (neg_counts / np.maximum(1, pos_counts)).clip(max=50.0)
    print("\nSuggested pos_weight (for BCE/Focal BCE):", pos_weight_np)
    print("Note: Current model uses BCE without pos_weight. Consider adding if imbalance is severe.")
    
    print("\n" + "="*70)
    print("Part 4 Complete - DataLoaders created and tested successfully")
    print("="*70)
    
except Exception as e:
    print(f"\n✗ Error in dataloader creation: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# ==================== Part 5: Metrics and Evaluation Functions ====================

def compute_sentiment_metrics(y_true, y_pred_logits):
    """Compute sentiment classification metrics"""
    y_pred = torch.argmax(y_pred_logits, dim=1).cpu().numpy()
    y_true = y_true.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    
    return {
        'sentiment_accuracy': accuracy,
        'sentiment_precision': precision,
        'sentiment_recall': recall,
        'sentiment_f1': f1
    }

def compute_emotion_metrics(y_true, y_pred_logits):
    """Compute emotion classification metrics (multi-label)"""
    y_pred = (torch.sigmoid(y_pred_logits) > 0.5).float().cpu().numpy()
    y_true = y_true.cpu().numpy()
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='samples', zero_division=0
    )
    
    return {
        'emotion_precision': precision,
        'emotion_recall': recall,
        'emotion_f1': f1
    }

def compute_intensity_metrics(y_true, y_pred):
    """Compute intensity regression metrics"""
    y_true = y_true.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    try:
        spearman, _ = spearmanr(y_true, y_pred)
        if np.isnan(spearman):
            spearman = 0.0
    except:
        spearman = 0.0
    
    return {
        'intensity_rmse': rmse,
        'intensity_mae': mae,
        'intensity_spearman': spearman
    }

# ==================== DIAGNOSTIC 2: Confusion Matrix & Per-Label Report ====================

def eval_multilabel(y_true_tensor, logits_tensor, thresholds=None):
    """Evaluate multi-label classification with per-label confusion matrix"""
    probs = torch.sigmoid(logits_tensor).detach().cpu().numpy()
    y_true = y_true_tensor.detach().cpu().numpy().astype(int)
    C = probs.shape[1]
    if thresholds is None:
        thresholds = np.array([0.5]*C)
    y_pred = (probs >= thresholds).astype(int)

    for i, name in enumerate(LABELS):
        tn, fp, fn, tp = confusion_matrix(y_true[:,i], y_pred[:,i], labels=[0,1]).ravel()
        ap = average_precision_score(y_true[:,i], probs[:,i])
        print(f"\n=== {name.upper()} ===")
        print(f"TP={tp} FP={fp} FN={fn} TN={tn} | AUPRC={ap:.4f}")
        print(classification_report(y_true[:,i], y_pred[:,i], digits=4))
    return y_pred, probs

# ==================== DIAGNOSTIC 3: Threshold Tuning ====================

def tune_thresholds(y_true_tensor, logits_tensor, grid=None):
    """Tune thresholds for each emotion label to maximize F1"""
    if grid is None:
        grid = np.linspace(0.05, 0.9, 18)
    probs = torch.sigmoid(logits_tensor).detach().cpu().numpy()
    y_true = y_true_tensor.detach().cpu().numpy().astype(int)
    C = probs.shape[1]
    best = [0.5]*C
    for c, name in enumerate(LABELS):
        fbest, tbest = -1, 0.5
        for t in grid:
            pred = (probs[:,c] >= t).astype(int)
            f = f1_score(y_true[:,c], pred, zero_division=0)
            if f > fbest:
                fbest, tbest = f, t
        best[c] = tbest
        print(f"{name}: best_thr={tbest:.2f} | best_F1={fbest:.4f}")
    return np.array(best)

# ==================== Test Metrics ====================

try:
    print("\n" + "="*70)
    print("Testing metrics computation...")
    print("="*70)
    
    # Create dummy predictions
    dummy_sentiment_logits = torch.randn(10, cfg['NUM_SENTIMENT_CLASSES'])
    dummy_sentiment_labels = torch.randint(0, cfg['NUM_SENTIMENT_CLASSES'], (10,))
    
    dummy_emotion_logits = torch.randn(10, cfg['NUM_EMOTION_CLASSES'])
    dummy_emotion_labels = torch.randint(0, 2, (10, cfg['NUM_EMOTION_CLASSES'])).float()
    
    dummy_intensity_pred = torch.rand(10)
    dummy_intensity_true = torch.rand(10)
    
    # Compute metrics
    sentiment_metrics = compute_sentiment_metrics(dummy_sentiment_labels, dummy_sentiment_logits)
    emotion_metrics = compute_emotion_metrics(dummy_emotion_labels, dummy_emotion_logits)
    intensity_metrics = compute_intensity_metrics(dummy_intensity_true, dummy_intensity_pred)
    
    print("\n✓ Sentiment Metrics:")
    for key, value in sentiment_metrics.items():
        print(f"    {key}: {value:.4f}")
    
    print("\n✓ Emotion Metrics:")
    for key, value in emotion_metrics.items():
        print(f"    {key}: {value:.4f}")
    
    print("\n✓ Intensity Metrics:")
    for key, value in intensity_metrics.items():
        print(f"    {key}: {value:.4f}")
    
    print("\n" + "="*70)
    print("Part 5 Complete - Metrics tested successfully")
    print("="*70)
    
except Exception as e:
    print(f"\n✗ Error in metrics computation: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# ==================== Part 6: Trainer Class ====================

class Trainer:
    """Trainer for multi-task meme analysis"""
    
    def __init__(self, model, cfg, train_loader, val_loader, device):
        self.model = model
        self.cfg = cfg
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        
        self.optimizer = self.make_optimizer()
        self.scheduler = self.make_scheduler()
        self.scaler = GradScaler() if cfg['USE_AMP'] else None
        
        self.current_epoch = 0
        self.best_metric = -float('inf')
        self.gradient_accumulation_steps = cfg.get('GRADIENT_ACCUMULATION_STEPS', 1)
        
        # Store validation outputs for diagnostics
        self.val_emotion_logits = None
        self.val_emotion_labels = None
        
        print("✓ Trainer initialized")
        
    def make_optimizer(self):
        """Create optimizer with parameter groups"""
        head_params = []
        backbone_params = []
        
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                if 'text_model' in name or 'image_model' in name:
                    backbone_params.append(param)
                else:
                    head_params.append(param)
        
        param_groups = [
            {'params': head_params, 'lr': self.cfg['LR_HEADS']},
        ]
        
        if backbone_params:
            param_groups.append({'params': backbone_params, 'lr': self.cfg['LR_BACKBONE']})
        
        print(f"  Optimizer: {len(head_params)} head params, {len(backbone_params)} backbone params")
        return torch.optim.AdamW(param_groups, weight_decay=self.cfg['WEIGHT_DECAY'])
    
    def make_scheduler(self):
        """Create learning rate scheduler"""
        if self.cfg['SCHEDULER'] == 'cosine':
            return torch.optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer, 
                T_max=self.cfg['EPOCHS']
            )
        return None
    
    def train_epoch(self, epoch):
        """Train for one epoch"""
        self.model.train()
        total_loss = 0
        total_sent_loss = 0
        total_emotion_loss = 0
        total_intensity_loss = 0
        
        pbar = tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.cfg['EPOCHS']} [Train]")
        
        self.optimizer.zero_grad()
        
        for batch_idx, batch in enumerate(pbar):
            # Move to device
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            image = batch['image'].to(self.device)
            sentiment_label = batch['sentiment_label'].to(self.device)
            emotion_labels = batch['emotion_labels'].to(self.device)
            intensity = batch['intensity'].to(self.device)
            
            batch_device = {
                'sentiment_label': sentiment_label,
                'emotion_labels': emotion_labels,
                'intensity': intensity
            }
            
            # Forward pass with AMP
            if self.cfg['USE_AMP']:
                with autocast():
                    outputs = self.model(input_ids, attention_mask, image)
                    loss, l_sent, l_emotion, l_int = combined_loss(outputs, batch_device, self.cfg)
                    loss = loss / self.gradient_accumulation_steps
                
                self.scaler.scale(loss).backward()
                
                if (batch_idx + 1) % self.gradient_accumulation_steps == 0:
                    if self.cfg['GRADIENT_CLIP'] > 0:
                        self.scaler.unscale_(self.optimizer)
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.cfg['GRADIENT_CLIP'])
                    
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                    self.optimizer.zero_grad()
            else:
                outputs = self.model(input_ids, attention_mask, image)
                loss, l_sent, l_emotion, l_int = combined_loss(outputs, batch_device, self.cfg)
                loss = loss / self.gradient_accumulation_steps
                loss.backward()
                
                if (batch_idx + 1) % self.gradient_accumulation_steps == 0:
                    if self.cfg['GRADIENT_CLIP'] > 0:
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.cfg['GRADIENT_CLIP'])
                    
                    self.optimizer.step()
                    self.optimizer.zero_grad()
            
            total_loss += loss.item() * self.gradient_accumulation_steps
            total_sent_loss += l_sent.item()
            total_emotion_loss += l_emotion.item()
            total_intensity_loss += l_int.item()
            
            pbar.set_postfix({
                'loss': f"{loss.item() * self.gradient_accumulation_steps:.4f}",
                'sent': f"{l_sent.item():.4f}",
                'emo': f"{l_emotion.item():.4f}",
                'int': f"{l_int.item():.4f}"
            })
        
        avg_loss = total_loss / len(self.train_loader)
        print(f"\nTrain Loss: {avg_loss:.4f}")
        
        return avg_loss
    
    def validate(self, epoch):
        """Validate the model"""
        self.model.eval()
        total_loss = 0
        
        all_sentiment_labels = []
        all_sentiment_logits = []
        all_emotion_labels = []
        all_emotion_logits = []
        all_intensity_true = []
        all_intensity_pred = []
        
        pbar = tqdm(self.val_loader, desc=f"Epoch {epoch+1}/{self.cfg['EPOCHS']} [Val]")
        
        with torch.no_grad():
            for batch in pbar:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                image = batch['image'].to(self.device)
                sentiment_label = batch['sentiment_label'].to(self.device)
                emotion_labels = batch['emotion_labels'].to(self.device)
                intensity = batch['intensity'].to(self.device)
                
                batch_device = {
                    'sentiment_label': sentiment_label,
                    'emotion_labels': emotion_labels,
                    'intensity': intensity
                }
                
                outputs = self.model(input_ids, attention_mask, image)
                loss, _, _, _ = combined_loss(outputs, batch_device, self.cfg)
                
                total_loss += loss.item()
                
                all_sentiment_labels.append(sentiment_label)
                all_sentiment_logits.append(outputs['sentiment_logits'])
                all_emotion_labels.append(emotion_labels)
                all_emotion_logits.append(outputs['emotion_logits'])
                all_intensity_true.append(intensity)
                all_intensity_pred.append(outputs['intensity'])
        
        # Aggregate metrics
        all_sentiment_labels = torch.cat(all_sentiment_labels)
        all_sentiment_logits = torch.cat(all_sentiment_logits)
        all_emotion_labels = torch.cat(all_emotion_labels)
        all_emotion_logits = torch.cat(all_emotion_logits)
        all_intensity_true = torch.cat(all_intensity_true)
        all_intensity_pred = torch.cat(all_intensity_pred)
        
        # Store for diagnostics
        self.val_emotion_logits = all_emotion_logits
        self.val_emotion_labels = all_emotion_labels
        
        sentiment_metrics = compute_sentiment_metrics(all_sentiment_labels, all_sentiment_logits)
        emotion_metrics = compute_emotion_metrics(all_emotion_labels, all_emotion_logits)
        intensity_metrics = compute_intensity_metrics(all_intensity_true, all_intensity_pred)
        
        avg_loss = total_loss / len(self.val_loader)
        
        metrics = {
            'val_loss': avg_loss,
            **sentiment_metrics,
            **emotion_metrics,
            **intensity_metrics
        }
        
        print(f"\nValidation Metrics:")
        print(f"  Loss: {avg_loss:.4f}")
        print(f"  Sentiment F1: {sentiment_metrics['sentiment_f1']:.4f}")
        print(f"  Emotion F1: {emotion_metrics['emotion_f1']:.4f}")
        print(f"  Intensity RMSE: {intensity_metrics['intensity_rmse']:.4f}")
        
        return metrics
    
    def run_diagnostics(self, epoch):
        """Run diagnostic checks after validation"""
        if self.val_emotion_logits is None or self.val_emotion_labels is None:
            return
        
        print(f"\n{'='*70}")
        print(f"DIAGNOSTIC: Per-Label Analysis (Epoch {epoch+1})")
        print(f"{'='*70}")
        
        # Per-label confusion matrix
        _ = eval_multilabel(self.val_emotion_labels, self.val_emotion_logits, 
                           thresholds=np.array(self.cfg.get('EMO_THRESHOLDS', [0.5,0.5,0.5,0.5])))
        
        # Threshold tuning
        print(f"\n{'='*70}")
        print("DIAGNOSTIC: Threshold Tuning")
        print(f"{'='*70}")
        best_thr = tune_thresholds(self.val_emotion_labels, self.val_emotion_logits)
        self.cfg['EMO_THRESHOLDS'] = best_thr.tolist()
        print(f"\nUpdated cfg thresholds: {self.cfg['EMO_THRESHOLDS']}")
        
        # Re-run confusion with tuned thresholds
        print(f"\n{'='*70}")
        print("DIAGNOSTIC: Results with Tuned Thresholds")
        print(f"{'='*70}")
        _ = eval_multilabel(self.val_emotion_labels, self.val_emotion_logits, thresholds=best_thr)
    
    def fit(self):
        """Train the model"""
        print(f"\n{'='*70}")
        print(f"Starting training for {self.cfg['EPOCHS']} epochs")
        print(f"{'='*70}\n")
        
        for epoch in range(self.cfg['EPOCHS']):
            self.current_epoch = epoch
            
            # Unfreeze backbone at specific epoch
            if epoch == self.cfg.get('UNFREEZE_BACKBONE_EPOCH', -1):
                print(f"\n{'='*50}")
                print(f"Unfreezing backbone at epoch {epoch+1}")
                print(f"{'='*50}\n")
                self.model.unfreeze_backbone(layers_to_unfreeze=2)
                self.optimizer = self.make_optimizer()
                self.scheduler = self.make_scheduler()
            
            # Train
            train_loss = self.train_epoch(epoch)
            
            # Validate
            val_metrics = self.validate(epoch)
            
            # Run diagnostics (every 2 epochs or last epoch)
            if epoch % 2 == 0 or epoch == self.cfg['EPOCHS'] - 1:
                self.run_diagnostics(epoch)
            
            # Scheduler step
            if self.scheduler:
                self.scheduler.step()
            
            # Composite metric for checkpoint
            composite_metric = (val_metrics['sentiment_f1'] + 
                              val_metrics['emotion_f1'] - 
                              val_metrics['intensity_rmse'])
            
            # Save best checkpoint
            if composite_metric > self.best_metric:
                self.best_metric = composite_metric
                checkpoint_path = os.path.join(self.cfg['CHECKPOINT_PATH'], 'best_model.pt')
                save_checkpoint({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'best_metric': self.best_metric,
                    'metrics': val_metrics,
                    'config': self.cfg
                }, checkpoint_path)
                print(f"✓ Saved best model (composite: {composite_metric:.4f})")
            
            print(f"\n{'='*70}\n")
        
        print("Training completed!")
        return self.best_metric

# ==================== Initialize Trainer ====================

try:
    print("\nInitializing trainer...")
    trainer = Trainer(model, cfg, train_loader, val_loader, device)
    
    print("\n" + "="*70)
    print("Part 6 Complete - Trainer ready")
    print("Run Part 7 to start training")
    print("="*70)
    
except Exception as e:
    print(f"\n✗ Error initializing trainer: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# ==================== Part 7: Start Training (with Diagnostic 1) ====================

print("\n" + "="*70)
print("Part 7: Training with Diagnostics")
print("="*70)

# ==================== DIAGNOSTIC 1: Tiny-batch Overfit & Gradient Sanity ====================

print("\n" + "="*70)
print("DIAGNOSTIC 1: Tiny-batch Overfit & Gradient Sanity Check")
print("="*70)

tiny_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

model.train()
opt = torch.optim.AdamW(model.parameters(), lr=1e-4)

print("\nRunning 200 steps on tiny batch to verify model can overfit...")
for step, batch in enumerate(tiny_loader):
    if step > 200: break
    
    # Move batch to device
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    image = batch['image'].to(device)
    
    batch_device = {
        'sentiment_label': batch['sentiment_label'].to(device),
        'emotion_labels': batch['emotion_labels'].to(device),
        'intensity': batch['intensity'].to(device)
    }
    
    outputs = model(input_ids, attention_mask, image)
    total, l_sent, l_emo, l_int = combined_loss(outputs, batch_device, cfg)

    opt.zero_grad(set_to_none=True)
    total.backward()

    # gradient NaN/inf check
    bad_grad = False
    for p in model.parameters():
        if p.grad is not None:
            if torch.isnan(p.grad).any() or torch.isinf(p.grad).any():
                bad_grad = True
                break
    if bad_grad:
        print("✗ Bad gradient detected (NaN/Inf). Stop & inspect loss/labels.")
        break

    opt.step()
    if step % 20 == 0:
        print(f"{step:03d}: total={total.item():.4f} | sent={l_sent.item():.4f} | emo={l_emo.item():.4f} | int={l_int.item():.4f}")

print("✓ Tiny-batch sanity done. Model can learn from data.")
print("Emotion loss should have decreased. If not, check labels/loss function.")

# Reinitialize model and trainer for actual training
print("\nReinitializing model for full training...")
model = EnhancedFusionModel(cfg).to(device)
trainer = Trainer(model, cfg, train_loader, val_loader, device)

# ==================== Start Full Training ====================

try:
    print("\n" + "="*70)
    print("STARTING FULL TRAINING")
    print("="*70)
    
    best_metric = trainer.fit()
    
    print(f"\n{'='*70}")
    print(f"Training Completed!")
    print(f"Best composite metric: {best_metric:.4f}")
    print(f"Best model saved to: {cfg['CHECKPOINT_PATH']}/best_model.pt")
    print(f"{'='*70}")
    
    # Save model card
    model_card = f"""# Meme Sentiment and Emotion Analysis Model

## Model Details
- **Text Encoder**: {cfg['TEXT_MODEL']}
- **Image Encoder**: {cfg['IMAGE_MODEL']}
- **Architecture**: Multi-modal fusion with cross-attention
- **Tasks**: 
  - Sentiment Classification (5 classes)
  - Emotion Classification (4 binary classes: humor, sarcasm, offensive, motivational)
  - Intensity Regression (continuous 0-1)

## Training Configuration
- Epochs: {cfg['EPOCHS']}
- Batch Size: {cfg['BATCH_SIZE']}
- Gradient Accumulation: {cfg.get('GRADIENT_ACCUMULATION_STEPS', 1)}
- Learning Rate (Heads): {cfg['LR_HEADS']}
- Learning Rate (Backbone): {cfg['LR_BACKBONE']}
- Mixed Precision: {cfg['USE_AMP']}

## Performance
- Best Composite Metric: {best_metric:.4f}

## Dataset
- Training Samples: {len(train_df)}
- Validation Samples: {len(val_df)}

## Tuned Emotion Thresholds
{cfg['EMO_THRESHOLDS']}
"""
    
    model_card_path = os.path.join(cfg['CHECKPOINT_PATH'], 'model_card.md')
    with open(model_card_path, 'w') as f:
        f.write(model_card)
    print(f"\n✓ Model card saved to: {model_card_path}")
    
    print("\n" + "="*70)
    print("Part 7 Complete - Training finished")
    print("="*70)
    
except KeyboardInterrupt:
    print("\n\n⚠ Training interrupted by user")
    
except Exception as e:
    print(f"\n✗ Error during training: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# ==================== Part 8: Inference on New Data ====================

class Predictor:
    """Class to perform inference using the trained model"""

    def __init__(self, checkpoint_path: str, device: str = 'cuda'):
        print("Initializing predictor...")
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        
        # Load checkpoint and config
        checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
        self.cfg = checkpoint['config']
        
        # Load model
        try:
            print("  Loading EnhancedFusionModel...")
            self.model = EnhancedFusionModel(self.cfg).to(self.device)
            print("  ✓ Using EnhancedFusionModel")
        except NameError:
            raise RuntimeError("EnhancedFusionModel not found! Run Part 3 first.")
        
        # Load model weights
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        print(f"✓ Model loaded from epoch {checkpoint['epoch']} with best metric {checkpoint['best_metric']:.4f}")
        
        # Load tokenizer and image transform
        self.tokenizer = get_tokenizer(self.cfg)
        self.transform = get_image_transform(self.cfg, is_train=False)
        
        # Create reverse mappings
        self.sentiment_map_rev = {
            0: 'very_positive',
            1: 'positive', 
            2: 'neutral',
            3: 'negative',
            4: 'very_negative'
        }
        self.emotion_labels = ['humor', 'sarcasm', 'offensive', 'motivational']
        
        print("✓ Predictor initialized successfully")

    def predict(self, text: str, image_path: str) -> Dict:
        """Predict sentiment, emotions, and intensity for a single text-image pair"""
        
        # Preprocess Text
        encoding = self.tokenizer(
            text,
            max_length=self.cfg['MAX_LEN'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)

        # Preprocess Image
        try:
            image = Image.open(image_path).convert('RGB')
            image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        except Exception as e:
            print(f"Could not load image: {e}. Using blank image.")
            image_tensor = torch.zeros(1, 3, self.cfg['IMG_SIZE'], self.cfg['IMG_SIZE']).to(self.device)

        # Run Inference
        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask, image_tensor)
        
        # Post-process Outputs
        sentiment_pred_idx = torch.argmax(outputs['sentiment_logits'], dim=1).item()
        sentiment_label = self.sentiment_map_rev.get(sentiment_pred_idx, "unknown")
        sentiment_probs = torch.softmax(outputs['sentiment_logits'], dim=1).squeeze().cpu().numpy()
        
        emotion_probs = torch.sigmoid(outputs['emotion_logits']).squeeze().cpu().numpy()
        emotion_preds = {label: f"{prob:.2%}" for label, prob in zip(self.emotion_labels, emotion_probs)}
        
        intensity_score = outputs['intensity'].item()
        
        return {
            'sentiment': sentiment_label,
            'sentiment_confidence': f"{sentiment_probs[sentiment_pred_idx]:.2%}",
            'sentiment_probs': {self.sentiment_map_rev[i]: f"{p:.2%}" for i, p in enumerate(sentiment_probs)},
            'emotions': emotion_preds,
            'intensity_score': f"{intensity_score:.4f}"
        }
    
    def predict_batch(self, text_list, image_path_list):
        """Predict for multiple samples at once"""
        results = []
        for text, img_path in zip(text_list, image_path_list):
            result = self.predict(text, img_path)
            results.append(result)
        return results


# ==================== Run Inference Example ====================

try:
    CHECKPOINT_DIR = "/kaggle/working/checkpoints"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    BEST_MODEL_PATH = os.path.join(CHECKPOINT_DIR, 'best_model.pt')
    
    if not os.path.exists(BEST_MODEL_PATH):
        print(f"✗ Checkpoint not found at {BEST_MODEL_PATH}")
        print("  Please train the model first (run Part 7)")
        raise FileNotFoundError(f"Model checkpoint not found: {BEST_MODEL_PATH}")
    
    print("\n" + "="*70)
    print("INITIALIZING PREDICTOR")
    print("="*70)
    
    predictor = Predictor(checkpoint_path=BEST_MODEL_PATH, device=DEVICE)
    
    print("\n" + "="*70)
    print("RUNNING SAMPLE PREDICTIONS")
    print("="*70)
    
    # Get column names
    image_col_options = ['image_name', 'image', 'img_name', 'filename', 'Unnamed: 0']
    image_col = None
    for col in image_col_options:
        if col in val_df.columns:
            image_col = col
            break
    if image_col is None:
        image_col = val_df.columns[0]
    
    text_col_options = ['text', 'ocr', 'ocr_text', 'caption', 'OCR_extracted_text']
    text_col = None
    for col in text_col_options:
        if col in val_df.columns:
            text_col = col
            break
    
    # Select 10 random samples
    import random
    random.seed(42)
    total_samples = len(val_df)
    sample_indices = random.sample(range(total_samples), min(10, total_samples))
    sample_indices.sort()
    
    print(f"\nSelected {len(sample_indices)} samples: {sample_indices}")
    
    for i, idx in enumerate(sample_indices, 1):
        if idx >= len(val_df):
            continue
            
        sample_row = val_df.iloc[idx]
        sample_text = str(sample_row.get(text_col, '')) if text_col else ''
        sample_image_name = str(sample_row.get(image_col, f'{idx}.jpg'))
        
        if not any(sample_image_name.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
            sample_image_name = f"{sample_image_name}.jpg"
            
        sample_image_path = os.path.join(val_img_dir, sample_image_name)
        
        print(f"\n{'='*70}")
        print(f"Sample {idx}:")
        print(f"  Text: '{sample_text[:100]}{'...' if len(sample_text) > 100 else ''}'")
        print(f"  Image: {sample_image_name}")
        print("="*70)
        
        if not os.path.exists(sample_image_path):
            print(f"⚠️  Image not found at: {sample_image_path}")
            continue
        
        # Make prediction
        prediction_result = predictor.predict(text=sample_text, image_path=sample_image_path)
        
        print("\nPrediction Result:")
        print(f"  Sentiment: {prediction_result['sentiment']} (confidence: {prediction_result['sentiment_confidence']})")
        print(f"  Intensity: {prediction_result['intensity_score']}")
        print("  Emotions:")
        for emotion, prob in prediction_result['emotions'].items():
            print(f"    - {emotion.capitalize()}: {prob}")
        
        # Display the image
        try:
            import matplotlib.pyplot as plt
            fig, ax = plt.subplots(figsize=(6, 6))
            img = Image.open(sample_image_path)
            ax.imshow(img)
            
            true_sentiment = sample_row.get('overall', sample_row.get('sentiment', 'N/A'))
            
            ax.set_title(
                f"Predicted: {prediction_result['sentiment']}\n"
                f"True: {true_sentiment}",
                fontsize=12
            )
            ax.axis('off')
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"  Could not display image: {e}")
    
    print("\n" + "="*70)
    print("Part 8 Complete - Inference pipeline tested")
    print("="*70)
    
except FileNotFoundError as e:
    print(f"\n✗ {e}")
    print("   Train the model first by running Parts 1-7")
    
except Exception as e:
    print(f"\n✗ Error during inference: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# ==================== Part 9: Detailed Evaluation & Error Analysis ====================

from sklearn.metrics import ConfusionMatrixDisplay

def run_full_evaluation(predictor, val_loader):
    """Runs model on entire validation set"""
    
    predictor.model.eval()
    all_preds = []
    
    pbar = tqdm(val_loader, desc="Running Full Evaluation")
    
    with torch.no_grad():
        for batch in pbar:
            input_ids = batch['input_ids'].to(predictor.device)
            attention_mask = batch['attention_mask'].to(predictor.device)
            image = batch['image'].to(predictor.device)
            
            outputs = predictor.model(input_ids, attention_mask, image)
            
            sentiment_preds = torch.argmax(outputs['sentiment_logits'], dim=1).cpu().numpy()
            emotion_preds = (torch.sigmoid(outputs['emotion_logits']) > 0.5).int().cpu().numpy()
            intensity_preds = outputs['intensity'].cpu().numpy().flatten()
            
            sentiment_true = batch['sentiment_label'].cpu().numpy()
            emotion_true = batch['emotion_labels'].int().cpu().numpy()
            intensity_true = batch['intensity'].cpu().numpy().flatten()
            
            for i in range(len(input_ids)):
                all_preds.append({
                    'sentiment_pred': sentiment_preds[i],
                    'sentiment_true': sentiment_true[i],
                    'emotion_pred': emotion_preds[i],
                    'emotion_true': emotion_true[i],
                    'intensity_pred': intensity_preds[i],
                    'intensity_true': intensity_true[i]
                })

    results_df = pd.DataFrame(all_preds)
    results_df['sentiment_pred_label'] = results_df['sentiment_pred'].map(predictor.sentiment_map_rev)
    results_df['sentiment_true_label'] = results_df['sentiment_true'].map(predictor.sentiment_map_rev)
    
    return results_df

try:
    print("\n" + "="*70)
    print("STARTING DETAILED EVALUATION")
    print("="*70)
    
    if 'predictor' not in locals() or predictor is None:
        print("✗ Predictor not found. Please run Part 8 first.")
        raise NameError("Predictor not initialized")
    
    print("\nStarting full evaluation on validation set...")
    eval_results_df = run_full_evaluation(predictor, val_loader)
    
    # Merge with original validation dataframe
    full_analysis_df = pd.concat([val_df.reset_index(drop=True), eval_results_df], axis=1)
    full_analysis_df['sentiment_correct'] = full_analysis_df['sentiment_pred'] == full_analysis_df['sentiment_true']
    full_analysis_df['intensity_error'] = np.abs(full_analysis_df['intensity_pred'] - full_analysis_df['intensity_true'])
    
    print(f"\n✓ Full evaluation complete. Analysis DataFrame shape: {full_analysis_df.shape}")

    # ==================== Error Analysis ====================
    
    print("\n" + "="*70)
    print("Error Analysis: Top 5 Worst Sentiment Predictions")
    print("="*70)
    
    sentiment_errors = full_analysis_df[full_analysis_df['sentiment_correct'] == False].sort_values(by='intensity_error', ascending=False)
    
    if sentiment_errors.empty:
        print("No sentiment prediction errors found!")
    else:
        for i, (_, row) in enumerate(sentiment_errors.head(5).iterrows()):
            img_name = str(row.get(image_col, f'{i}.jpg'))
            if not any(img_name.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
                img_name += '.jpg'
            img_path = os.path.join(val_img_dir, img_name)
            
            text = str(row.get(text_col, 'N/A')) if text_col else 'N/A'
            
            print(f"\n{i+1}. Error Case:")
            print(f"   Text: {text[:100]}{'...' if len(text) > 100 else ''}")
            print(f"   True: {row['sentiment_true_label']} | Pred: {row['sentiment_pred_label']}")
            print(f"   Intensity Error: {row['intensity_error']:.4f}")
            
            try:
                plt.figure(figsize=(5, 5))
                plt.imshow(Image.open(img_path))
                plt.title(f"True: {row['sentiment_true_label']} | Pred: {row['sentiment_pred_label']}")
                plt.axis('off')
                plt.tight_layout()
                plt.show()
            except Exception as e:
                print(f"   (Could not display image: {e})")
    
    # ==================== Performance Summary ====================
    
    print("\n" + "="*70)
    print("PERFORMANCE SUMMARY")
    print("="*70)
    
    overall_acc = full_analysis_df['sentiment_correct'].mean()
    avg_intensity_error = full_analysis_df['intensity_error'].mean()
    
    print(f"\nOverall Metrics:")
    print(f"  Sentiment Accuracy: {overall_acc:.2%}")
    print(f"  Average Intensity Error: {avg_intensity_error:.4f}")
    
    print(f"\nPer-Class Sentiment Accuracy:")
    for sent_label in full_analysis_df['sentiment_true_label'].unique():
        class_df = full_analysis_df[full_analysis_df['sentiment_true_label'] == sent_label]
        if len(class_df) > 0:
            class_acc = class_df['sentiment_correct'].mean()
            print(f"  {sent_label}: {class_acc:.2%} ({len(class_df)} samples)")
    
    # ==================== Confusion Matrix ====================
    
    print("\n" + "="*70)
    print("CONFUSION MATRIX")
    print("="*70)
    
    sentiment_labels = sorted(full_analysis_df['sentiment_true_label'].unique())
    
    cm = confusion_matrix(
        full_analysis_df['sentiment_true_label'], 
        full_analysis_df['sentiment_pred_label'],
        labels=sentiment_labels
    )
    
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=sentiment_labels)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    disp.plot(cmap=plt.cm.Blues, ax=ax, xticks_rotation='vertical', values_format='d')
    plt.title('Sentiment Classification Confusion Matrix', fontsize=14, pad=20)
    plt.tight_layout()
    
    cm_path = '/kaggle/working/confusion_matrix.png'
    plt.savefig(cm_path, dpi=300, bbox_inches='tight')
    print(f"✓ Confusion matrix saved to {cm_path}")
    plt.show()
    
    # ==================== Save Results ====================
    
    results_csv_path = '/kaggle/working/detailed_evaluation_results.csv'
    full_analysis_df.to_csv(results_csv_path, index=False)
    print(f"\n✓ Detailed results saved to {results_csv_path}")
    
    print("\n" + "="*70)
    print("Part 9 Complete - Detailed evaluation finished")
    print("="*70)

except NameError as e:
    print(f"\n✗ {e}")
    print("   Run Part 8 first")
    
except Exception as e:
    print(f"\n✗ Error during detailed evaluation: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# ==================== Part 10: DIAGNOSTIC 5 - Feature Visualization (t-SNE) ====================

try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    
    print("\n" + "="*70)
    print("DIAGNOSTIC 5: Feature Visualization (t-SNE for Sarcasm Separability)")
    print("="*70)
    
    if 'predictor' not in locals() or predictor is None:
        print("✗ Predictor not found. Please run Part 8 first.")
        raise NameError("Predictor not initialized")
    
    model.eval()
    all_feat, all_sarcasm = [], []
    
    feature_key = 'text_features'  # Can also use 'image_features'
    
    print(f"\nExtracting {feature_key} from validation set...")
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Extracting features"):
            for k in batch:
                batch[k] = batch[k].to(device)
            out = model(batch)
            feat = out[feature_key]
            all_feat.append(feat.detach().cpu().numpy())
            all_sarcasm.append(batch['emotion_labels'][:,1].detach().cpu().numpy())  # idx 1 = sarcasm

    X = np.concatenate(all_feat, 0)
    y = np.concatenate(all_sarcasm, 0)
    
    print(f"Feature matrix shape: {X.shape}")
    print(f"Sarcasm label distribution: {np.sum(y==1)} sarcastic, {np.sum(y==0)} non-sarcastic")
    
    print("\nRunning t-SNE dimensionality reduction (this may take a minute)...")
    Z = TSNE(n_components=2, perplexity=30, learning_rate='auto', init='pca', random_state=42).fit_transform(X)
    
    plt.figure(figsize=(10, 8))
    plt.scatter(Z[y==0,0], Z[y==0,1], s=6, alpha=0.5, label='non-sarcasm', c='blue')
    plt.scatter(Z[y==1,0], Z[y==1,1], s=12, alpha=0.8, label='sarcasm', c='red')
    plt.legend()
    plt.title(f"t-SNE of {feature_key} — Sarcasm Separability", fontsize=14)
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    tsne_path = '/kaggle/working/tsne_sarcasm_separability.png'
    plt.savefig(tsne_path, dpi=300, bbox_inches='tight')
    print(f"✓ t-SNE plot saved to {tsne_path}")
    plt.show()
    
    print("\nInterpretation:")
    print("  - If clusters overlap completely: features don't capture sarcasm well")
    print("  - If clusters are somewhat separated: model has learned some sarcasm patterns")
    print("  - Consider using ASL loss or better text augmentation if separability is poor")
    
    print("\n" + "="*70)
    print("Part 10 Complete - All diagnostics finished!")
    print("="*70)
    print("\nGenerated files:")
    print("  1. /kaggle/working/checkpoints/best_model.pt")
    print("  2. /kaggle/working/checkpoints/model_card.md")
    print("  3. /kaggle/working/confusion_matrix.png")
    print("  4. /kaggle/working/detailed_evaluation_results.csv")
    print("  5. /kaggle/working/tsne_sarcasm_separability.png")
    print("\n" + "="*70)
    print("ALL PARTS COMPLETE! 🎉")
    print("="*70)
    
except Exception as e:
    print(f"\n✗ Error during t-SNE visualization: {e}")
    import traceback
    traceback.print_exc()

In [1]:
"""
Enhanced Multi-modal Meme Analysis with Advanced Imbalance Handling
=====================================================================

Key Improvements:
- Stratified multi-label split (instead of last 1500 rows)
- ASL loss for emotions (γ_neg=6, clip=0.05)
- Class-balanced CE for sentiment
- WeightedRandomSampler for motivational oversampling
- Precision-targeted threshold tuning
- Earlier backbone unfreezing (epoch 2)
- Label priors tracking and adjustment
"""

# ==================== A) ENHANCED DATA PREPARATION CELL ====================
# This section includes stratified splitting, proper label normalization,
# and saving label priors for training

!pip install -q transformers accelerate torch timm scikit-learn pandas matplotlib
!pip install -U "requests>=2.31.0" "transformers>=4.40.0" "huggingface_hub>=0.18.0"
!pip install -q iterstrat  # For iterative stratification

!gdown 1jEJ2nf5CDJknq80ogzU-Uyz7jbBi-1LZ --fuzzy
!unzip *.zip
!gdown --folder 19yaav8ORSVj9DeJUaHKq1H3HtVnkClBw --remaining-ok

import zipfile
import os
import json

# Path to the zip file
zip_path = '/kaggle/working/Memotion 3/memotion3.zip'
extract_to = '/kaggle/working/'
password = b'memotion3taskaaai@22'

# Unzip the file with password
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to, pwd=password)
print(f"Successfully extracted to: {extract_to}")

print("\nExtracted files/folders:")
for item in os.listdir(extract_to):
    print(f" - {item}")

import pandas as pd
import numpy as np
import shutil
from tqdm import tqdm
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

# --- 1. DEFINE PATHS AND PARAMETERS ---
ORIGINAL_TRAIN_IMG_DIR = '/kaggle/working/trainImages/'
ORIGINAL_CSV_PATH = '/kaggle/working/memotion3/train.csv'
IMAGE_FILENAME_COLUMN = 'Unnamed: 0'  # Will be auto-detected
VALIDATION_SPLIT_RATIO = 0.15  # ~1500 samples if total ~10000

# --- 2. DEFINE OUTPUT DIRECTORIES ---
OUTPUT_BASE_DIR = '/kaggle/working/'
NEW_VAL_DIR = os.path.join(OUTPUT_BASE_DIR, 'validation_images/')
NEW_TRAIN_DIR = os.path.join(OUTPUT_BASE_DIR, 'new_train_images/')

os.makedirs(NEW_VAL_DIR, exist_ok=True)
os.makedirs(NEW_TRAIN_DIR, exist_ok=True)
print(f"Created directories:\n- {NEW_VAL_DIR}\n- {NEW_TRAIN_DIR}")

# --- 3. LOAD AND NORMALIZE LABELS ---
print("\nLoading and normalizing CSV file...")
df = pd.read_csv(ORIGINAL_CSV_PATH)
print(f"CSV columns: {list(df.columns)}")
print(f"\nOriginal dataframe shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

# Auto-detect image column
possible_image_cols = ['image_name', 'image', 'img_name', 'filename', 'Unnamed: 0']
IMAGE_FILENAME_COLUMN = next((col for col in possible_image_cols if col in df.columns), df.columns[0])
print(f"\n✅ Detected image column: '{IMAGE_FILENAME_COLUMN}'")

# --- 4. LABEL NORMALIZATION ---
print("\n" + "="*70)
print("Label Normalization")
print("="*70)

# Normalize string labels (lowercase + strip)
for col in ['offensive', 'motivational', 'humour', 'humor', 'sarcastic', 'sarcasm', 'overall', 'sentiment']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.lower().str.strip()

# Create binary emotion columns for stratification
def create_binary_label(value, positive_values):
    """Convert label to binary"""
    if pd.isna(value) or value in ['nan', 'none', '']:
        return 0
    return 1 if value in positive_values else 0

# Binary columns for emotions
if 'offensive' in df.columns:
    df['offensive_bin'] = df['offensive'].apply(
        lambda x: create_binary_label(x, ['slight', 'very_offensive', 'hateful_offensive'])
    )
else:
    df['offensive_bin'] = 0

if 'motivational' in df.columns:
    df['motivational_bin'] = df['motivational'].apply(
        lambda x: create_binary_label(x, ['motivational'])
    )
else:
    df['motivational_bin'] = 0

if 'humour' in df.columns or 'humor' in df.columns:
    humor_col = 'humour' if 'humour' in df.columns else 'humor'
    df['humor_bin'] = df[humor_col].apply(
        lambda x: create_binary_label(x, ['funny', 'very_funny', 'hilarious'])
    )
else:
    df['humor_bin'] = 0

if 'sarcastic' in df.columns or 'sarcasm' in df.columns:
    sarcasm_col = 'sarcastic' if 'sarcastic' in df.columns else 'sarcasm'
    df['sarcasm_bin'] = df[sarcasm_col].apply(
        lambda x: create_binary_label(x, ['general', 'twisted_meaning', 'very_twisted'])
    )
else:
    df['sarcasm_bin'] = 0

print("\n✅ Binary emotion columns created for stratification")

# --- 5. STRATIFIED MULTI-LABEL SPLIT ---
print("\n" + "="*70)
print("Stratified Multi-Label Split")
print("="*70)

# Prepare multi-label matrix for stratification
# Primary: offensive_bin, motivational_bin (key imbalanced labels)
# Secondary: sentiment (if available)
stratify_columns = ['offensive_bin', 'motivational_bin']

# Add sentiment as stratification factor if available
sentiment_col = None
for col in ['overall', 'sentiment', 'overall_sentiment']:
    if col in df.columns:
        sentiment_col = col
        # Convert sentiment to numeric for stratification
        sentiment_map = {
            'very_positive': 0, 'positive': 1, 'neutral': 2, 
            'negative': 3, 'very_negative': 4
        }
        df['sentiment_num'] = df[col].map(sentiment_map).fillna(2).astype(int)
        stratify_columns.append('sentiment_num')
        break

print(f"Stratifying by: {stratify_columns}")

# Create stratification matrix
y_stratify = df[stratify_columns].values

# Perform stratified split
msss = MultilabelStratifiedShuffleSplit(
    n_splits=1, 
    test_size=VALIDATION_SPLIT_RATIO, 
    random_state=42
)

train_idx, val_idx = next(msss.split(df, y_stratify))
train_df = df.iloc[train_idx].reset_index(drop=True)
val_df = df.iloc[val_idx].reset_index(drop=True)

print(f"\n📊 Stratified Split Results:")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")

# Verify stratification quality
print("\n📊 Label Distribution Comparison:")
for col in stratify_columns:
    train_pos = train_df[col].sum()
    val_pos = val_df[col].sum()
    train_rate = train_pos / len(train_df)
    val_rate = val_pos / len(val_df)
    print(f"\n{col}:")
    print(f"  Train: {train_pos}/{len(train_df)} = {train_rate:.2%}")
    print(f"  Val:   {val_pos}/{len(val_df)} = {val_rate:.2%}")
    print(f"  Difference: {abs(train_rate - val_rate):.2%}")

# --- 6. CALCULATE AND SAVE LABEL PRIORS ---
print("\n" + "="*70)
print("Calculating Label Priors for Training")
print("="*70)

label_priors = {}

# Emotion priors (for ASL adjustment)
label_priors['offensive_pos_rate'] = float(train_df['offensive_bin'].sum() / len(train_df))
label_priors['motivational_pos_rate'] = float(train_df['motivational_bin'].sum() / len(train_df))
label_priors['humor_pos_rate'] = float(train_df['humor_bin'].sum() / len(train_df))
label_priors['sarcasm_pos_rate'] = float(train_df['sarcasm_bin'].sum() / len(train_df))

print(f"\nEmotion Positive Rates:")
print(f"  Offensive: {label_priors['offensive_pos_rate']:.2%}")
print(f"  Motivational: {label_priors['motivational_pos_rate']:.2%}")
print(f"  Humor: {label_priors['humor_pos_rate']:.2%}")
print(f"  Sarcasm: {label_priors['sarcasm_pos_rate']:.2%}")

# Sentiment class distribution (for class-balanced weights)
if sentiment_col:
    sentiment_dist = train_df['sentiment_num'].value_counts().sort_index().to_dict()
    label_priors['sentiment_distribution'] = sentiment_dist
    
    # Calculate effective number of samples for class-balanced weights
    # Formula: (1 - β^n) / (1 - β) where β = (N-1)/N
    N = len(train_df)
    beta = (N - 1) / N
    
    sentiment_weights = {}
    for class_id, count in sentiment_dist.items():
        effective_num = (1 - beta**count) / (1 - beta)
        sentiment_weights[int(class_id)] = (1 - beta) * N / effective_num
    
    # Normalize weights
    weight_sum = sum(sentiment_weights.values())
    sentiment_weights = {k: v / weight_sum * len(sentiment_weights) 
                        for k, v in sentiment_weights.items()}
    
    label_priors['sentiment_cb_weights'] = sentiment_weights
    
    print(f"\nSentiment Distribution:")
    for class_id, count in sorted(sentiment_dist.items()):
        sentiment_labels = {0: 'very_positive', 1: 'positive', 2: 'neutral', 
                          3: 'negative', 4: 'very_negative'}
        print(f"  {sentiment_labels[class_id]}: {count} ({count/N:.2%})")
    
    print(f"\nClass-Balanced Weights:")
    for class_id, weight in sorted(sentiment_weights.items()):
        sentiment_labels = {0: 'very_positive', 1: 'positive', 2: 'neutral', 
                          3: 'negative', 4: 'very_negative'}
        print(f"  {sentiment_labels[class_id]}: {weight:.4f}")

# Save priors to JSON
priors_path = os.path.join(OUTPUT_BASE_DIR, 'label_priors.json')
with open(priors_path, 'w') as f:
    json.dump(label_priors, f, indent=2)
print(f"\n✅ Label priors saved to: {priors_path}")

# --- 7. COPY IMAGE FILES WITH EXTENSION HANDLING ---
def copy_images_with_extension(df_subset, dest_dir, source_dir, image_col):
    """Copy images with proper extension handling"""
    copied = 0
    missing = 0
    
    for idx in tqdm(df_subset[image_col]):
        # Convert to string and handle extensions
        filename = str(idx)
        
        # Add extension if missing
        if not any(filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
            # Try common extensions
            for ext in ['.jpg', '.jpeg', '.png']:
                test_path = os.path.join(source_dir, f"{filename}{ext}")
                if os.path.exists(test_path):
                    filename = f"{filename}{ext}"
                    break
            else:
                # Default to .jpg
                filename = f"{filename}.jpg"
        
        source_path = os.path.join(source_dir, filename)
        
        if os.path.exists(source_path):
            destination_path = os.path.join(dest_dir, filename)
            shutil.copy(source_path, destination_path)
            copied += 1
        else:
            missing += 1
            if missing <= 5:
                print(f"Missing: {source_path}")
    
    return copied, missing

print(f"\n📁 Copying validation images...")
copied_val, missing_val = copy_images_with_extension(
    val_df, NEW_VAL_DIR, ORIGINAL_TRAIN_IMG_DIR, IMAGE_FILENAME_COLUMN
)

print(f"\n📁 Copying training images...")
copied_train, missing_train = copy_images_with_extension(
    train_df, NEW_TRAIN_DIR, ORIGINAL_TRAIN_IMG_DIR, IMAGE_FILENAME_COLUMN
)

# --- 8. SAVE CSV FILES ---
new_train_csv_path = os.path.join(OUTPUT_BASE_DIR, 'train_split.csv')
new_val_csv_path = os.path.join(OUTPUT_BASE_DIR, 'validation_split.csv')

train_df.to_csv(new_train_csv_path, index=False)
val_df.to_csv(new_val_csv_path, index=False)

print(f"\n✅ Successfully created stratified split datasets!")
print(f"Training CSV saved to: {new_train_csv_path}")
print(f"Validation CSV saved to: {new_val_csv_path}")

# --- 9. VERIFICATION ---
print("\n📊 Final Verification:")
print(f"Training CSV rows: {len(train_df)}")
print(f"Validation CSV rows: {len(val_df)}")
print(f"Images copied to training directory: {copied_train} (missing: {missing_train})")
print(f"Images copied to validation directory: {copied_val} (missing: {missing_val})")
print(f"Images in training directory: {len(os.listdir(NEW_TRAIN_DIR))}")
print(f"Images in validation directory: {len(os.listdir(NEW_VAL_DIR))}")

# --- 10. CREATE ZIP ARCHIVES ---
TRAIN_PACKAGE_DIR = os.path.join(OUTPUT_BASE_DIR, 'training_dataset/')
VAL_PACKAGE_DIR = os.path.join(OUTPUT_BASE_DIR, 'validation_dataset/')

os.makedirs(TRAIN_PACKAGE_DIR, exist_ok=True)
os.makedirs(VAL_PACKAGE_DIR, exist_ok=True)

# Copy CSVs and images
shutil.copy(new_train_csv_path, os.path.join(TRAIN_PACKAGE_DIR, 'train_split.csv'))
shutil.copytree(NEW_TRAIN_DIR, os.path.join(TRAIN_PACKAGE_DIR, 'new_train_images'), dirs_exist_ok=True)

shutil.copy(new_val_csv_path, os.path.join(VAL_PACKAGE_DIR, 'validation_split.csv'))
shutil.copytree(NEW_VAL_DIR, os.path.join(VAL_PACKAGE_DIR, 'validation_images'), dirs_exist_ok=True)

# Create ZIP archives
train_zip_path = shutil.make_archive(
    os.path.join(OUTPUT_BASE_DIR, 'training_dataset'), 'zip', TRAIN_PACKAGE_DIR
)
val_zip_path = shutil.make_archive(
    os.path.join(OUTPUT_BASE_DIR, 'validation_dataset'), 'zip', VAL_PACKAGE_DIR
)

print("\n✅ Created ZIP files:")
print(f"📦 Training dataset ZIP: {train_zip_path}")
print(f"📦 Validation dataset ZIP: {val_zip_path}")

print("\n" + "="*70)
print("DATA PREPARATION COMPLETE")
print("="*70)
print("\nKey Improvements:")
print("✓ Stratified multi-label split (not last 1500 rows)")
print("✓ Label normalization (lowercase + strip)")
print("✓ Binary emotion columns for diagnostics")
print("✓ Label priors saved for training")
print("✓ Extension handling for images")


# ==================== PART 1: ENHANCED CONFIGURATION ====================
"""
Part 1: Enhanced Configuration with ASL and Class-Balanced Weights
"""

import os
import yaml
import random
import zipfile
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.cuda.amp import GradScaler, autocast

from transformers import AutoModel, AutoTokenizer, CLIPModel, CLIPProcessor
import torchvision.transforms as transforms

from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, 
    mean_squared_error, mean_absolute_error, confusion_matrix, 
    classification_report, average_precision_score, f1_score,
    precision_score, recall_score
)
from scipy.stats import spearmanr

print("✓ All imports successful")

# Load label priors
PRIORS_PATH = '/kaggle/working/label_priors.json'
with open(PRIORS_PATH, 'r') as f:
    LABEL_PRIORS = json.load(f)

print(f"✓ Loaded label priors from {PRIORS_PATH}")

# ==================== ENHANCED CONFIGURATION ====================
CONFIG = f"""
# Enhanced Multi-modal Fusion Configuration

# Model Architecture
TEXT_MODEL: "google/muril-base-cased"
IMAGE_MODEL: "openai/clip-vit-base-patch32"
TEXT_DIM: 768
IMAGE_DIM: 768
FUSION_DIM: 512
FUSION_OUT_DIM: 512

# Training Parameters
MAX_LEN: 128
IMG_SIZE: 224
BATCH_SIZE: 16
GRADIENT_ACCUMULATION_STEPS: 2
LR_HEADS: 0.001
LR_BACKBONE: 0.00001
WEIGHT_DECAY: 0.01
EPOCHS: 5
SEED: 42
DEVICE: "cuda"
CHECKPOINT_PATH: "/kaggle/working/checkpoints"

# Task Configuration
NUM_SENTIMENT_CLASSES: 5
NUM_EMOTION_CLASSES: 4  # humor, sarcasm, offensive, motivational

# Loss Weights (ENHANCED: Higher sentiment weight)
LOSS_WEIGHTS:
  sentiment: 1.5
  emotion: 1.0
  intensity: 0.5

# ASL Hyperparameters for Emotions
ASL_GAMMA_NEG: 6.0
ASL_GAMMA_POS: 0.0
ASL_CLIP: 0.05

# Emotion Priors (from training data)
EMO_PRIORS: [{LABEL_PRIORS['humor_pos_rate']}, {LABEL_PRIORS['sarcasm_pos_rate']}, {LABEL_PRIORS['offensive_pos_rate']}, {LABEL_PRIORS['motivational_pos_rate']}]

# Sentiment Class-Balanced Weights
SENTIMENT_CB_WEIGHTS:
  0: {LABEL_PRIORS.get('sentiment_cb_weights', {}).get('0', 1.0)}
  1: {LABEL_PRIORS.get('sentiment_cb_weights', {}).get('1', 1.0)}
  2: {LABEL_PRIORS.get('sentiment_cb_weights', {}).get('2', 1.0)}
  3: {LABEL_PRIORS.get('sentiment_cb_weights', {}).get('3', 1.0)}
  4: {LABEL_PRIORS.get('sentiment_cb_weights', {}).get('4', 1.0)}

# Emotion Thresholds (will be tuned during training)
EMO_THRESHOLDS: [0.5, 0.5, 0.60, 0.60]  # humor, sarcasm, offensive, motivational

# Precision Targets for Threshold Tuning
PRECISION_TARGETS:
  humor: 0.70
  sarcasm: 0.70
  offensive: 0.75
  motivational: 0.60

# Training Strategy
POOLING: "mean"
USE_AMP: true
GRADIENT_CLIP: 1.0
SCHEDULER: "cosine"
WARMUP_EPOCHS: 1
UNFREEZE_BACKBONE_EPOCH: 2  # CHANGED: Earlier unfreezing (was 3)
LABEL_SMOOTHING: 0.1

# Cross-Attention
CROSS_ATTN_HEADS: 8
CROSS_ATTN_DROPOUT: 0.1

# Oversampling for Motivational
MOTIVATIONAL_OVERSAMPLE_FACTOR: 5.0
"""

# ==================== UTILITY FUNCTIONS ====================
def set_seed(seed: int):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"✓ Random seed set to {seed}")

def load_config(config_str: str) -> dict:
    """Load configuration from YAML string"""
    cfg = yaml.safe_load(config_str)
    print("✓ Configuration loaded")
    return cfg

def save_checkpoint(state: dict, path: str):
    """Save model checkpoint"""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(state, path)
    print(f"✓ Checkpoint saved to {path}")

def load_checkpoint(model, optimizer, path: str, device: str):
    """Load model checkpoint"""
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print(f"✓ Checkpoint loaded from {path}")
    return checkpoint['epoch'], checkpoint.get('best_metric', 0)

def get_tokenizer(cfg: dict):
    """Get tokenizer for text model"""
    tokenizer = AutoTokenizer.from_pretrained(cfg['TEXT_MODEL'])
    print(f"✓ Tokenizer loaded: {cfg['TEXT_MODEL']}")
    return tokenizer

def get_image_transform(cfg: dict, is_train: bool = True):
    """Get image transforms - CLIP-specific normalization"""
    if is_train:
        transform = transforms.Compose([
            transforms.Resize((cfg['IMG_SIZE'], cfg['IMG_SIZE'])),
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711]
            )
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((cfg['IMG_SIZE'], cfg['IMG_SIZE'])),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711]
            )
        ])
    print(f"✓ Image transform created (train={is_train})")
    return transform

def explore_dataset(df, name="Dataset"):
    """Explore dataset structure and label distributions"""
    print(f"\n{'='*70}")
    print(f"{name} Exploration")
    print(f"{'='*70}")
    print(f"\nShape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    
    # Check for label columns
    label_cols = ['humour', 'humor', 'sarcastic', 'sarcasm', 'offensive', 'motivational', 'overall', 'sentiment']
    for col in label_cols:
        if col in df.columns:
            print(f"\n{col} distribution:")
            print(df[col].value_counts())
    
    print(f"\nMissing values:")
    print(df.isnull().sum())
    
    print(f"\nSample rows:")
    print(df.head(3))
    print(f"{'='*70}\n")

# Initialize configuration
cfg = load_config(CONFIG)
set_seed(cfg['SEED'])

# Set device
device = torch.device(cfg['DEVICE'] if torch.cuda.is_available() else 'cpu')
print(f"✓ Using device: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  CUDA Version: {torch.version.cuda}")

print("\n" + "="*70)
print("Part 1 Complete - Enhanced Configuration Loaded")
print("="*70)
print("\nKey Changes:")
print("✓ ASL hyperparameters configured (γ_neg=6.0, clip=0.05)")
print("✓ Emotion priors loaded from training data")
print("✓ Class-balanced weights calculated for sentiment")
print("✓ Sentiment loss weight increased to 1.5")
print("✓ Backbone unfreezing moved to epoch 2")
print("✓ Precision targets set for threshold tuning")
print("✓ Motivational oversampling factor: 5.0x")


# ==================== PART 2: DATA PREPARATION AND DATASET ====================
"""
Part 2: Data Preparation and Enhanced Dataset Class
"""

def prepare_data(cfg):
    """Extract and prepare dataset"""
    print("Preparing datasets...")
    
    train_dir = '/kaggle/working/train'
    val_dir = '/kaggle/working/val'
    
    # Extract if needed
    if not os.path.exists(train_dir):
        print("Extracting training dataset...")
        with zipfile.ZipFile('/kaggle/working/training_dataset.zip', 'r') as zip_ref:
            zip_ref.extractall('/kaggle/working/train')
    else:
        print("✓ Training dataset already extracted")
    
    if not os.path.exists(val_dir):
        print("Extracting validation dataset...")
        with zipfile.ZipFile('/kaggle/working/validation_dataset.zip', 'r') as zip_ref:
            zip_ref.extractall('/kaggle/working/val')
    else:
        print("✓ Validation dataset already extracted")
    
    # Find CSV files
    train_csv = None
    val_csv = None
    
    for root, dirs, files in os.walk(train_dir):
        for file in files:
            if file.endswith('.csv'):
                train_csv = os.path.join(root, file)
                break
        if train_csv:
            break
    
    for root, dirs, files in os.walk(val_dir):
        for file in files:
            if file.endswith('.csv'):
                val_csv = os.path.join(root, file)
                break
        if val_csv:
            break
    
    if train_csv is None or val_csv is None:
        raise FileNotFoundError("Could not find CSV files")
    
    print(f"✓ Loading training data from {train_csv}")
    train_df = pd.read_csv(train_csv)
    
    print(f"✓ Loading validation data from {val_csv}")
    val_df = pd.read_csv(val_csv)
    
    # Find image directories
    train_img_dir = None
    val_img_dir = None
    
    for root, dirs, files in os.walk(train_dir):
        img_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png'))]
        if img_files:
            train_img_dir = root
            break
        if 'images' in dirs:
            train_img_dir = os.path.join(root, 'images')
            break
    
    for root, dirs, files in os.walk(val_dir):
        img_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png'))]
        if img_files:
            val_img_dir = root
            break
        if 'images' in dirs:
            val_img_dir = os.path.join(root, 'images')
            break
    
    if train_img_dir is None or val_img_dir is None:
        raise FileNotFoundError("Could not find image directories")
    
    print(f"\n{'='*70}")
    print(f"Dataset Information:")
    print(f"  Train CSV: {train_csv}")
    print(f"  Train Images: {train_img_dir}")
    print(f"  Train Samples: {len(train_df)}")
    print(f"  Val CSV: {val_csv}")
    print(f"  Val Images: {val_img_dir}")
    print(f"  Val Samples: {len(val_df)}")
    print(f"{'='*70}\n")
    
    return train_df, val_df, train_img_dir, val_img_dir


class MemeDataset(Dataset):
    """Enhanced Dataset for meme sentiment and emotion analysis"""
    
    def __init__(self, df: pd.DataFrame, tokenizer, image_transform, 
                 image_dir: str, cfg: dict):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.image_transform = image_transform
        self.image_dir = image_dir
        self.cfg = cfg
        
        self._detect_columns()
        print(f"✓ Dataset initialized with {len(self.df)} samples")
    
    def _detect_columns(self):
        """Detect column names in the dataframe"""
        cols = self.df.columns.tolist()
        
        # Image column
        possible_image_cols = ['image_name', 'image', 'img_name', 'filename', 'image_id', 'Unnamed: 0']
        self.image_col = next((col for col in possible_image_cols if col in cols), cols[0])
        
        # Text column
        possible_text_cols = ['text', 'ocr_text', 'caption', 'extracted_text', 'OCR_extracted_text', 'ocr']
        self.text_col = next((col for col in possible_text_cols if col in cols), None)
        
        # Sentiment column
        possible_sentiment_cols = ['sentiment', 'overall_sentiment', 'sentiment_label', 'overall']
        self.sentiment_col = next((col for col in possible_sentiment_cols if col in cols), None)
        
        print(f"  Detected columns - Image: {self.image_col}, Text: {self.text_col}, Sentiment: {self.sentiment_col}")
        
        self._create_label_mappings()
    
    def _create_label_mappings(self):
        """Create mappings for string labels to numeric values"""
        # Sentiment mapping (5 classes)
        self.sentiment_map = {
            'very_positive': 0,
            'positive': 1,
            'neutral': 2,
            'negative': 3,
            'very_negative': 4
        }
        
        # Humor mapping (binary)
        self.humor_map = {
            'not_funny': 0,
            'funny': 1,
            'very_funny': 1,
            'hilarious': 1
        }
        
        # Sarcasm mapping (binary)
        self.sarcasm_map = {
            'not_sarcastic': 0,
            'general': 1,
            'twisted_meaning': 1,
            'very_twisted': 1
        }
        
        # Offensive mapping (binary)
        self.offensive_map = {
            'not_offensive': 0,
            'slight': 1,
            'very_offensive': 1,
            'hateful_offensive': 1
        }
        
        # Motivational mapping (binary)
        self.motivational_map = {
            'not_motivational': 0,
            'motivational': 1
        }
    
    def _map_label(self, value, mapping, default=0):
        """Map string label to numeric value"""
        if pd.isna(value):
            return default
        if isinstance(value, str):
            return mapping.get(value.lower().strip(), default)
        try:
            return int(value)
        except:
            return default
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int) -> Dict:
        row = self.df.iloc[idx]
        
        # Load image with extension handling
        image_name = row[self.image_col] if self.image_col else f"{idx}.jpg"
        
        if isinstance(image_name, (int, float)):
            image_name = f"{int(image_name)}.jpg"
        else:
            image_name = str(image_name)
        
        # Add extension if missing
        if not any(image_name.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
            image_name = f"{image_name}.jpg"
        
        image_path = os.path.join(self.image_dir, image_name)
        
        try:
            image = Image.open(image_path).convert('RGB')
            image = self.image_transform(image)
        except Exception as e:
            # Use blank image on error
            image = torch.zeros(3, self.cfg['IMG_SIZE'], self.cfg['IMG_SIZE'])
        
        # Tokenize text
        text = str(row.get(self.text_col, '')) if self.text_col else ''
        if text in ['nan', '', 'None'] or text is None:
            text = 'No text available'
        
        encoding = self.tokenizer(
            text,
            max_length=self.cfg['MAX_LEN'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Get sentiment label (5 classes)
        sentiment_label = 2  # default neutral
        if self.sentiment_col and self.sentiment_col in row:
            sent_val = row[self.sentiment_col]
            sentiment_label = self._map_label(sent_val, self.sentiment_map, default=2)
        sentiment_label = torch.tensor(sentiment_label, dtype=torch.long)
        
        # Get emotion labels (multi-label binary)
        humor_val = row.get('humour', row.get('humor', 0))
        sarcasm_val = row.get('sarcastic', row.get('sarcasm', 0))
        offensive_val = row.get('offensive', 0)
        motivational_val = row.get('motivational', 0)
        
        emotion_labels = torch.tensor([
            float(self._map_label(humor_val, self.humor_map, default=0)),
            float(self._map_label(sarcasm_val, self.sarcasm_map, default=0)),
            float(self._map_label(offensive_val, self.offensive_map, default=0)),
            float(self._map_label(motivational_val, self.motivational_map, default=0))
        ], dtype=torch.float)
        
        # Get intensity score (normalized 0-1)
        intensity_val = row.get('overall_sentiment', row.get('overall', 0.0))
        if isinstance(intensity_val, str):
            intensity_score_map = {
                'very_positive': 1.0,
                'positive': 0.75,
                'neutral': 0.5,
                'negative': 0.25,
                'very_negative': 0.0
            }
            intensity_val = intensity_score_map.get(intensity_val.lower().strip(), 0.5)
        
        intensity = torch.tensor(
            float(intensity_val) if not pd.isna(intensity_val) else 0.5, 
            dtype=torch.float
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'image': image,
            'sentiment_label': sentiment_label,
            'emotion_labels': emotion_labels,
            'intensity': intensity,
            'motivational_flag': emotion_labels[3]  # For weighted sampling
        }


def collate_fn(batch: List[Dict]):
    """Collate function for batching"""
    return {
        'input_ids': torch.stack([item['input_ids'] for item in batch]),
        'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
        'image': torch.stack([item['image'] for item in batch]),
        'sentiment_label': torch.stack([item['sentiment_label'] for item in batch]),
        'emotion_labels': torch.stack([item['emotion_labels'] for item in batch]),
        'intensity': torch.stack([item['intensity'] for item in batch])
    }


# ==================== Run Data Preparation ====================
try:
    train_df, val_df, train_img_dir, val_img_dir = prepare_data(cfg)
    explore_dataset(train_df, "Training Set")
    explore_dataset(val_df, "Validation Set")
    
    print("\n" + "="*70)
    print("Part 2 Complete - Data preparation successful")
    print("="*70)
except Exception as e:
    print(f"\n✗ Error in data preparation: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 3: ENHANCED MODEL WITH ASL LOSS ====================
"""
Part 3: Model Architecture with ASL Loss and Class-Balanced CE
"""

class CrossAttentionFusion(nn.Module):
    """Bidirectional cross-attention between text and image modalities"""
    
    def __init__(self, dim, num_heads=8, dropout=0.1):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        assert self.head_dim * num_heads == dim, "dim must be divisible by num_heads"
        
        # Text queries attend to Image keys/values
        self.text_to_image_attn = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        # Image queries attend to Text keys/values
        self.image_to_text_attn = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        # Layer norms
        self.text_norm = nn.LayerNorm(dim)
        self.image_norm = nn.LayerNorm(dim)
        
        # Feed-forward networks
        self.text_ffn = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 4, dim),
            nn.Dropout(dropout)
        )
        
        self.image_ffn = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 4, dim),
            nn.Dropout(dropout)
        )
        
        self.ffn_norm_text = nn.LayerNorm(dim)
        self.ffn_norm_image = nn.LayerNorm(dim)
    
    def forward(self, text_emb, image_emb):
        # Add sequence dimension for attention (batch, 1, dim)
        text_seq = text_emb.unsqueeze(1)
        image_seq = image_emb.unsqueeze(1)
        
        # Text attends to Image
        text_attended, _ = self.text_to_image_attn(
            query=text_seq,
            key=image_seq,
            value=image_seq
        )
        text_attended = text_attended.squeeze(1)
        text_out = self.text_norm(text_emb + text_attended)
        
        # Image attends to Text
        image_attended, _ = self.image_to_text_attn(
            query=image_seq,
            key=text_seq,
            value=text_seq
        )
        image_attended = image_attended.squeeze(1)
        image_out = self.image_norm(image_emb + image_attended)
        
        # Feed-forward with residual
        text_ffn_out = self.text_ffn(text_out)
        text_final = self.ffn_norm_text(text_out + text_ffn_out)
        
        image_ffn_out = self.image_ffn(image_out)
        image_final = self.ffn_norm_image(image_out + image_ffn_out)
        
        return text_final, image_final


class EnhancedFusionModel(nn.Module):
    """Multi-modal fusion model with Cross-Attention"""
    
    def __init__(self, cfg: dict):
        super().__init__()
        self.cfg = cfg
        
        print(f"Loading text model: {cfg['TEXT_MODEL']}")
        self.text_model = AutoModel.from_pretrained(cfg['TEXT_MODEL'])
        
        print(f"Loading image model: {cfg['IMAGE_MODEL']}")
        clip_model = CLIPModel.from_pretrained(cfg['IMAGE_MODEL'])
        self.image_model = clip_model.vision_model
        
        # Freeze encoders initially
        self._freeze_encoders()
        
        # Projection layers to common dimension
        self.text_proj = nn.Linear(cfg['TEXT_DIM'], cfg['FUSION_DIM'])
        self.image_proj = nn.Linear(cfg['IMAGE_DIM'], cfg['FUSION_DIM'])
        
        # Initialize projections
        nn.init.xavier_uniform_(self.text_proj.weight)
        nn.init.zeros_(self.text_proj.bias)
        nn.init.xavier_uniform_(self.image_proj.weight)
        nn.init.zeros_(self.image_proj.bias)
        
        # Cross-Attention Fusion
        self.cross_attention = CrossAttentionFusion(
            dim=cfg['FUSION_DIM'],
            num_heads=8,
            dropout=0.1
        )
        
        # Final fusion MLP
        fusion_input_dim = cfg['FUSION_DIM'] * 2
        self.fusion_norm = nn.LayerNorm(fusion_input_dim)
        self.fusion_mlp = nn.Sequential(
            nn.Linear(fusion_input_dim, 512),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(512, cfg['FUSION_OUT_DIM']),
            nn.LayerNorm(cfg['FUSION_OUT_DIM'])
        )
        
        # Task-specific heads
        self.sentiment_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, cfg['NUM_SENTIMENT_CLASSES'])
        )
        
        self.emotion_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, cfg['NUM_EMOTION_CLASSES'])
        )
        
        self.intensity_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )
        
        print("✓ Enhanced model with Cross-Attention initialized")
    
    def _freeze_encoders(self):
        """Freeze encoder parameters"""
        for param in self.text_model.parameters():
            param.requires_grad = False
        for param in self.image_model.parameters():
            param.requires_grad = False
        print("✓ Encoders frozen")
    
    def unfreeze_backbone(self, layers_to_unfreeze: int = 2):
        """Unfreeze top layers of encoders"""
        if hasattr(self.text_model, 'encoder') and hasattr(self.text_model.encoder, 'layer'):
            text_layers = list(self.text_model.encoder.layer[-layers_to_unfreeze:])
            for layer in text_layers:
                for param in layer.parameters():
                    param.requires_grad = True
            print(f"✓ Unfroze {layers_to_unfreeze} text encoder layers")
        
        if hasattr(self.image_model, 'encoder') and hasattr(self.image_model.encoder, 'layers'):
            image_layers = list(self.image_model.encoder.layers[-layers_to_unfreeze:])
            for layer in image_layers:
                for param in layer.parameters():
                    param.requires_grad = True
            print(f"✓ Unfroze {layers_to_unfreeze} image encoder layers")
    
    def pool_text(self, model_output, attention_mask):
        """Pool text embeddings"""
        last_hidden = model_output.last_hidden_state
        if self.cfg['POOLING'] == 'cls':
            return last_hidden[:, 0]
        elif self.cfg['POOLING'] == 'mean':
            mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
            sum_embeddings = torch.sum(last_hidden * mask_expanded, 1)
            sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
            return sum_embeddings / sum_mask
        else:
            return last_hidden[:, 0]
    
    def forward(self, input_ids, attention_mask, image):
        """Forward pass with cross-attention fusion"""
        # Text encoding
        text_output = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_emb = self.pool_text(text_output, attention_mask)
        
        # Image encoding
        image_output = self.image_model(pixel_values=image)
        image_emb = image_output.pooler_output
        
        # Project to common dimension
        text_proj = self.text_proj(text_emb)
        image_proj = self.image_proj(image_emb)
        
        # Cross-Attention Fusion
        text_cross, image_cross = self.cross_attention(text_proj, image_proj)
        
        # Concatenate and fuse
        fused = torch.cat([text_cross, image_cross], dim=1)
        fused = self.fusion_norm(fused)
        fused = self.fusion_mlp(fused)
        
        # Task predictions
        sentiment_logits = self.sentiment_head(fused)
        emotion_logits = self.emotion_head(fused)
        intensity = self.intensity_head(fused).squeeze(-1)
        
        return {
            'sentiment_logits': sentiment_logits,
            'emotion_logits': emotion_logits,
            'intensity': intensity,
            'text_features': text_cross,
            'image_features': image_cross
        }


# ==================== ENHANCED LOSS FUNCTIONS ====================

class AsymmetricLoss(nn.Module):
    """
    Asymmetric Loss (ASL) for multi-label classification
    https://arxiv.org/abs/2009.14119
    """
    
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8):
        super(AsymmetricLoss, self).__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.eps = eps
    
    def forward(self, logits, targets):
        """
        Parameters:
        -----------
        logits : Tensor
            Predicted logits (batch_size, num_classes)
        targets : Tensor
            Ground truth binary labels (batch_size, num_classes)
        """
        # Sigmoid
        xs_pos = torch.sigmoid(logits)
        xs_neg = 1 - xs_pos
        
        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)
        
        # Basic CE calculation
        los_pos = targets * torch.log(xs_pos.clamp(min=self.eps))
        los_neg = (1 - targets) * torch.log(xs_neg.clamp(min=self.eps))
        
        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            pt0 = xs_pos * targets
            pt1 = xs_neg * (1 - targets)
            pt = pt0 + pt1
            one_sided_gamma = self.gamma_pos * targets + self.gamma_neg * (1 - targets)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)
            loss = one_sided_w * (los_pos + los_neg)
        else:
            loss = los_pos + los_neg
        
        return -loss.mean()


def adjust_logits_with_priors(logits, priors, tau=1.0):
    """
    Adjust logits based on training priors
    
    Parameters:
    -----------
    logits : Tensor
        Raw logits (batch_size, num_classes)
    priors : list or Tensor
        Prior probabilities for each class
    tau : float
        Temperature for adjustment
    """
    if isinstance(priors, list):
        priors = torch.tensor(priors, device=logits.device)
    
    # Logit adjustment: logit - τ * log(prior)
    adjustment = tau * torch.log(priors.clamp(min=1e-8))
    adjusted_logits = logits - adjustment
    
    return adjusted_logits


def sentiment_loss_fn(logits, labels, cfg):
    """
    Sentiment classification loss with class-balanced weights
    """
    # Get class weights
    weights = cfg.get('SENTIMENT_CB_WEIGHTS', {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0})
    weight_tensor = torch.tensor(
        [weights[i] for i in range(cfg['NUM_SENTIMENT_CLASSES'])],
        device=logits.device,
        dtype=torch.float
    )
    
    # Class-balanced cross-entropy
    loss = F.cross_entropy(
        logits, 
        labels, 
        weight=weight_tensor,
        label_smoothing=cfg.get('LABEL_SMOOTHING', 0.0)
    )
    
    return loss


def emotion_loss_fn(logits, labels, cfg):
    """
    Emotion classification loss using ASL with prior adjustment
    """
    # Prior adjustment (optional but recommended)
    priors = cfg.get('EMO_PRIORS', [0.5, 0.5, 0.5, 0.5])
    adjusted_logits = adjust_logits_with_priors(logits, priors, tau=1.0)
    
    # ASL loss
    asl = AsymmetricLoss(
        gamma_neg=cfg.get('ASL_GAMMA_NEG', 4.0),
        gamma_pos=cfg.get('ASL_GAMMA_POS', 0.0),
        clip=cfg.get('ASL_CLIP', 0.05)
    )
    
    loss = asl(adjusted_logits, labels)
    
    return loss


def intensity_loss_fn(preds, targets, cfg):
    """Intensity regression loss"""
    return F.smooth_l1_loss(preds, targets)


def combined_loss(outputs, batch, cfg):
    """Combined multi-task loss"""
    loss_sent = sentiment_loss_fn(outputs['sentiment_logits'], batch['sentiment_label'], cfg)
    loss_emotion = emotion_loss_fn(outputs['emotion_logits'], batch['emotion_labels'], cfg)
    loss_intensity = intensity_loss_fn(outputs['intensity'], batch['intensity'], cfg)
    
    total_loss = (
        cfg['LOSS_WEIGHTS']['sentiment'] * loss_sent +
        cfg['LOSS_WEIGHTS']['emotion'] * loss_emotion +
        cfg['LOSS_WEIGHTS']['intensity'] * loss_intensity
    )
    
    return total_loss, loss_sent, loss_emotion, loss_intensity


# ==================== Initialize and Test Model ====================
try:
    print("\nInitializing Enhanced Model with Cross-Attention and ASL...")
    model = EnhancedFusionModel(cfg).to(device)
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\n{'='*70}")
    print("Model Summary:")
    print(f"  Total parameters: {total_params:,}")
    print(f"  Trainable parameters: {trainable_params:,}")
    print(f"  Frozen parameters: {total_params - trainable_params:,}")
    print(f"{'='*70}")
    
    print("\nTesting forward pass with dummy data...")
    dummy_input_ids = torch.randint(0, 1000, (2, cfg['MAX_LEN'])).to(device)
    dummy_attention_mask = torch.ones(2, cfg['MAX_LEN']).to(device)
    dummy_image = torch.randn(2, 3, cfg['IMG_SIZE'], cfg['IMG_SIZE']).to(device)
    
    with torch.no_grad():
        test_output = model(dummy_input_ids, dummy_attention_mask, dummy_image)
    
    print(f"✓ Forward pass successful!")
    print(f"  Sentiment logits shape: {test_output['sentiment_logits'].shape}")
    print(f"  Emotion logits shape: {test_output['emotion_logits'].shape}")
    print(f"  Intensity shape: {test_output['intensity'].shape}")
    
    print("\n" + "="*70)
    print("Part 3 Complete - Enhanced Model with ASL Loss initialized")
    print("="*70)
    print("\nKey Changes:")
    print("✓ ASL loss for emotions (γ_neg=6.0, γ_pos=0.0, clip=0.05)")
    print("✓ Prior logit adjustment for emotions")
    print("✓ Class-balanced CE for sentiment")
    print("✓ Label smoothing as fallback")

except Exception as e:
    print(f"\n✗ Error in model initialization: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 4: DATALOADERS WITH OVERSAMPLING ====================
"""
Part 4: DataLoader Creation with Weighted Sampling for Motivational
"""

LABELS = ["humor", "sarcasm", "offensive", "motivational"]

try:
    print("Creating tokenizer and transforms...")
    tokenizer = get_tokenizer(cfg)
    train_transform = get_image_transform(cfg, is_train=True)
    val_transform = get_image_transform(cfg, is_train=False)
    
    print("\nCreating datasets...")
    train_dataset = MemeDataset(train_df, tokenizer, train_transform, train_img_dir, cfg)
    val_dataset = MemeDataset(val_df, tokenizer, val_transform, val_img_dir, cfg)
    
    # ==================== WEIGHTED SAMPLING FOR MOTIVATIONAL ====================
    print("\n" + "="*70)
    print("Creating Weighted Sampler for Motivational Oversampling")
    print("="*70)
    
    # Calculate sample weights
    sample_weights = []
    motivational_count = 0
    non_motivational_count = 0
    
    for idx in range(len(train_dataset)):
        item = train_dataset[idx]
        is_motivational = item['motivational_flag'].item()
        
        if is_motivational:
            # Oversample motivational samples
            weight = cfg.get('MOTIVATIONAL_OVERSAMPLE_FACTOR', 5.0)
            motivational_count += 1
        else:
            weight = 1.0
            non_motivational_count += 1
        
        sample_weights.append(weight)
    
    print(f"\nMotivational samples: {motivational_count} ({motivational_count/len(train_dataset):.2%})")
    print(f"Non-motivational samples: {non_motivational_count}")
    print(f"Oversampling factor: {cfg.get('MOTIVATIONAL_OVERSAMPLE_FACTOR', 5.0)}x")
    
    # Create weighted sampler
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    print("✓ Weighted sampler created")
    
    # ==================== CREATE DATALOADERS ====================
    print("\nCreating dataloaders...")
    num_workers = 0 if device.type == 'cpu' else 2
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg['BATCH_SIZE'],
        sampler=sampler,  # Use weighted sampler instead of shuffle
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=collate_fn,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=cfg['BATCH_SIZE'],
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=collate_fn
    )
    
    print(f"✓ Train batches: {len(train_loader)}")
    print(f"✓ Val batches: {len(val_loader)}")
    
    # ==================== Test DataLoader ====================
    print("\nTesting dataloader with real batch...")
    test_batch = next(iter(train_loader))
    print(f"  Input IDs shape: {test_batch['input_ids'].shape}")
    print(f"  Attention mask shape: {test_batch['attention_mask'].shape}")
    print(f"  Image shape: {test_batch['image'].shape}")
    print(f"  Sentiment labels shape: {test_batch['sentiment_label'].shape}")
    print(f"  Emotion labels shape: {test_batch['emotion_labels'].shape}")
    print(f"  Intensity shape: {test_batch['intensity'].shape}")
    
    print("\nSample emotion labels (motivational column):")
    print(test_batch['emotion_labels'][:, 3])  # Motivational is index 3
    
    # ==================== Test Model with Real Batch ====================
    print("\nTesting model forward pass with real batch...")
    model.eval()
    with torch.no_grad():
        batch_input_ids = test_batch['input_ids'][:2].to(device)
        batch_attention_mask = test_batch['attention_mask'][:2].to(device)
        batch_image = test_batch['image'][:2].to(device)
        
        real_output = model(batch_input_ids, batch_attention_mask, batch_image)
    
    print(f"✓ Real batch forward pass successful!")
    
    # Test loss computation
    print("\nTesting loss computation with ASL...")
    batch_device = {
        'sentiment_label': test_batch['sentiment_label'][:2].to(device),
        'emotion_labels': test_batch['emotion_labels'][:2].to(device),
        'intensity': test_batch['intensity'][:2].to(device)
    }
    
    total_loss, loss_sent, loss_emotion, loss_int = combined_loss(real_output, batch_device, cfg)
    
    print(f"✓ Loss computation successful!")
    print(f"  Total loss: {total_loss.item():.4f}")
    print(f"  Sentiment loss (CB-CE): {loss_sent.item():.4f}")
    print(f"  Emotion loss (ASL): {loss_emotion.item():.4f}")
    print(f"  Intensity loss: {loss_int.item():.4f}")
    
    # ==================== Dataset Imbalance Stats ====================
    print("\n" + "="*70)
    print("Dataset Imbalance Statistics")
    print("="*70)
    
    def label_stats(loader):
        cnt = np.zeros(len(LABELS), dtype=np.int64)
        total = 0
        for batch in loader:
            y = batch['emotion_labels'].numpy()
            cnt += y.sum(axis=0).astype(int)
            total += y.shape[0]
        
        print("Total samples:", total)
        for i, name in enumerate(LABELS):
            pos = cnt[i]
            print(f"{name}: pos={pos}, pos_rate={pos/total:.4f}, neg={total-pos}")
        
        return cnt, total
    
    pos_counts, N = label_stats(train_loader)
    neg_counts = N - pos_counts
    
    print("\n" + "="*70)
    print("Part 4 Complete - DataLoaders with Oversampling created")
    print("="*70)
    print("\nKey Changes:")
    print("✓ WeightedRandomSampler for motivational oversampling (5x)")
    print("✓ Proper extension handling in dataset")
    print("✓ Label normalization (lowercase + strip)")
    print("✓ ASL loss tested successfully")

except Exception as e:
    print(f"\n✗ Error in dataloader creation: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 5: METRICS WITH PRECISION-TARGETED THRESHOLDS ====================
"""
Part 5: Enhanced Metrics and Precision-Targeted Threshold Tuning
"""

def compute_sentiment_metrics(y_true, y_pred_logits):
    """Compute sentiment classification metrics"""
    y_pred = torch.argmax(y_pred_logits, dim=1).cpu().numpy()
    y_true = y_true.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    
    return {
        'sentiment_accuracy': accuracy,
        'sentiment_precision': precision,
        'sentiment_recall': recall,
        'sentiment_f1': f1
    }


def compute_emotion_metrics(y_true, y_pred_logits, thresholds=None):
    """Compute emotion classification metrics (multi-label)"""
    probs = torch.sigmoid(y_pred_logits).cpu().numpy()
    y_true_np = y_true.cpu().numpy()
    
    if thresholds is None:
        thresholds = np.array([0.5] * y_true_np.shape[1])
    else:
        thresholds = np.array(thresholds)
    
    y_pred = (probs >= thresholds).astype(float)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true_np, y_pred, average='samples', zero_division=0
    )
    
    return {
        'emotion_precision': precision,
        'emotion_recall': recall,
        'emotion_f1': f1
    }


def compute_intensity_metrics(y_true, y_pred):
    """Compute intensity regression metrics"""
    y_true = y_true.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    try:
        spearman, _ = spearmanr(y_true, y_pred)
        if np.isnan(spearman):
            spearman = 0.0
    except:
        spearman = 0.0
    
    return {
        'intensity_rmse': rmse,
        'intensity_mae': mae,
        'intensity_spearman': spearman
    }


# ==================== PRECISION-TARGETED THRESHOLD TUNING ====================

def tune_thresholds_for_precision(y_true_tensor, logits_tensor, 
                                  precision_targets=None, grid=None):
    """
    Tune thresholds for each emotion label to achieve target precision
    while maximizing F1 score
    
    Parameters:
    -----------
    y_true_tensor : Tensor
        True labels (batch_size, num_classes)
    logits_tensor : Tensor
        Predicted logits (batch_size, num_classes)
    precision_targets : dict
        Target precision for each label
    grid : array-like
        Threshold values to try
    """
    if grid is None:
        grid = np.linspace(0.05, 0.95, 50)
    
    if precision_targets is None:
        precision_targets = {
            'humor': 0.70,
            'sarcasm': 0.70,
            'offensive': 0.75,
            'motivational': 0.60
        }
    
    probs = torch.sigmoid(logits_tensor).detach().cpu().numpy()
    y_true = y_true_tensor.detach().cpu().numpy().astype(int)
    C = probs.shape[1]
    
    best_thresholds = []
    
    print(f"\n{'='*70}")
    print("Precision-Targeted Threshold Tuning")
    print(f"{'='*70}")
    
    for c, name in enumerate(LABELS):
        target_precision = precision_targets.get(name, 0.70)
        
        best_thr = 0.5
        best_f1 = -1
        best_prec = 0
        best_rec = 0
        
        for t in grid:
            pred = (probs[:, c] >= t).astype(int)
            
            # Calculate metrics
            prec = precision_score(y_true[:, c], pred, zero_division=0)
            rec = recall_score(y_true[:, c], pred, zero_division=0)
            f1 = f1_score(y_true[:, c], pred, zero_division=0)
            
            # Check if precision meets target
            if prec >= target_precision:
                if f1 > best_f1:
                    best_f1 = f1
                    best_thr = t
                    best_prec = prec
                    best_rec = rec
        
        # If no threshold meets target, use the one with highest F1
        if best_f1 == -1:
            for t in grid:
                pred = (probs[:, c] >= t).astype(int)
                f1 = f1_score(y_true[:, c], pred, zero_division=0)
                prec = precision_score(y_true[:, c], pred, zero_division=0)
                rec = recall_score(y_true[:, c], pred, zero_division=0)
                
                if f1 > best_f1:
                    best_f1 = f1
                    best_thr = t
                    best_prec = prec
                    best_rec = rec
        
        best_thresholds.append(best_thr)
        
        # Calculate AUPRC
        ap = average_precision_score(y_true[:, c], probs[:, c])
        
        status = "✓" if best_prec >= target_precision else "⚠"
        print(f"\n{status} {name.upper()}:")
        print(f"  Target Precision: {target_precision:.2f}")
        print(f"  Best Threshold: {best_thr:.3f}")
        print(f"  Precision: {best_prec:.4f}")
        print(f"  Recall: {best_rec:.4f}")
        print(f"  F1: {best_f1:.4f}")
        print(f"  AUPRC: {ap:.4f}")
    
    return np.array(best_thresholds)


# ==================== Per-Label Confusion Matrix ====================

def eval_multilabel(y_true_tensor, logits_tensor, thresholds=None):
    """Evaluate multi-label classification with per-label confusion matrix"""
    probs = torch.sigmoid(logits_tensor).detach().cpu().numpy()
    y_true = y_true_tensor.detach().cpu().numpy().astype(int)
    C = probs.shape[1]
    
    if thresholds is None:
        thresholds = np.array([0.5] * C)
    
    y_pred = (probs >= thresholds).astype(int)
    
    print(f"\n{'='*70}")
    print("Per-Label Evaluation")
    print(f"{'='*70}")
    
    for i, name in enumerate(LABELS):
        tn, fp, fn, tp = confusion_matrix(
            y_true[:, i], y_pred[:, i], labels=[0, 1]
        ).ravel()
        ap = average_precision_score(y_true[:, i], probs[:, i])
        
        print(f"\n=== {name.upper()} ===")
        print(f"TP={tp} FP={fp} FN={fn} TN={tn} | AUPRC={ap:.4f}")
        print(classification_report(y_true[:, i], y_pred[:, i], digits=4))
    
    return y_pred, probs


# ==================== Test Metrics ====================
try:
    print("\n" + "="*70)
    print("Testing enhanced metrics computation...")
    print("="*70)
    
    # Create dummy predictions
    dummy_sentiment_logits = torch.randn(10, cfg['NUM_SENTIMENT_CLASSES'])
    dummy_sentiment_labels = torch.randint(0, cfg['NUM_SENTIMENT_CLASSES'], (10,))
    
    dummy_emotion_logits = torch.randn(10, cfg['NUM_EMOTION_CLASSES'])
    dummy_emotion_labels = torch.randint(0, 2, (10, cfg['NUM_EMOTION_CLASSES'])).float()
    
    dummy_intensity_pred = torch.rand(10)
    dummy_intensity_true = torch.rand(10)
    
    # Compute metrics
    sentiment_metrics = compute_sentiment_metrics(dummy_sentiment_labels, dummy_sentiment_logits)
    emotion_metrics = compute_emotion_metrics(dummy_emotion_labels, dummy_emotion_logits)
    intensity_metrics = compute_intensity_metrics(dummy_intensity_true, dummy_intensity_pred)
    
    print("\n✓ Sentiment Metrics:")
    for key, value in sentiment_metrics.items():
        print(f"  {key}: {value:.4f}")
    
    print("\n✓ Emotion Metrics:")
    for key, value in emotion_metrics.items():
        print(f"  {key}: {value:.4f}")
    
    print("\n✓ Intensity Metrics:")
    for key, value in intensity_metrics.items():
        print(f"  {key}: {value:.4f}")
    
    print("\n" + "="*70)
    print("Part 5 Complete - Enhanced metrics tested successfully")
    print("="*70)
    print("\nKey Changes:")
    print("✓ Precision-targeted threshold tuning")
    print("✓ Per-label confusion matrix evaluation")
    print("✓ AUPRC tracking for each emotion")
    print("✓ Customizable precision targets")

except Exception as e:
    print(f"\n✗ Error in metrics computation: {e}")
    import traceback
    traceback.print_exc()




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 13.8 MB/s eta 0:00:0000:01
ERROR: pip's depend

ModuleNotFoundError: No module named 'iterstrat'

In [ ]:
# ==================== PART 6: ENHANCED TRAINER WITH EARLIER UNFREEZING ====================
"""
Part 6: Enhanced Trainer with Earlier Backbone Unfreezing
"""

class Trainer:
    """Enhanced Trainer for multi-task meme analysis"""
    
    def __init__(self, model, cfg, train_loader, val_loader, device):
        self.model = model
        self.cfg = cfg
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        
        self.optimizer = self.make_optimizer()
        self.scheduler = self.make_scheduler()
        self.scaler = GradScaler() if cfg['USE_AMP'] else None
        
        self.current_epoch = 0
        self.best_metric = -float('inf')
        self.gradient_accumulation_steps = cfg.get('GRADIENT_ACCUMULATION_STEPS', 1)
        
        # Store validation outputs for diagnostics
        self.val_emotion_logits = None
        self.val_emotion_labels = None
        
        print("✓ Trainer initialized")
    
    def make_optimizer(self):
        """Create optimizer with parameter groups"""
        head_params = []
        backbone_params = []
        
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                if 'text_model' in name or 'image_model' in name:
                    backbone_params.append(param)
                else:
                    head_params.append(param)
        
        param_groups = [
            {'params': head_params, 'lr': self.cfg['LR_HEADS']},
        ]
        
        if backbone_params:
            param_groups.append({'params': backbone_params, 'lr': self.cfg['LR_BACKBONE']})
        
        print(f"  Optimizer: {len(head_params)} head params, {len(backbone_params)} backbone params")
        
        return torch.optim.AdamW(param_groups, weight_decay=self.cfg['WEIGHT_DECAY'])
    
    def make_scheduler(self):
        """Create learning rate scheduler"""
        if self.cfg['SCHEDULER'] == 'cosine':
            return torch.optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer, T_max=self.cfg['EPOCHS']
            )
        return None
    
    def train_epoch(self, epoch):
        """Train for one epoch"""
        self.model.train()
        total_loss = 0
        total_sent_loss = 0
        total_emotion_loss = 0
        total_intensity_loss = 0
        
        pbar = tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.cfg['EPOCHS']} [Train]")
        self.optimizer.zero_grad()
        
        for batch_idx, batch in enumerate(pbar):
            # Move to device
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            image = batch['image'].to(self.device)
            sentiment_label = batch['sentiment_label'].to(self.device)
            emotion_labels = batch['emotion_labels'].to(self.device)
            intensity = batch['intensity'].to(self.device)
            
            batch_device = {
                'sentiment_label': sentiment_label,
                'emotion_labels': emotion_labels,
                'intensity': intensity
            }
            
            # Forward pass with AMP
            if self.cfg['USE_AMP']:
                with autocast():
                    outputs = self.model(input_ids, attention_mask, image)
                    loss, l_sent, l_emotion, l_int = combined_loss(outputs, batch_device, self.cfg)
                    loss = loss / self.gradient_accumulation_steps
                
                self.scaler.scale(loss).backward()
                
                if (batch_idx + 1) % self.gradient_accumulation_steps == 0:
                    if self.cfg['GRADIENT_CLIP'] > 0:
                        self.scaler.unscale_(self.optimizer)
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.cfg['GRADIENT_CLIP'])
                    
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                    self.optimizer.zero_grad()
            else:
                outputs = self.model(input_ids, attention_mask, image)
                loss, l_sent, l_emotion, l_int = combined_loss(outputs, batch_device, self.cfg)
                loss = loss / self.gradient_accumulation_steps
                loss.backward()
                
                if (batch_idx + 1) % self.gradient_accumulation_steps == 0:
                    if self.cfg['GRADIENT_CLIP'] > 0:
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.cfg['GRADIENT_CLIP'])
                    
                    self.optimizer.step()
                    self.optimizer.zero_grad()
            
            total_loss += loss.item() * self.gradient_accumulation_steps
            total_sent_loss += l_sent.item()
            total_emotion_loss += l_emotion.item()
            total_intensity_loss += l_int.item()
            
            pbar.set_postfix({
                'loss': f"{loss.item() * self.gradient_accumulation_steps:.4f}",
                'sent': f"{l_sent.item():.4f}",
                'emo': f"{l_emotion.item():.4f}",
                'int': f"{l_int.item():.4f}"
            })
        
        avg_loss = total_loss / len(self.train_loader)
        print(f"\nTrain Loss: {avg_loss:.4f}")
        print(f"  Sentiment: {total_sent_loss/len(self.train_loader):.4f}")
        print(f"  Emotion (ASL): {total_emotion_loss/len(self.train_loader):.4f}")
        print(f"  Intensity: {total_intensity_loss/len(self.train_loader):.4f}")
        
        return avg_loss
    
    def validate(self, epoch):
        """Validate the model"""
        self.model.eval()
        total_loss = 0
        
        all_sentiment_labels = []
        all_sentiment_logits = []
        all_emotion_labels = []
        all_emotion_logits = []
        all_intensity_true = []
        all_intensity_pred = []
        
        pbar = tqdm(self.val_loader, desc=f"Epoch {epoch+1}/{self.cfg['EPOCHS']} [Val]")
        
        with torch.no_grad():
            for batch in pbar:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                image = batch['image'].to(self.device)
                sentiment_label = batch['sentiment_label'].to(self.device)
                emotion_labels = batch['emotion_labels'].to(self.device)
                intensity = batch['intensity'].to(self.device)
                
                batch_device = {
                    'sentiment_label': sentiment_label,
                    'emotion_labels': emotion_labels,
                    'intensity': intensity
                }
                
                outputs = self.model(input_ids, attention_mask, image)
                loss, _, _, _ = combined_loss(outputs, batch_device, self.cfg)
                
                total_loss += loss.item()
                
                all_sentiment_labels.append(sentiment_label)
                all_sentiment_logits.append(outputs['sentiment_logits'])
                all_emotion_labels.append(emotion_labels)
                all_emotion_logits.append(outputs['emotion_logits'])
                all_intensity_true.append(intensity)
                all_intensity_pred.append(outputs['intensity'])
        
        # Aggregate metrics
        all_sentiment_labels = torch.cat(all_sentiment_labels)
        all_sentiment_logits = torch.cat(all_sentiment_logits)
        all_emotion_labels = torch.cat(all_emotion_labels)
        all_emotion_logits = torch.cat(all_emotion_logits)
        all_intensity_true = torch.cat(all_intensity_true)
        all_intensity_pred = torch.cat(all_intensity_pred)
        
        # Store for diagnostics
        self.val_emotion_logits = all_emotion_logits
        self.val_emotion_labels = all_emotion_labels
        
        sentiment_metrics = compute_sentiment_metrics(all_sentiment_labels, all_sentiment_logits)
        emotion_metrics = compute_emotion_metrics(
            all_emotion_labels, all_emotion_logits, 
            thresholds=self.cfg.get('EMO_THRESHOLDS', [0.5, 0.5, 0.6, 0.6])
        )
        intensity_metrics = compute_intensity_metrics(all_intensity_true, all_intensity_pred)
        
        avg_loss = total_loss / len(self.val_loader)
        
        metrics = {
            'val_loss': avg_loss,
            **sentiment_metrics,
            **emotion_metrics,
            **intensity_metrics
        }
        
        print(f"\nValidation Metrics:")
        print(f"  Loss: {avg_loss:.4f}")
        print(f"  Sentiment F1 (macro): {sentiment_metrics['sentiment_f1']:.4f}")
        print(f"  Emotion F1 (samples): {emotion_metrics['emotion_f1']:.4f}")
        print(f"  Intensity RMSE: {intensity_metrics['intensity_rmse']:.4f}")
        
        return metrics
    
    def run_diagnostics(self, epoch):
        """Run diagnostic checks after validation"""
        if self.val_emotion_logits is None or self.val_emotion_labels is None:
            return
        
        print(f"\n{'='*70}")
        print(f"DIAGNOSTIC: Per-Label Analysis (Epoch {epoch+1})")
        print(f"{'='*70}")
        
        # Per-label confusion matrix with current thresholds
        current_thresholds = np.array(self.cfg.get('EMO_THRESHOLDS', [0.5, 0.5, 0.6, 0.6]))
        _ = eval_multilabel(self.val_emotion_labels, self.val_emotion_logits, 
                           thresholds=current_thresholds)
        
        # Precision-targeted threshold tuning
        print(f"\n{'='*70}")
        print("DIAGNOSTIC: Precision-Targeted Threshold Tuning")
        print(f"{'='*70}")
        
        precision_targets = self.cfg.get('PRECISION_TARGETS', {
            'humor': 0.70,
            'sarcasm': 0.70,
            'offensive': 0.75,
            'motivational': 0.60
        })
        
        best_thr = tune_thresholds_for_precision(
            self.val_emotion_labels, 
            self.val_emotion_logits,
            precision_targets=precision_targets
        )
        
        self.cfg['EMO_THRESHOLDS'] = best_thr.tolist()
        print(f"\n✓ Updated thresholds: {self.cfg['EMO_THRESHOLDS']}")
        
        # Re-run evaluation with tuned thresholds
        print(f"\n{'='*70}")
        print("DIAGNOSTIC: Results with Tuned Thresholds")
        print(f"{'='*70}")
        _ = eval_multilabel(self.val_emotion_labels, self.val_emotion_logits, 
                           thresholds=best_thr)
    
    def fit(self):
        """Train the model"""
        print(f"\n{'='*70}")
        print(f"Starting Training for {self.cfg['EPOCHS']} epochs")
        print(f"{'='*70}\n")
        
        for epoch in range(self.cfg['EPOCHS']):
            self.current_epoch = epoch
            
            # CHANGED: Earlier backbone unfreezing at epoch 2 (was 3)
            if epoch == self.cfg.get('UNFREEZE_BACKBONE_EPOCH', 2):
                print(f"\n{'='*50}")
                print(f"⚡ Unfreezing backbone at epoch {epoch+1} (EARLIER!)")
                print(f"{'='*50}\n")
                self.model.unfreeze_backbone(layers_to_unfreeze=2)
                self.optimizer = self.make_optimizer()
                self.scheduler = self.make_scheduler()
            
            # Train
            train_loss = self.train_epoch(epoch)
            
            # Validate
            val_metrics = self.validate(epoch)
            
            # Run diagnostics (every 2 epochs or last epoch)
            if epoch % 2 == 0 or epoch == self.cfg['EPOCHS'] - 1:
                self.run_diagnostics(epoch)
            
            # Scheduler step
            if self.scheduler:
                self.scheduler.step()
            
            # Composite metric for checkpoint (macro F1 preferred)
            composite_metric = (
                val_metrics['sentiment_f1'] + 
                val_metrics['emotion_f1'] - 
                val_metrics['intensity_rmse']
            )
            
            # Save best checkpoint
            if composite_metric > self.best_metric:
                self.best_metric = composite_metric
                checkpoint_path = os.path.join(self.cfg['CHECKPOINT_PATH'], 'best_model.pt')
                save_checkpoint({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'best_metric': self.best_metric,
                    'metrics': val_metrics,
                    'config': self.cfg
                }, checkpoint_path)
                print(f"✓ Saved best model (composite: {composite_metric:.4f})")
            
            print(f"\n{'='*70}\n")
        
        print("Training completed!")
        return self.best_metric


# ==================== Initialize Trainer ====================
try:
    print("\nInitializing enhanced trainer...")
    trainer = Trainer(model, cfg, train_loader, val_loader, device)
    
    print("\n" + "="*70)
    print("Part 6 Complete - Enhanced Trainer ready")
    print("="*70)
    print("\nKey Changes:")
    print("✓ Backbone unfreezing moved to epoch 2 (was 3)")
    print("✓ Precision-targeted threshold tuning integrated")
    print("✓ ASL loss tracked separately in training")
    print("✓ Macro F1 used for early stopping")
    print("\nRun Part 7 to start training")

except Exception as e:
    print(f"\n✗ Error initializing trainer: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 7: TRAINING WITH DIAGNOSTICS ====================
"""
Part 7: Start Training with Comprehensive Diagnostics
"""

print("\n" + "="*70)
print("Part 7: Training with Enhanced Diagnostics")
print("="*70)

# ==================== DIAGNOSTIC 1: Tiny-batch Overfit & Gradient Sanity ====================
print("\n" + "="*70)
print("DIAGNOSTIC 1: Tiny-batch Overfit & Gradient Sanity Check")
print("="*70)

tiny_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
model.train()
opt = torch.optim.AdamW(model.parameters(), lr=1e-4)

print("\nRunning 200 steps on tiny batch to verify model can overfit...")
for step, batch in enumerate(tiny_loader):
    if step > 200:
        break
    
    # Move batch to device
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    image = batch['image'].to(device)
    
    batch_device = {
        'sentiment_label': batch['sentiment_label'].to(device),
        'emotion_labels': batch['emotion_labels'].to(device),
        'intensity': batch['intensity'].to(device)
    }
    
    outputs = model(input_ids, attention_mask, image)
    total, l_sent, l_emo, l_int = combined_loss(outputs, batch_device, cfg)
    
    opt.zero_grad(set_to_none=True)
    total.backward()
    
    # Gradient NaN/inf check
    bad_grad = False
    for p in model.parameters():
        if p.grad is not None:
            if torch.isnan(p.grad).any() or torch.isinf(p.grad).any():
                bad_grad = True
                break
    
    if bad_grad:
        print("✗ Bad gradient detected (NaN/Inf). Stop & inspect loss/labels.")
        break
    
    opt.step()
    
    if step % 20 == 0:
        print(f"{step:03d}: total={total.item():.4f} | sent={l_sent.item():.4f} | emo(ASL)={l_emo.item():.4f} | int={l_int.item():.4f}")

print("✓ Tiny-batch sanity done. Model can learn from data.")
print("Emotion loss (ASL) should have decreased. If not, check labels/loss function.")

# Reinitialize model and trainer for actual training
print("\nReinitializing model for full training...")
model = EnhancedFusionModel(cfg).to(device)
trainer = Trainer(model, cfg, train_loader, val_loader, device)

# ==================== Start Full Training ====================
try:
    print("\n" + "="*70)
    print("STARTING FULL TRAINING")
    print("="*70)
    
    best_metric = trainer.fit()
    
    print(f"\n{'='*70}")
    print(f"Training Completed!")
    print(f"Best composite metric: {best_metric:.4f}")
    print(f"Best model saved to: {cfg['CHECKPOINT_PATH']}/best_model.pt")
    print(f"{'='*70}")
    
    # Save model card
    model_card = f"""# Enhanced Meme Sentiment and Emotion Analysis Model

## Model Details
- **Text Encoder**: {cfg['TEXT_MODEL']}
- **Image Encoder**: {cfg['IMAGE_MODEL']}
- **Architecture**: Multi-modal fusion with cross-attention
- **Tasks**:
  - Sentiment Classification (5 classes)
  - Emotion Classification (4 binary classes: humor, sarcasm, offensive, motivational)
  - Intensity Regression (continuous 0-1)

## Training Configuration
- Epochs: {cfg['EPOCHS']}
- Batch Size: {cfg['BATCH_SIZE']}
- Gradient Accumulation: {cfg.get('GRADIENT_ACCUMULATION_STEPS', 1)}
- Learning Rate (Heads): {cfg['LR_HEADS']}
- Learning Rate (Backbone): {cfg['LR_BACKBONE']}
- Mixed Precision: {cfg['USE_AMP']}

## Enhanced Imbalance Handling
- **Stratified Split**: Multi-label stratification on offensive, motivational, and sentiment
- **Loss Functions**:
  - Sentiment: Class-Balanced Cross-Entropy (weight: {cfg['LOSS_WEIGHTS']['sentiment']})
  - Emotions: Asymmetric Loss (ASL) with γ_neg={cfg['ASL_GAMMA_NEG']}, clip={cfg['ASL_CLIP']}
  - Intensity: Smooth L1 Loss (weight: {cfg['LOSS_WEIGHTS']['intensity']})
- **Oversampling**: Motivational samples oversampled {cfg.get('MOTIVATIONAL_OVERSAMPLE_FACTOR', 5.0)}x
- **Prior Adjustment**: Emotion logits adjusted by training priors
- **Backbone Unfreezing**: Earlier at epoch {cfg.get('UNFREEZE_BACKBONE_EPOCH', 2)}

## Performance
- Best Composite Metric: {best_metric:.4f}

## Dataset
- Training Samples: {len(train_df)}
- Validation Samples: {len(val_df)}
- Train/val priors matched via stratified split
- Emotions: offensive≈{LABEL_PRIORS['offensive_pos_rate']:.1%}, motivational≈{LABEL_PRIORS['motivational_pos_rate']:.1%}

## Tuned Emotion Thresholds (Precision-Targeted)
- Humor: {cfg['EMO_THRESHOLDS'][0]:.3f}
- Sarcasm: {cfg['EMO_THRESHOLDS'][1]:.3f}
- Offensive: {cfg['EMO_THRESHOLDS'][2]:.3f}
- Motivational: {cfg['EMO_THRESHOLDS'][3]:.3f}

## Key Improvements
✓ Mitigated imbalance using ASL (γ_neg=6) for emotions, class-balanced CE for sentiment
✓ Oversampling for motivational, precision-targeted thresholds
✓ Earlier unfreeze (epoch 2) + higher sentiment task weight reduced neutral collapse
✓ Stratified split ensures train/val prior matching
"""
    
    model_card_path = os.path.join(cfg['CHECKPOINT_PATH'], 'model_card.md')
    with open(model_card_path, 'w') as f:
        f.write(model_card)
    
    print(f"\n✓ Model card saved to: {model_card_path}")
    
    print("\n" + "="*70)
    print("Part 7 Complete - Training finished")
    print("="*70)

except KeyboardInterrupt:
    print("\n\n⚠ Training interrupted by user")
except Exception as e:
    print(f"\n✗ Error during training: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 8: ENHANCED INFERENCE ====================
"""
Part 8: Inference with Tuned Thresholds
"""

class Predictor:
    """Enhanced predictor using tuned thresholds"""
    
    def __init__(self, checkpoint_path: str, device: str = 'cuda'):
        print("Initializing predictor...")
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        
        # Load checkpoint and config
        checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
        self.cfg = checkpoint['config']
        
        # Load model
        try:
            print("  Loading EnhancedFusionModel...")
            self.model = EnhancedFusionModel(self.cfg).to(self.device)
            print("  ✓ Using EnhancedFusionModel")
        except NameError:
            raise RuntimeError("EnhancedFusionModel not found! Run Part 3 first.")
        
        # Load model weights
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        
        print(f"✓ Model loaded from epoch {checkpoint['epoch']} with best metric {checkpoint['best_metric']:.4f}")
        
        # Load tokenizer and image transform
        self.tokenizer = get_tokenizer(self.cfg)
        self.transform = get_image_transform(self.cfg, is_train=False)
        
        # Get tuned thresholds
        self.emotion_thresholds = self.cfg.get('EMO_THRESHOLDS', [0.5, 0.5, 0.6, 0.6])
        print(f"✓ Using tuned thresholds: {self.emotion_thresholds}")
        
        # Create reverse mappings
        self.sentiment_map_rev = {
            0: 'very_positive',
            1: 'positive',
            2: 'neutral',
            3: 'negative',
            4: 'very_negative'
        }
        
        self.emotion_labels = ['humor', 'sarcasm', 'offensive', 'motivational']
        
        print("✓ Predictor initialized successfully")
    
    def predict(self, text: str, image_path: str) -> Dict:
        """Predict with tuned thresholds"""
        # Preprocess Text
        encoding = self.tokenizer(
            text,
            max_length=self.cfg['MAX_LEN'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        
        # Preprocess Image
        try:
            image = Image.open(image_path).convert('RGB')
            image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        except Exception as e:
            print(f"Could not load image: {e}. Using blank image.")
            image_tensor = torch.zeros(1, 3, self.cfg['IMG_SIZE'], self.cfg['IMG_SIZE']).to(self.device)
        
        # Run Inference
        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask, image_tensor)
        
        # Post-process Outputs
        sentiment_pred_idx = torch.argmax(outputs['sentiment_logits'], dim=1).item()
        sentiment_label = self.sentiment_map_rev.get(sentiment_pred_idx, "unknown")
        sentiment_probs = torch.softmax(outputs['sentiment_logits'], dim=1).squeeze().cpu().numpy()
        
        # Use tuned thresholds for emotions
        emotion_probs = torch.sigmoid(outputs['emotion_logits']).squeeze().cpu().numpy()
        emotion_binary = (emotion_probs >= np.array(self.emotion_thresholds)).astype(int)
        
        emotion_preds = {
            label: {
                'probability': f"{prob:.2%}",
                'predicted': bool(binary),
                'threshold': f"{thr:.3f}"
            }
            for label, prob, binary, thr in zip(
                self.emotion_labels, 
                emotion_probs, 
                emotion_binary,
                self.emotion_thresholds
            )
        }
        
        intensity_score = outputs['intensity'].item()
        
        return {
            'sentiment': sentiment_label,
            'sentiment_confidence': f"{sentiment_probs[sentiment_pred_idx]:.2%}",
            'sentiment_probs': {
                self.sentiment_map_rev[i]: f"{p:.2%}" 
                for i, p in enumerate(sentiment_probs)
            },
            'emotions': emotion_preds,
            'intensity_score': f"{intensity_score:.4f}"
        }
    
    def predict_batch(self, text_list, image_path_list):
        """Predict for multiple samples"""
        results = []
        for text, img_path in zip(text_list, image_path_list):
            result = self.predict(text, img_path)
            results.append(result)
        return results


# ==================== Run Inference Example ====================
try:
    CHECKPOINT_DIR = "/kaggle/working/checkpoints"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    BEST_MODEL_PATH = os.path.join(CHECKPOINT_DIR, 'best_model.pt')
    
    if not os.path.exists(BEST_MODEL_PATH):
        print(f"✗ Checkpoint not found at {BEST_MODEL_PATH}")
        print("  Please train the model first (run Part 7)")
        raise FileNotFoundError(f"Model checkpoint not found: {BEST_MODEL_PATH}")
    
    print("\n" + "="*70)
    print("INITIALIZING ENHANCED PREDICTOR")
    print("="*70)
    
    predictor = Predictor(checkpoint_path=BEST_MODEL_PATH, device=DEVICE)
    
    print("\n" + "="*70)
    print("RUNNING SAMPLE PREDICTIONS")
    print("="*70)
    
    # Get column names
    image_col_options = ['image_name', 'image', 'img_name', 'filename', 'Unnamed: 0']
    image_col = next((col for col in image_col_options if col in val_df.columns), val_df.columns[0])
    
    text_col_options = ['text', 'ocr', 'ocr_text', 'caption', 'OCR_extracted_text']
    text_col = next((col for col in text_col_options if col in val_df.columns), None)
    
    # Select 5 random samples
    import random
    random.seed(42)
    sample_indices = random.sample(range(len(val_df)), min(5, len(val_df)))
    
    for idx in sample_indices:
        sample_row = val_df.iloc[idx]
        sample_text = str(sample_row.get(text_col, '')) if text_col else ''
        sample_image_name = str(sample_row.get(image_col, f'{idx}.jpg'))
        
        if not any(sample_image_name.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
            sample_image_name = f"{sample_image_name}.jpg"
        
        sample_image_path = os.path.join(val_img_dir, sample_image_name)
        
        print(f"\n{'='*70}")
        print(f"Sample {idx}:")
        print(f"  Text: '{sample_text[:100]}...'")
        print("="*70)
        
        if not os.path.exists(sample_image_path):
            print(f"⚠️ Image not found: {sample_image_path}")
            continue
        
        # Make prediction
        result = predictor.predict(text=sample_text, image_path=sample_image_path)
        
        print("\nPrediction Result:")
        print(f"  Sentiment: {result['sentiment']} (confidence: {result['sentiment_confidence']})")
        print(f"  Intensity: {result['intensity_score']}")
        print("  Emotions (with tuned thresholds):")
        for emotion, info in result['emotions'].items():
            pred_str = "✓ YES" if info['predicted'] else "✗ NO"
            print(f"    {emotion.capitalize()}: {info['probability']} {pred_str} (threshold: {info['threshold']})")
    
    print("\n" + "="*70)
    print("Part 8 Complete - Enhanced inference pipeline tested")
    print("="*70)

except FileNotFoundError as e:
    print(f"\n✗ {e}")
except Exception as e:
    print(f"\n✗ Error during inference: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 9: DETAILED EVALUATION ====================
"""
Part 9: Comprehensive Evaluation and Error Analysis
"""

from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def run_full_evaluation(predictor, val_loader):
    """Run model on entire validation set"""
    predictor.model.eval()
    all_preds = []
    
    pbar = tqdm(val_loader, desc="Running Full Evaluation")
    
    with torch.no_grad():
        for batch in pbar:
            input_ids = batch['input_ids'].to(predictor.device)
            attention_mask = batch['attention_mask'].to(predictor.device)
            image = batch['image'].to(predictor.device)
            
            outputs = predictor.model(input_ids, attention_mask, image)
            
            sentiment_preds = torch.argmax(outputs['sentiment_logits'], dim=1).cpu().numpy()
            
            # Use tuned thresholds for emotions
            emotion_probs = torch.sigmoid(outputs['emotion_logits']).cpu().numpy()
            emotion_thresholds = np.array(predictor.emotion_thresholds)
            emotion_preds = (emotion_probs >= emotion_thresholds).astype(int)
            
            intensity_preds = outputs['intensity'].cpu().numpy().flatten()
            
            sentiment_true = batch['sentiment_label'].cpu().numpy()
            emotion_true = batch['emotion_labels'].int().cpu().numpy()
            intensity_true = batch['intensity'].cpu().numpy().flatten()
            
            for i in range(len(input_ids)):
                all_preds.append({
                    'sentiment_pred': sentiment_preds[i],
                    'sentiment_true': sentiment_true[i],
                    'emotion_pred': emotion_preds[i],
                    'emotion_true': emotion_true[i],
                    'intensity_pred': intensity_preds[i],
                    'intensity_true': intensity_true[i]
                })
    
    results_df = pd.DataFrame(all_preds)
    results_df['sentiment_pred_label'] = results_df['sentiment_pred'].map(predictor.sentiment_map_rev)
    results_df['sentiment_true_label'] = results_df['sentiment_true'].map(predictor.sentiment_map_rev)
    
    return results_df


try:
    print("\n" + "="*70)
    print("STARTING DETAILED EVALUATION")
    print("="*70)
    
    if 'predictor' not in locals() or predictor is None:
        print("✗ Predictor not found. Please run Part 8 first.")
        raise NameError("Predictor not initialized")
    
    print("\nStarting full evaluation on validation set...")
    eval_results_df = run_full_evaluation(predictor, val_loader)
    
    # Merge with original validation dataframe
    full_analysis_df = pd.concat([val_df.reset_index(drop=True), eval_results_df], axis=1)
    full_analysis_df['sentiment_correct'] = full_analysis_df['sentiment_pred'] == full_analysis_df['sentiment_true']
    full_analysis_df['intensity_error'] = np.abs(
        full_analysis_df['intensity_pred'] - full_analysis_df['intensity_true']
    )
    
    print(f"\n✓ Full evaluation complete. Analysis DataFrame shape: {full_analysis_df.shape}")
    
    # ==================== Performance Summary ====================
    print("\n" + "="*70)
    print("PERFORMANCE SUMMARY")
    print("="*70)
    
    overall_acc = full_analysis_df['sentiment_correct'].mean()
    avg_intensity_error = full_analysis_df['intensity_error'].mean()
    
    print(f"\nOverall Metrics:")
    print(f"  Sentiment Accuracy: {overall_acc:.2%}")
    print(f"  Average Intensity Error: {avg_intensity_error:.4f}")
    
    print(f"\nPer-Class Sentiment Accuracy:")
    for sent_label in full_analysis_df['sentiment_true_label'].unique():
        class_df = full_analysis_df[full_analysis_df['sentiment_true_label'] == sent_label]
        if len(class_df) > 0:
            class_acc = class_df['sentiment_correct'].mean()
            print(f"  {sent_label}: {class_acc:.2%} ({len(class_df)} samples)")
    
    # ==================== Confusion Matrix ====================
    print("\n" + "="*70)
    print("CONFUSION MATRIX")
    print("="*70)
    
    sentiment_labels = sorted(full_analysis_df['sentiment_true_label'].unique())
    cm = confusion_matrix(
        full_analysis_df['sentiment_true_label'],
        full_analysis_df['sentiment_pred_label'],
        labels=sentiment_labels
    )
    
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=sentiment_labels)
    fig, ax = plt.subplots(figsize=(10, 10))
    disp.plot(cmap=plt.cm.Blues, ax=ax, xticks_rotation='vertical', values_format='d')
    plt.title('Sentiment Classification Confusion Matrix', fontsize=14, pad=20)
    plt.tight_layout()
    
    cm_path = '/kaggle/working/confusion_matrix.png'
    plt.savefig(cm_path, dpi=300, bbox_inches='tight')
    print(f"✓ Confusion matrix saved to {cm_path}")
    plt.show()
    
    # ==================== Save Results ====================
    results_csv_path = '/kaggle/working/detailed_evaluation_results.csv'
    full_analysis_df.to_csv(results_csv_path, index=False)
    print(f"\n✓ Detailed results saved to {results_csv_path}")
    
    print("\n" + "="*70)
    print("Part 9 Complete - Detailed evaluation finished")
    print("="*70)

except NameError as e:
    print(f"\n✗ {e}")
    print("  Run Part 8 first")
except Exception as e:
    print(f"\n✗ Error during detailed evaluation: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 10: FEATURE VISUALIZATION ====================
"""
Part 10: t-SNE Feature Visualization
"""

try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    
    print("\n" + "="*70)
    print("DIAGNOSTIC 5: Feature Visualization (t-SNE for Emotion Separability)")
    print("="*70)
    
    if 'predictor' not in locals() or predictor is None:
        print("✗ Predictor not found. Please run Part 8 first.")
        raise NameError("Predictor not initialized")
    
    model.eval()
    all_feat, all_offensive = [], []
    feature_key = 'text_features'
    
    print(f"\nExtracting {feature_key} from validation set...")
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Extracting features"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            image = batch['image'].to(device)
            
            out = model(input_ids, attention_mask, image)
            feat = out[feature_key]
            
            all_feat.append(feat.detach().cpu().numpy())
            all_offensive.append(batch['emotion_labels'][:, 2].detach().cpu().numpy())  # offensive is index 2
    
    X = np.concatenate(all_feat, 0)
    y = np.concatenate(all_offensive, 0)
    
    print(f"Feature matrix shape: {X.shape}")
    print(f"Offensive label distribution: {np.sum(y==1)} offensive, {np.sum(y==0)} non-offensive")
    
    print("\nRunning t-SNE dimensionality reduction...")
    Z = TSNE(n_components=2, perplexity=30, learning_rate='auto', 
             init='pca', random_state=42).fit_transform(X)
    
    plt.figure(figsize=(10, 8))
    plt.scatter(Z[y==0, 0], Z[y==0, 1], s=6, alpha=0.5, label='non-offensive', c='blue')
    plt.scatter(Z[y==1, 0], Z[y==1, 1], s=12, alpha=0.8, label='offensive', c='red')
    plt.legend()
    plt.title(f"t-SNE of {feature_key} — Offensive Separability", fontsize=14)
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    tsne_path = '/kaggle/working/tsne_offensive_separability.png'
    plt.savefig(tsne_path, dpi=300, bbox_inches='tight')
    print(f"✓ t-SNE plot saved to {tsne_path}")
    plt.show()
    
    print("\n" + "="*70)
    print("Part 10 Complete - All diagnostics finished!")
    print("="*70)
    
    print("\nGenerated Files:")
    print("  1. /kaggle/working/checkpoints/best_model.pt")
    print("  2. /kaggle/working/checkpoints/model_card.md")
    print("  3. /kaggle/working/confusion_matrix.png")
    print("  4. /kaggle/working/detailed_evaluation_results.csv")
    print("  5. /kaggle/working/tsne_offensive_separability.png")
    print("  6. /kaggle/working/label_priors.json")
    
    print("\n" + "="*70)
    print("ALL PARTS COMPLETE! 🎉")
    print("="*70)
    
    print("\n📊 SUMMARY OF ENHANCEMENTS:")
    print("="*70)
    print("✓ Stratified multi-label split (not last 1500 rows)")
    print("✓ ASL loss for emotions (γ_neg=6, clip=0.05)")
    print("✓ Class-balanced CE for sentiment")
    print("✓ WeightedRandomSampler for motivational (5x)")
    print("✓ Precision-targeted threshold tuning")
    print("✓ Earlier backbone unfreezing (epoch 2)")
    print("✓ Prior logit adjustment for emotions")
    print("✓ Label normalization and proper extension handling")
    print("✓ Comprehensive diagnostics and evaluation")
    print("="*70)

except Exception as e:
    print(f"\n✗ Error during t-SNE visualization: {e}")
    import traceback
    traceback.print_exc()

In [2]:
# ==================== PART 0: ENV & INSTALLS (FIXED) ====================
# Use quiet installs but correct package names. Add an import guard for iterative-stratification.

!pip -q install "transformers>=4.40.0" "accelerate" "torch" "timm" "scikit-learn" "pandas" "matplotlib" "huggingface_hub>=0.18.0" "gdown"
!pip -q install "iterative-stratification"  # <-- CORRECT PACKAGE (not 'iterstrat')

# Robust import guard (works in Kaggle/Colab without needing a restart)
import sys, subprocess
def _safe_import_iterstrat():
    try:
        from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
        return MultilabelStratifiedShuffleSplit
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "iterative-stratification"])
        from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
        return MultilabelStratifiedShuffleSplit

MultilabelStratifiedShuffleSplit = _safe_import_iterstrat()

print("✓ Environment ready. Iterative stratification is available.")


✓ Environment ready. Iterative stratification is available.


In [3]:
"""
Enhanced Multi-modal Meme Analysis with Advanced Imbalance Handling
=====================================================================

Key Improvements:
- Stratified multi-label split (instead of last 1500 rows)
- ASL loss for emotions (γ_neg=6, clip=0.05)
- Class-balanced CE for sentiment
- WeightedRandomSampler for motivational oversampling
- Precision-targeted threshold tuning
- Earlier backbone unfreezing (epoch 2)
- Label priors tracking and adjustment
"""

# ==================== A) ENHANCED DATA PREPARATION CELL ====================
# This section includes stratified splitting, proper label normalization,
# and saving label priors for training


!gdown 1jEJ2nf5CDJknq80ogzU-Uyz7jbBi-1LZ --fuzzy
!unzip *.zip
!gdown --folder 19yaav8ORSVj9DeJUaHKq1H3HtVnkClBw --remaining-ok

import zipfile
import os
import json

# Path to the zip file
zip_path = '/kaggle/working/Memotion 3/memotion3.zip'
extract_to = '/kaggle/working/'
password = b'memotion3taskaaai@22'

# Unzip the file with password
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to, pwd=password)
print(f"Successfully extracted to: {extract_to}")

print("\nExtracted files/folders:")
for item in os.listdir(extract_to):
    print(f" - {item}")

import pandas as pd
import numpy as np
import shutil
from tqdm import tqdm
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

# --- 1. DEFINE PATHS AND PARAMETERS ---
ORIGINAL_TRAIN_IMG_DIR = '/kaggle/working/trainImages/'
ORIGINAL_CSV_PATH = '/kaggle/working/memotion3/train.csv'
IMAGE_FILENAME_COLUMN = 'Unnamed: 0'  # Will be auto-detected
VALIDATION_SPLIT_RATIO = 0.15  # ~1500 samples if total ~10000

# --- 2. DEFINE OUTPUT DIRECTORIES ---
OUTPUT_BASE_DIR = '/kaggle/working/'
NEW_VAL_DIR = os.path.join(OUTPUT_BASE_DIR, 'validation_images/')
NEW_TRAIN_DIR = os.path.join(OUTPUT_BASE_DIR, 'new_train_images/')

os.makedirs(NEW_VAL_DIR, exist_ok=True)
os.makedirs(NEW_TRAIN_DIR, exist_ok=True)
print(f"Created directories:\n- {NEW_VAL_DIR}\n- {NEW_TRAIN_DIR}")

# --- 3. LOAD AND NORMALIZE LABELS ---
print("\nLoading and normalizing CSV file...")
df = pd.read_csv(ORIGINAL_CSV_PATH)
print(f"CSV columns: {list(df.columns)}")
print(f"\nOriginal dataframe shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

# Auto-detect image column
possible_image_cols = ['image_name', 'image', 'img_name', 'filename', 'Unnamed: 0']
IMAGE_FILENAME_COLUMN = next((col for col in possible_image_cols if col in df.columns), df.columns[0])
print(f"\n✅ Detected image column: '{IMAGE_FILENAME_COLUMN}'")

# --- 4. LABEL NORMALIZATION ---
print("\n" + "="*70)
print("Label Normalization")
print("="*70)

# Normalize string labels (lowercase + strip)
for col in ['offensive', 'motivational', 'humour', 'humor', 'sarcastic', 'sarcasm', 'overall', 'sentiment']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.lower().str.strip()

# Create binary emotion columns for stratification
def create_binary_label(value, positive_values):
    """Convert label to binary"""
    if pd.isna(value) or value in ['nan', 'none', '']:
        return 0
    return 1 if value in positive_values else 0

# Binary columns for emotions
if 'offensive' in df.columns:
    df['offensive_bin'] = df['offensive'].apply(
        lambda x: create_binary_label(x, ['slight', 'very_offensive', 'hateful_offensive'])
    )
else:
    df['offensive_bin'] = 0

if 'motivational' in df.columns:
    df['motivational_bin'] = df['motivational'].apply(
        lambda x: create_binary_label(x, ['motivational'])
    )
else:
    df['motivational_bin'] = 0

if 'humour' in df.columns or 'humor' in df.columns:
    humor_col = 'humour' if 'humour' in df.columns else 'humor'
    df['humor_bin'] = df[humor_col].apply(
        lambda x: create_binary_label(x, ['funny', 'very_funny', 'hilarious'])
    )
else:
    df['humor_bin'] = 0

if 'sarcastic' in df.columns or 'sarcasm' in df.columns:
    sarcasm_col = 'sarcastic' if 'sarcastic' in df.columns else 'sarcasm'
    df['sarcasm_bin'] = df[sarcasm_col].apply(
        lambda x: create_binary_label(x, ['general', 'twisted_meaning', 'very_twisted'])
    )
else:
    df['sarcasm_bin'] = 0

print("\n✅ Binary emotion columns created for stratification")

# --- 5. STRATIFIED MULTI-LABEL SPLIT ---
print("\n" + "="*70)
print("Stratified Multi-Label Split")
print("="*70)

# Prepare multi-label matrix for stratification
# Primary: offensive_bin, motivational_bin (key imbalanced labels)
# Secondary: sentiment (if available)
stratify_columns = ['offensive_bin', 'motivational_bin']

# Add sentiment as stratification factor if available
sentiment_col = None
for col in ['overall', 'sentiment', 'overall_sentiment']:
    if col in df.columns:
        sentiment_col = col
        # Convert sentiment to numeric for stratification
        sentiment_map = {
            'very_positive': 0, 'positive': 1, 'neutral': 2, 
            'negative': 3, 'very_negative': 4
        }
        df['sentiment_num'] = df[col].map(sentiment_map).fillna(2).astype(int)
        stratify_columns.append('sentiment_num')
        break

print(f"Stratifying by: {stratify_columns}")

# Create stratification matrix
y_stratify = df[stratify_columns].values

# Perform stratified split
msss = MultilabelStratifiedShuffleSplit(
    n_splits=1, 
    test_size=VALIDATION_SPLIT_RATIO, 
    random_state=42
)

train_idx, val_idx = next(msss.split(df, y_stratify))
train_df = df.iloc[train_idx].reset_index(drop=True)
val_df = df.iloc[val_idx].reset_index(drop=True)

print(f"\n📊 Stratified Split Results:")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")

# Verify stratification quality
print("\n📊 Label Distribution Comparison:")
for col in stratify_columns:
    train_pos = train_df[col].sum()
    val_pos = val_df[col].sum()
    train_rate = train_pos / len(train_df)
    val_rate = val_pos / len(val_df)
    print(f"\n{col}:")
    print(f"  Train: {train_pos}/{len(train_df)} = {train_rate:.2%}")
    print(f"  Val:   {val_pos}/{len(val_df)} = {val_rate:.2%}")
    print(f"  Difference: {abs(train_rate - val_rate):.2%}")

# --- 6. CALCULATE AND SAVE LABEL PRIORS ---
print("\n" + "="*70)
print("Calculating Label Priors for Training")
print("="*70)

label_priors = {}

# Emotion priors (for ASL adjustment)
label_priors['offensive_pos_rate'] = float(train_df['offensive_bin'].sum() / len(train_df))
label_priors['motivational_pos_rate'] = float(train_df['motivational_bin'].sum() / len(train_df))
label_priors['humor_pos_rate'] = float(train_df['humor_bin'].sum() / len(train_df))
label_priors['sarcasm_pos_rate'] = float(train_df['sarcasm_bin'].sum() / len(train_df))

print(f"\nEmotion Positive Rates:")
print(f"  Offensive: {label_priors['offensive_pos_rate']:.2%}")
print(f"  Motivational: {label_priors['motivational_pos_rate']:.2%}")
print(f"  Humor: {label_priors['humor_pos_rate']:.2%}")
print(f"  Sarcasm: {label_priors['sarcasm_pos_rate']:.2%}")

# Sentiment class distribution (for class-balanced weights)
if sentiment_col:
    sentiment_dist = train_df['sentiment_num'].value_counts().sort_index().to_dict()
    label_priors['sentiment_distribution'] = sentiment_dist
    
    # Calculate effective number of samples for class-balanced weights
    # Formula: (1 - β^n) / (1 - β) where β = (N-1)/N
    N = len(train_df)
    beta = (N - 1) / N
    
    sentiment_weights = {}
    for class_id, count in sentiment_dist.items():
        effective_num = (1 - beta**count) / (1 - beta)
        sentiment_weights[int(class_id)] = (1 - beta) * N / effective_num
    
    # Normalize weights
    weight_sum = sum(sentiment_weights.values())
    sentiment_weights = {k: v / weight_sum * len(sentiment_weights) 
                        for k, v in sentiment_weights.items()}
    
    label_priors['sentiment_cb_weights'] = sentiment_weights
    
    print(f"\nSentiment Distribution:")
    for class_id, count in sorted(sentiment_dist.items()):
        sentiment_labels = {0: 'very_positive', 1: 'positive', 2: 'neutral', 
                          3: 'negative', 4: 'very_negative'}
        print(f"  {sentiment_labels[class_id]}: {count} ({count/N:.2%})")
    
    print(f"\nClass-Balanced Weights:")
    for class_id, weight in sorted(sentiment_weights.items()):
        sentiment_labels = {0: 'very_positive', 1: 'positive', 2: 'neutral', 
                          3: 'negative', 4: 'very_negative'}
        print(f"  {sentiment_labels[class_id]}: {weight:.4f}")

# Save priors to JSON
priors_path = os.path.join(OUTPUT_BASE_DIR, 'label_priors.json')
with open(priors_path, 'w') as f:
    json.dump(label_priors, f, indent=2)
print(f"\n✅ Label priors saved to: {priors_path}")

# --- 7. COPY IMAGE FILES WITH EXTENSION HANDLING ---
def copy_images_with_extension(df_subset, dest_dir, source_dir, image_col):
    """Copy images with proper extension handling"""
    copied = 0
    missing = 0
    
    for idx in tqdm(df_subset[image_col]):
        # Convert to string and handle extensions
        filename = str(idx)
        
        # Add extension if missing
        if not any(filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
            # Try common extensions
            for ext in ['.jpg', '.jpeg', '.png']:
                test_path = os.path.join(source_dir, f"{filename}{ext}")
                if os.path.exists(test_path):
                    filename = f"{filename}{ext}"
                    break
            else:
                # Default to .jpg
                filename = f"{filename}.jpg"
        
        source_path = os.path.join(source_dir, filename)
        
        if os.path.exists(source_path):
            destination_path = os.path.join(dest_dir, filename)
            shutil.copy(source_path, destination_path)
            copied += 1
        else:
            missing += 1
            if missing <= 5:
                print(f"Missing: {source_path}")
    
    return copied, missing

print(f"\n📁 Copying validation images...")
copied_val, missing_val = copy_images_with_extension(
    val_df, NEW_VAL_DIR, ORIGINAL_TRAIN_IMG_DIR, IMAGE_FILENAME_COLUMN
)

print(f"\n📁 Copying training images...")
copied_train, missing_train = copy_images_with_extension(
    train_df, NEW_TRAIN_DIR, ORIGINAL_TRAIN_IMG_DIR, IMAGE_FILENAME_COLUMN
)

# --- 8. SAVE CSV FILES ---
new_train_csv_path = os.path.join(OUTPUT_BASE_DIR, 'train_split.csv')
new_val_csv_path = os.path.join(OUTPUT_BASE_DIR, 'validation_split.csv')

train_df.to_csv(new_train_csv_path, index=False)
val_df.to_csv(new_val_csv_path, index=False)

print(f"\n✅ Successfully created stratified split datasets!")
print(f"Training CSV saved to: {new_train_csv_path}")
print(f"Validation CSV saved to: {new_val_csv_path}")

# --- 9. VERIFICATION ---
print("\n📊 Final Verification:")
print(f"Training CSV rows: {len(train_df)}")
print(f"Validation CSV rows: {len(val_df)}")
print(f"Images copied to training directory: {copied_train} (missing: {missing_train})")
print(f"Images copied to validation directory: {copied_val} (missing: {missing_val})")
print(f"Images in training directory: {len(os.listdir(NEW_TRAIN_DIR))}")
print(f"Images in validation directory: {len(os.listdir(NEW_VAL_DIR))}")

# --- 10. CREATE ZIP ARCHIVES ---
TRAIN_PACKAGE_DIR = os.path.join(OUTPUT_BASE_DIR, 'training_dataset/')
VAL_PACKAGE_DIR = os.path.join(OUTPUT_BASE_DIR, 'validation_dataset/')

os.makedirs(TRAIN_PACKAGE_DIR, exist_ok=True)
os.makedirs(VAL_PACKAGE_DIR, exist_ok=True)

# Copy CSVs and images
shutil.copy(new_train_csv_path, os.path.join(TRAIN_PACKAGE_DIR, 'train_split.csv'))
shutil.copytree(NEW_TRAIN_DIR, os.path.join(TRAIN_PACKAGE_DIR, 'new_train_images'), dirs_exist_ok=True)

shutil.copy(new_val_csv_path, os.path.join(VAL_PACKAGE_DIR, 'validation_split.csv'))
shutil.copytree(NEW_VAL_DIR, os.path.join(VAL_PACKAGE_DIR, 'validation_images'), dirs_exist_ok=True)

# Create ZIP archives
train_zip_path = shutil.make_archive(
    os.path.join(OUTPUT_BASE_DIR, 'training_dataset'), 'zip', TRAIN_PACKAGE_DIR
)
val_zip_path = shutil.make_archive(
    os.path.join(OUTPUT_BASE_DIR, 'validation_dataset'), 'zip', VAL_PACKAGE_DIR
)

print("\n✅ Created ZIP files:")
print(f"📦 Training dataset ZIP: {train_zip_path}")
print(f"📦 Validation dataset ZIP: {val_zip_path}")

print("\n" + "="*70)
print("DATA PREPARATION COMPLETE")
print("="*70)
print("\nKey Improvements:")
print("✓ Stratified multi-label split (not last 1500 rows)")
print("✓ Label normalization (lowercase + strip)")
print("✓ Binary emotion columns for diagnostics")
print("✓ Label priors saved for training")
print("✓ Extension handling for images")


# ==================== PART 1: ENHANCED CONFIGURATION ====================
"""
Part 1: Enhanced Configuration with ASL and Class-Balanced Weights
"""

import os
import yaml
import random
import zipfile
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.cuda.amp import GradScaler, autocast

from transformers import AutoModel, AutoTokenizer, CLIPModel, CLIPProcessor
import torchvision.transforms as transforms

from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, 
    mean_squared_error, mean_absolute_error, confusion_matrix, 
    classification_report, average_precision_score, f1_score,
    precision_score, recall_score
)
from scipy.stats import spearmanr

print("✓ All imports successful")

# Load label priors
PRIORS_PATH = '/kaggle/working/label_priors.json'
with open(PRIORS_PATH, 'r') as f:
    LABEL_PRIORS = json.load(f)

print(f"✓ Loaded label priors from {PRIORS_PATH}")

# ==================== ENHANCED CONFIGURATION ====================
CONFIG = f"""
# Enhanced Multi-modal Fusion Configuration

# Model Architecture
TEXT_MODEL: "google/muril-base-cased"
IMAGE_MODEL: "openai/clip-vit-base-patch32"
TEXT_DIM: 768
IMAGE_DIM: 768
FUSION_DIM: 512
FUSION_OUT_DIM: 512

# Training Parameters
MAX_LEN: 128
IMG_SIZE: 224
BATCH_SIZE: 16
GRADIENT_ACCUMULATION_STEPS: 2
LR_HEADS: 0.001
LR_BACKBONE: 0.00001
WEIGHT_DECAY: 0.01
EPOCHS: 5
SEED: 42
DEVICE: "cuda"
CHECKPOINT_PATH: "/kaggle/working/checkpoints"

# Task Configuration
NUM_SENTIMENT_CLASSES: 5
NUM_EMOTION_CLASSES: 4  # humor, sarcasm, offensive, motivational

# Loss Weights (ENHANCED: Higher sentiment weight)
LOSS_WEIGHTS:
  sentiment: 1.5
  emotion: 1.0
  intensity: 0.5

# ASL Hyperparameters for Emotions
ASL_GAMMA_NEG: 6.0
ASL_GAMMA_POS: 0.0
ASL_CLIP: 0.05

# Emotion Priors (from training data)
EMO_PRIORS: [{LABEL_PRIORS['humor_pos_rate']}, {LABEL_PRIORS['sarcasm_pos_rate']}, {LABEL_PRIORS['offensive_pos_rate']}, {LABEL_PRIORS['motivational_pos_rate']}]

# Sentiment Class-Balanced Weights
SENTIMENT_CB_WEIGHTS:
  0: {LABEL_PRIORS.get('sentiment_cb_weights', {}).get('0', 1.0)}
  1: {LABEL_PRIORS.get('sentiment_cb_weights', {}).get('1', 1.0)}
  2: {LABEL_PRIORS.get('sentiment_cb_weights', {}).get('2', 1.0)}
  3: {LABEL_PRIORS.get('sentiment_cb_weights', {}).get('3', 1.0)}
  4: {LABEL_PRIORS.get('sentiment_cb_weights', {}).get('4', 1.0)}

# Emotion Thresholds (will be tuned during training)
EMO_THRESHOLDS: [0.5, 0.5, 0.60, 0.60]  # humor, sarcasm, offensive, motivational

# Precision Targets for Threshold Tuning
PRECISION_TARGETS:
  humor: 0.70
  sarcasm: 0.70
  offensive: 0.75
  motivational: 0.60

# Training Strategy
POOLING: "mean"
USE_AMP: true
GRADIENT_CLIP: 1.0
SCHEDULER: "cosine"
WARMUP_EPOCHS: 1
UNFREEZE_BACKBONE_EPOCH: 2  # CHANGED: Earlier unfreezing (was 3)
LABEL_SMOOTHING: 0.1

# Cross-Attention
CROSS_ATTN_HEADS: 8
CROSS_ATTN_DROPOUT: 0.1

# Oversampling for Motivational
MOTIVATIONAL_OVERSAMPLE_FACTOR: 5.0
"""

# ==================== UTILITY FUNCTIONS ====================
def set_seed(seed: int):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"✓ Random seed set to {seed}")

def load_config(config_str: str) -> dict:
    """Load configuration from YAML string"""
    cfg = yaml.safe_load(config_str)
    print("✓ Configuration loaded")
    return cfg

def save_checkpoint(state: dict, path: str):
    """Save model checkpoint"""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(state, path)
    print(f"✓ Checkpoint saved to {path}")

def load_checkpoint(model, optimizer, path: str, device: str):
    """Load model checkpoint"""
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print(f"✓ Checkpoint loaded from {path}")
    return checkpoint['epoch'], checkpoint.get('best_metric', 0)

def get_tokenizer(cfg: dict):
    """Get tokenizer for text model"""
    tokenizer = AutoTokenizer.from_pretrained(cfg['TEXT_MODEL'])
    print(f"✓ Tokenizer loaded: {cfg['TEXT_MODEL']}")
    return tokenizer

def get_image_transform(cfg: dict, is_train: bool = True):
    """Get image transforms - CLIP-specific normalization"""
    if is_train:
        transform = transforms.Compose([
            transforms.Resize((cfg['IMG_SIZE'], cfg['IMG_SIZE'])),
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711]
            )
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((cfg['IMG_SIZE'], cfg['IMG_SIZE'])),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711]
            )
        ])
    print(f"✓ Image transform created (train={is_train})")
    return transform

def explore_dataset(df, name="Dataset"):
    """Explore dataset structure and label distributions"""
    print(f"\n{'='*70}")
    print(f"{name} Exploration")
    print(f"{'='*70}")
    print(f"\nShape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    
    # Check for label columns
    label_cols = ['humour', 'humor', 'sarcastic', 'sarcasm', 'offensive', 'motivational', 'overall', 'sentiment']
    for col in label_cols:
        if col in df.columns:
            print(f"\n{col} distribution:")
            print(df[col].value_counts())
    
    print(f"\nMissing values:")
    print(df.isnull().sum())
    
    print(f"\nSample rows:")
    print(df.head(3))
    print(f"{'='*70}\n")

# Initialize configuration
cfg = load_config(CONFIG)
set_seed(cfg['SEED'])

# Set device
device = torch.device(cfg['DEVICE'] if torch.cuda.is_available() else 'cpu')
print(f"✓ Using device: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  CUDA Version: {torch.version.cuda}")

print("\n" + "="*70)
print("Part 1 Complete - Enhanced Configuration Loaded")
print("="*70)
print("\nKey Changes:")
print("✓ ASL hyperparameters configured (γ_neg=6.0, clip=0.05)")
print("✓ Emotion priors loaded from training data")
print("✓ Class-balanced weights calculated for sentiment")
print("✓ Sentiment loss weight increased to 1.5")
print("✓ Backbone unfreezing moved to epoch 2")
print("✓ Precision targets set for threshold tuning")
print("✓ Motivational oversampling factor: 5.0x")


# ==================== PART 2: DATA PREPARATION AND DATASET ====================
"""
Part 2: Data Preparation and Enhanced Dataset Class
"""

def prepare_data(cfg):
    """Extract and prepare dataset"""
    print("Preparing datasets...")
    
    train_dir = '/kaggle/working/train'
    val_dir = '/kaggle/working/val'
    
    # Extract if needed
    if not os.path.exists(train_dir):
        print("Extracting training dataset...")
        with zipfile.ZipFile('/kaggle/working/training_dataset.zip', 'r') as zip_ref:
            zip_ref.extractall('/kaggle/working/train')
    else:
        print("✓ Training dataset already extracted")
    
    if not os.path.exists(val_dir):
        print("Extracting validation dataset...")
        with zipfile.ZipFile('/kaggle/working/validation_dataset.zip', 'r') as zip_ref:
            zip_ref.extractall('/kaggle/working/val')
    else:
        print("✓ Validation dataset already extracted")
    
    # Find CSV files
    train_csv = None
    val_csv = None
    
    for root, dirs, files in os.walk(train_dir):
        for file in files:
            if file.endswith('.csv'):
                train_csv = os.path.join(root, file)
                break
        if train_csv:
            break
    
    for root, dirs, files in os.walk(val_dir):
        for file in files:
            if file.endswith('.csv'):
                val_csv = os.path.join(root, file)
                break
        if val_csv:
            break
    
    if train_csv is None or val_csv is None:
        raise FileNotFoundError("Could not find CSV files")
    
    print(f"✓ Loading training data from {train_csv}")
    train_df = pd.read_csv(train_csv)
    
    print(f"✓ Loading validation data from {val_csv}")
    val_df = pd.read_csv(val_csv)
    
    # Find image directories
    train_img_dir = None
    val_img_dir = None
    
    for root, dirs, files in os.walk(train_dir):
        img_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png'))]
        if img_files:
            train_img_dir = root
            break
        if 'images' in dirs:
            train_img_dir = os.path.join(root, 'images')
            break
    
    for root, dirs, files in os.walk(val_dir):
        img_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png'))]
        if img_files:
            val_img_dir = root
            break
        if 'images' in dirs:
            val_img_dir = os.path.join(root, 'images')
            break
    
    if train_img_dir is None or val_img_dir is None:
        raise FileNotFoundError("Could not find image directories")
    
    print(f"\n{'='*70}")
    print(f"Dataset Information:")
    print(f"  Train CSV: {train_csv}")
    print(f"  Train Images: {train_img_dir}")
    print(f"  Train Samples: {len(train_df)}")
    print(f"  Val CSV: {val_csv}")
    print(f"  Val Images: {val_img_dir}")
    print(f"  Val Samples: {len(val_df)}")
    print(f"{'='*70}\n")
    
    return train_df, val_df, train_img_dir, val_img_dir


class MemeDataset(Dataset):
    """Enhanced Dataset for meme sentiment and emotion analysis"""
    
    def __init__(self, df: pd.DataFrame, tokenizer, image_transform, 
                 image_dir: str, cfg: dict):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.image_transform = image_transform
        self.image_dir = image_dir
        self.cfg = cfg
        
        self._detect_columns()
        print(f"✓ Dataset initialized with {len(self.df)} samples")
    
    def _detect_columns(self):
        """Detect column names in the dataframe"""
        cols = self.df.columns.tolist()
        
        # Image column
        possible_image_cols = ['image_name', 'image', 'img_name', 'filename', 'image_id', 'Unnamed: 0']
        self.image_col = next((col for col in possible_image_cols if col in cols), cols[0])
        
        # Text column
        possible_text_cols = ['text', 'ocr_text', 'caption', 'extracted_text', 'OCR_extracted_text', 'ocr']
        self.text_col = next((col for col in possible_text_cols if col in cols), None)
        
        # Sentiment column
        possible_sentiment_cols = ['sentiment', 'overall_sentiment', 'sentiment_label', 'overall']
        self.sentiment_col = next((col for col in possible_sentiment_cols if col in cols), None)
        
        print(f"  Detected columns - Image: {self.image_col}, Text: {self.text_col}, Sentiment: {self.sentiment_col}")
        
        self._create_label_mappings()
    
    def _create_label_mappings(self):
        """Create mappings for string labels to numeric values"""
        # Sentiment mapping (5 classes)
        self.sentiment_map = {
            'very_positive': 0,
            'positive': 1,
            'neutral': 2,
            'negative': 3,
            'very_negative': 4
        }
        
        # Humor mapping (binary)
        self.humor_map = {
            'not_funny': 0,
            'funny': 1,
            'very_funny': 1,
            'hilarious': 1
        }
        
        # Sarcasm mapping (binary)
        self.sarcasm_map = {
            'not_sarcastic': 0,
            'general': 1,
            'twisted_meaning': 1,
            'very_twisted': 1
        }
        
        # Offensive mapping (binary)
        self.offensive_map = {
            'not_offensive': 0,
            'slight': 1,
            'very_offensive': 1,
            'hateful_offensive': 1
        }
        
        # Motivational mapping (binary)
        self.motivational_map = {
            'not_motivational': 0,
            'motivational': 1
        }
    
    def _map_label(self, value, mapping, default=0):
        """Map string label to numeric value"""
        if pd.isna(value):
            return default
        if isinstance(value, str):
            return mapping.get(value.lower().strip(), default)
        try:
            return int(value)
        except:
            return default
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int) -> Dict:
        row = self.df.iloc[idx]
        
        # Load image with extension handling
        image_name = row[self.image_col] if self.image_col else f"{idx}.jpg"
        
        if isinstance(image_name, (int, float)):
            image_name = f"{int(image_name)}.jpg"
        else:
            image_name = str(image_name)
        
        # Add extension if missing
        if not any(image_name.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
            image_name = f"{image_name}.jpg"
        
        image_path = os.path.join(self.image_dir, image_name)
        
        try:
            image = Image.open(image_path).convert('RGB')
            image = self.image_transform(image)
        except Exception as e:
            # Use blank image on error
            image = torch.zeros(3, self.cfg['IMG_SIZE'], self.cfg['IMG_SIZE'])
        
        # Tokenize text
        text = str(row.get(self.text_col, '')) if self.text_col else ''
        if text in ['nan', '', 'None'] or text is None:
            text = 'No text available'
        
        encoding = self.tokenizer(
            text,
            max_length=self.cfg['MAX_LEN'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Get sentiment label (5 classes)
        sentiment_label = 2  # default neutral
        if self.sentiment_col and self.sentiment_col in row:
            sent_val = row[self.sentiment_col]
            sentiment_label = self._map_label(sent_val, self.sentiment_map, default=2)
        sentiment_label = torch.tensor(sentiment_label, dtype=torch.long)
        
        # Get emotion labels (multi-label binary)
        humor_val = row.get('humour', row.get('humor', 0))
        sarcasm_val = row.get('sarcastic', row.get('sarcasm', 0))
        offensive_val = row.get('offensive', 0)
        motivational_val = row.get('motivational', 0)
        
        emotion_labels = torch.tensor([
            float(self._map_label(humor_val, self.humor_map, default=0)),
            float(self._map_label(sarcasm_val, self.sarcasm_map, default=0)),
            float(self._map_label(offensive_val, self.offensive_map, default=0)),
            float(self._map_label(motivational_val, self.motivational_map, default=0))
        ], dtype=torch.float)
        
        # Get intensity score (normalized 0-1)
        intensity_val = row.get('overall_sentiment', row.get('overall', 0.0))
        if isinstance(intensity_val, str):
            intensity_score_map = {
                'very_positive': 1.0,
                'positive': 0.75,
                'neutral': 0.5,
                'negative': 0.25,
                'very_negative': 0.0
            }
            intensity_val = intensity_score_map.get(intensity_val.lower().strip(), 0.5)
        
        intensity = torch.tensor(
            float(intensity_val) if not pd.isna(intensity_val) else 0.5, 
            dtype=torch.float
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'image': image,
            'sentiment_label': sentiment_label,
            'emotion_labels': emotion_labels,
            'intensity': intensity,
            'motivational_flag': emotion_labels[3]  # For weighted sampling
        }


def collate_fn(batch: List[Dict]):
    """Collate function for batching"""
    return {
        'input_ids': torch.stack([item['input_ids'] for item in batch]),
        'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
        'image': torch.stack([item['image'] for item in batch]),
        'sentiment_label': torch.stack([item['sentiment_label'] for item in batch]),
        'emotion_labels': torch.stack([item['emotion_labels'] for item in batch]),
        'intensity': torch.stack([item['intensity'] for item in batch])
    }


# ==================== Run Data Preparation ====================
try:
    train_df, val_df, train_img_dir, val_img_dir = prepare_data(cfg)
    explore_dataset(train_df, "Training Set")
    explore_dataset(val_df, "Validation Set")
    
    print("\n" + "="*70)
    print("Part 2 Complete - Data preparation successful")
    print("="*70)
except Exception as e:
    print(f"\n✗ Error in data preparation: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 3: ENHANCED MODEL WITH ASL LOSS ====================
"""
Part 3: Model Architecture with ASL Loss and Class-Balanced CE
"""

class CrossAttentionFusion(nn.Module):
    """Bidirectional cross-attention between text and image modalities"""
    
    def __init__(self, dim, num_heads=8, dropout=0.1):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        assert self.head_dim * num_heads == dim, "dim must be divisible by num_heads"
        
        # Text queries attend to Image keys/values
        self.text_to_image_attn = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        # Image queries attend to Text keys/values
        self.image_to_text_attn = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        
        # Layer norms
        self.text_norm = nn.LayerNorm(dim)
        self.image_norm = nn.LayerNorm(dim)
        
        # Feed-forward networks
        self.text_ffn = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 4, dim),
            nn.Dropout(dropout)
        )
        
        self.image_ffn = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 4, dim),
            nn.Dropout(dropout)
        )
        
        self.ffn_norm_text = nn.LayerNorm(dim)
        self.ffn_norm_image = nn.LayerNorm(dim)
    
    def forward(self, text_emb, image_emb):
        # Add sequence dimension for attention (batch, 1, dim)
        text_seq = text_emb.unsqueeze(1)
        image_seq = image_emb.unsqueeze(1)
        
        # Text attends to Image
        text_attended, _ = self.text_to_image_attn(
            query=text_seq,
            key=image_seq,
            value=image_seq
        )
        text_attended = text_attended.squeeze(1)
        text_out = self.text_norm(text_emb + text_attended)
        
        # Image attends to Text
        image_attended, _ = self.image_to_text_attn(
            query=image_seq,
            key=text_seq,
            value=text_seq
        )
        image_attended = image_attended.squeeze(1)
        image_out = self.image_norm(image_emb + image_attended)
        
        # Feed-forward with residual
        text_ffn_out = self.text_ffn(text_out)
        text_final = self.ffn_norm_text(text_out + text_ffn_out)
        
        image_ffn_out = self.image_ffn(image_out)
        image_final = self.ffn_norm_image(image_out + image_ffn_out)
        
        return text_final, image_final


class EnhancedFusionModel(nn.Module):
    """Multi-modal fusion model with Cross-Attention"""
    
    def __init__(self, cfg: dict):
        super().__init__()
        self.cfg = cfg
        
        print(f"Loading text model: {cfg['TEXT_MODEL']}")
        self.text_model = AutoModel.from_pretrained(cfg['TEXT_MODEL'])
        
        print(f"Loading image model: {cfg['IMAGE_MODEL']}")
        clip_model = CLIPModel.from_pretrained(cfg['IMAGE_MODEL'])
        self.image_model = clip_model.vision_model
        
        # Freeze encoders initially
        self._freeze_encoders()
        
        # Projection layers to common dimension
        self.text_proj = nn.Linear(cfg['TEXT_DIM'], cfg['FUSION_DIM'])
        self.image_proj = nn.Linear(cfg['IMAGE_DIM'], cfg['FUSION_DIM'])
        
        # Initialize projections
        nn.init.xavier_uniform_(self.text_proj.weight)
        nn.init.zeros_(self.text_proj.bias)
        nn.init.xavier_uniform_(self.image_proj.weight)
        nn.init.zeros_(self.image_proj.bias)
        
        # Cross-Attention Fusion
        self.cross_attention = CrossAttentionFusion(
            dim=cfg['FUSION_DIM'],
            num_heads=8,
            dropout=0.1
        )
        
        # Final fusion MLP
        fusion_input_dim = cfg['FUSION_DIM'] * 2
        self.fusion_norm = nn.LayerNorm(fusion_input_dim)
        self.fusion_mlp = nn.Sequential(
            nn.Linear(fusion_input_dim, 512),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(512, cfg['FUSION_OUT_DIM']),
            nn.LayerNorm(cfg['FUSION_OUT_DIM'])
        )
        
        # Task-specific heads
        self.sentiment_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, cfg['NUM_SENTIMENT_CLASSES'])
        )
        
        self.emotion_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, cfg['NUM_EMOTION_CLASSES'])
        )
        
        self.intensity_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )
        
        print("✓ Enhanced model with Cross-Attention initialized")
    
    def _freeze_encoders(self):
        """Freeze encoder parameters"""
        for param in self.text_model.parameters():
            param.requires_grad = False
        for param in self.image_model.parameters():
            param.requires_grad = False
        print("✓ Encoders frozen")
    
    def unfreeze_backbone(self, layers_to_unfreeze: int = 2):
        """Unfreeze top layers of encoders"""
        if hasattr(self.text_model, 'encoder') and hasattr(self.text_model.encoder, 'layer'):
            text_layers = list(self.text_model.encoder.layer[-layers_to_unfreeze:])
            for layer in text_layers:
                for param in layer.parameters():
                    param.requires_grad = True
            print(f"✓ Unfroze {layers_to_unfreeze} text encoder layers")
        
        if hasattr(self.image_model, 'encoder') and hasattr(self.image_model.encoder, 'layers'):
            image_layers = list(self.image_model.encoder.layers[-layers_to_unfreeze:])
            for layer in image_layers:
                for param in layer.parameters():
                    param.requires_grad = True
            print(f"✓ Unfroze {layers_to_unfreeze} image encoder layers")
    
    def pool_text(self, model_output, attention_mask):
        """Pool text embeddings"""
        last_hidden = model_output.last_hidden_state
        if self.cfg['POOLING'] == 'cls':
            return last_hidden[:, 0]
        elif self.cfg['POOLING'] == 'mean':
            mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
            sum_embeddings = torch.sum(last_hidden * mask_expanded, 1)
            sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
            return sum_embeddings / sum_mask
        else:
            return last_hidden[:, 0]
    
    def forward(self, input_ids, attention_mask, image):
        """Forward pass with cross-attention fusion"""
        # Text encoding
        text_output = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_emb = self.pool_text(text_output, attention_mask)
        
        # Image encoding
        image_output = self.image_model(pixel_values=image)
        image_emb = image_output.pooler_output
        
        # Project to common dimension
        text_proj = self.text_proj(text_emb)
        image_proj = self.image_proj(image_emb)
        
        # Cross-Attention Fusion
        text_cross, image_cross = self.cross_attention(text_proj, image_proj)
        
        # Concatenate and fuse
        fused = torch.cat([text_cross, image_cross], dim=1)
        fused = self.fusion_norm(fused)
        fused = self.fusion_mlp(fused)
        
        # Task predictions
        sentiment_logits = self.sentiment_head(fused)
        emotion_logits = self.emotion_head(fused)
        intensity = self.intensity_head(fused).squeeze(-1)
        
        return {
            'sentiment_logits': sentiment_logits,
            'emotion_logits': emotion_logits,
            'intensity': intensity,
            'text_features': text_cross,
            'image_features': image_cross
        }


# ==================== ENHANCED LOSS FUNCTIONS ====================

class AsymmetricLoss(nn.Module):
    """
    Asymmetric Loss (ASL) for multi-label classification
    https://arxiv.org/abs/2009.14119
    """
    
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8):
        super(AsymmetricLoss, self).__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.eps = eps
    
    def forward(self, logits, targets):
        """
        Parameters:
        -----------
        logits : Tensor
            Predicted logits (batch_size, num_classes)
        targets : Tensor
            Ground truth binary labels (batch_size, num_classes)
        """
        # Sigmoid
        xs_pos = torch.sigmoid(logits)
        xs_neg = 1 - xs_pos
        
        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)
        
        # Basic CE calculation
        los_pos = targets * torch.log(xs_pos.clamp(min=self.eps))
        los_neg = (1 - targets) * torch.log(xs_neg.clamp(min=self.eps))
        
        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            pt0 = xs_pos * targets
            pt1 = xs_neg * (1 - targets)
            pt = pt0 + pt1
            one_sided_gamma = self.gamma_pos * targets + self.gamma_neg * (1 - targets)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)
            loss = one_sided_w * (los_pos + los_neg)
        else:
            loss = los_pos + los_neg
        
        return -loss.mean()


def adjust_logits_with_priors(logits, priors, tau=1.0):
    """
    Adjust logits based on training priors
    
    Parameters:
    -----------
    logits : Tensor
        Raw logits (batch_size, num_classes)
    priors : list or Tensor
        Prior probabilities for each class
    tau : float
        Temperature for adjustment
    """
    if isinstance(priors, list):
        priors = torch.tensor(priors, device=logits.device)
    
    # Logit adjustment: logit - τ * log(prior)
    adjustment = tau * torch.log(priors.clamp(min=1e-8))
    adjusted_logits = logits - adjustment
    
    return adjusted_logits


def sentiment_loss_fn(logits, labels, cfg):
    """
    Sentiment classification loss with class-balanced weights
    """
    # Get class weights
    weights = cfg.get('SENTIMENT_CB_WEIGHTS', {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0})
    weight_tensor = torch.tensor(
        [weights[i] for i in range(cfg['NUM_SENTIMENT_CLASSES'])],
        device=logits.device,
        dtype=torch.float
    )
    
    # Class-balanced cross-entropy
    loss = F.cross_entropy(
        logits, 
        labels, 
        weight=weight_tensor,
        label_smoothing=cfg.get('LABEL_SMOOTHING', 0.0)
    )
    
    return loss


def emotion_loss_fn(logits, labels, cfg):
    """
    Emotion classification loss using ASL with prior adjustment
    """
    # Prior adjustment (optional but recommended)
    priors = cfg.get('EMO_PRIORS', [0.5, 0.5, 0.5, 0.5])
    adjusted_logits = adjust_logits_with_priors(logits, priors, tau=1.0)
    
    # ASL loss
    asl = AsymmetricLoss(
        gamma_neg=cfg.get('ASL_GAMMA_NEG', 4.0),
        gamma_pos=cfg.get('ASL_GAMMA_POS', 0.0),
        clip=cfg.get('ASL_CLIP', 0.05)
    )
    
    loss = asl(adjusted_logits, labels)
    
    return loss


def intensity_loss_fn(preds, targets, cfg):
    """Intensity regression loss"""
    return F.smooth_l1_loss(preds, targets)


def combined_loss(outputs, batch, cfg):
    """Combined multi-task loss"""
    loss_sent = sentiment_loss_fn(outputs['sentiment_logits'], batch['sentiment_label'], cfg)
    loss_emotion = emotion_loss_fn(outputs['emotion_logits'], batch['emotion_labels'], cfg)
    loss_intensity = intensity_loss_fn(outputs['intensity'], batch['intensity'], cfg)
    
    total_loss = (
        cfg['LOSS_WEIGHTS']['sentiment'] * loss_sent +
        cfg['LOSS_WEIGHTS']['emotion'] * loss_emotion +
        cfg['LOSS_WEIGHTS']['intensity'] * loss_intensity
    )
    
    return total_loss, loss_sent, loss_emotion, loss_intensity


# ==================== Initialize and Test Model ====================
try:
    print("\nInitializing Enhanced Model with Cross-Attention and ASL...")
    model = EnhancedFusionModel(cfg).to(device)
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\n{'='*70}")
    print("Model Summary:")
    print(f"  Total parameters: {total_params:,}")
    print(f"  Trainable parameters: {trainable_params:,}")
    print(f"  Frozen parameters: {total_params - trainable_params:,}")
    print(f"{'='*70}")
    
    print("\nTesting forward pass with dummy data...")
    dummy_input_ids = torch.randint(0, 1000, (2, cfg['MAX_LEN'])).to(device)
    dummy_attention_mask = torch.ones(2, cfg['MAX_LEN']).to(device)
    dummy_image = torch.randn(2, 3, cfg['IMG_SIZE'], cfg['IMG_SIZE']).to(device)
    
    with torch.no_grad():
        test_output = model(dummy_input_ids, dummy_attention_mask, dummy_image)
    
    print(f"✓ Forward pass successful!")
    print(f"  Sentiment logits shape: {test_output['sentiment_logits'].shape}")
    print(f"  Emotion logits shape: {test_output['emotion_logits'].shape}")
    print(f"  Intensity shape: {test_output['intensity'].shape}")
    
    print("\n" + "="*70)
    print("Part 3 Complete - Enhanced Model with ASL Loss initialized")
    print("="*70)
    print("\nKey Changes:")
    print("✓ ASL loss for emotions (γ_neg=6.0, γ_pos=0.0, clip=0.05)")
    print("✓ Prior logit adjustment for emotions")
    print("✓ Class-balanced CE for sentiment")
    print("✓ Label smoothing as fallback")

except Exception as e:
    print(f"\n✗ Error in model initialization: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 4: DATALOADERS WITH OVERSAMPLING ====================
"""
Part 4: DataLoader Creation with Weighted Sampling for Motivational
"""

LABELS = ["humor", "sarcasm", "offensive", "motivational"]

try:
    print("Creating tokenizer and transforms...")
    tokenizer = get_tokenizer(cfg)
    train_transform = get_image_transform(cfg, is_train=True)
    val_transform = get_image_transform(cfg, is_train=False)
    
    print("\nCreating datasets...")
    train_dataset = MemeDataset(train_df, tokenizer, train_transform, train_img_dir, cfg)
    val_dataset = MemeDataset(val_df, tokenizer, val_transform, val_img_dir, cfg)
    
    # ==================== WEIGHTED SAMPLING FOR MOTIVATIONAL ====================
    print("\n" + "="*70)
    print("Creating Weighted Sampler for Motivational Oversampling")
    print("="*70)
    
    # Calculate sample weights
    sample_weights = []
    motivational_count = 0
    non_motivational_count = 0
    
    for idx in range(len(train_dataset)):
        item = train_dataset[idx]
        is_motivational = item['motivational_flag'].item()
        
        if is_motivational:
            # Oversample motivational samples
            weight = cfg.get('MOTIVATIONAL_OVERSAMPLE_FACTOR', 5.0)
            motivational_count += 1
        else:
            weight = 1.0
            non_motivational_count += 1
        
        sample_weights.append(weight)
    
    print(f"\nMotivational samples: {motivational_count} ({motivational_count/len(train_dataset):.2%})")
    print(f"Non-motivational samples: {non_motivational_count}")
    print(f"Oversampling factor: {cfg.get('MOTIVATIONAL_OVERSAMPLE_FACTOR', 5.0)}x")
    
    # Create weighted sampler
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    print("✓ Weighted sampler created")
    
    # ==================== CREATE DATALOADERS ====================
    print("\nCreating dataloaders...")
    num_workers = 0 if device.type == 'cpu' else 2
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg['BATCH_SIZE'],
        sampler=sampler,  # Use weighted sampler instead of shuffle
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=collate_fn,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=cfg['BATCH_SIZE'],
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        collate_fn=collate_fn
    )
    
    print(f"✓ Train batches: {len(train_loader)}")
    print(f"✓ Val batches: {len(val_loader)}")
    
    # ==================== Test DataLoader ====================
    print("\nTesting dataloader with real batch...")
    test_batch = next(iter(train_loader))
    print(f"  Input IDs shape: {test_batch['input_ids'].shape}")
    print(f"  Attention mask shape: {test_batch['attention_mask'].shape}")
    print(f"  Image shape: {test_batch['image'].shape}")
    print(f"  Sentiment labels shape: {test_batch['sentiment_label'].shape}")
    print(f"  Emotion labels shape: {test_batch['emotion_labels'].shape}")
    print(f"  Intensity shape: {test_batch['intensity'].shape}")
    
    print("\nSample emotion labels (motivational column):")
    print(test_batch['emotion_labels'][:, 3])  # Motivational is index 3
    
    # ==================== Test Model with Real Batch ====================
    print("\nTesting model forward pass with real batch...")
    model.eval()
    with torch.no_grad():
        batch_input_ids = test_batch['input_ids'][:2].to(device)
        batch_attention_mask = test_batch['attention_mask'][:2].to(device)
        batch_image = test_batch['image'][:2].to(device)
        
        real_output = model(batch_input_ids, batch_attention_mask, batch_image)
    
    print(f"✓ Real batch forward pass successful!")
    
    # Test loss computation
    print("\nTesting loss computation with ASL...")
    batch_device = {
        'sentiment_label': test_batch['sentiment_label'][:2].to(device),
        'emotion_labels': test_batch['emotion_labels'][:2].to(device),
        'intensity': test_batch['intensity'][:2].to(device)
    }
    
    total_loss, loss_sent, loss_emotion, loss_int = combined_loss(real_output, batch_device, cfg)
    
    print(f"✓ Loss computation successful!")
    print(f"  Total loss: {total_loss.item():.4f}")
    print(f"  Sentiment loss (CB-CE): {loss_sent.item():.4f}")
    print(f"  Emotion loss (ASL): {loss_emotion.item():.4f}")
    print(f"  Intensity loss: {loss_int.item():.4f}")
    
    # ==================== Dataset Imbalance Stats ====================
    print("\n" + "="*70)
    print("Dataset Imbalance Statistics")
    print("="*70)
    
    def label_stats(loader):
        cnt = np.zeros(len(LABELS), dtype=np.int64)
        total = 0
        for batch in loader:
            y = batch['emotion_labels'].numpy()
            cnt += y.sum(axis=0).astype(int)
            total += y.shape[0]
        
        print("Total samples:", total)
        for i, name in enumerate(LABELS):
            pos = cnt[i]
            print(f"{name}: pos={pos}, pos_rate={pos/total:.4f}, neg={total-pos}")
        
        return cnt, total
    
    pos_counts, N = label_stats(train_loader)
    neg_counts = N - pos_counts
    
    print("\n" + "="*70)
    print("Part 4 Complete - DataLoaders with Oversampling created")
    print("="*70)
    print("\nKey Changes:")
    print("✓ WeightedRandomSampler for motivational oversampling (5x)")
    print("✓ Proper extension handling in dataset")
    print("✓ Label normalization (lowercase + strip)")
    print("✓ ASL loss tested successfully")

except Exception as e:
    print(f"\n✗ Error in dataloader creation: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 5: METRICS WITH PRECISION-TARGETED THRESHOLDS ====================
"""
Part 5: Enhanced Metrics and Precision-Targeted Threshold Tuning
"""

def compute_sentiment_metrics(y_true, y_pred_logits):
    """Compute sentiment classification metrics"""
    y_pred = torch.argmax(y_pred_logits, dim=1).cpu().numpy()
    y_true = y_true.cpu().numpy()
    
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    
    return {
        'sentiment_accuracy': accuracy,
        'sentiment_precision': precision,
        'sentiment_recall': recall,
        'sentiment_f1': f1
    }


def compute_emotion_metrics(y_true, y_pred_logits, thresholds=None):
    """Compute emotion classification metrics (multi-label)"""
    probs = torch.sigmoid(y_pred_logits).cpu().numpy()
    y_true_np = y_true.cpu().numpy()
    
    if thresholds is None:
        thresholds = np.array([0.5] * y_true_np.shape[1])
    else:
        thresholds = np.array(thresholds)
    
    y_pred = (probs >= thresholds).astype(float)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true_np, y_pred, average='samples', zero_division=0
    )
    
    return {
        'emotion_precision': precision,
        'emotion_recall': recall,
        'emotion_f1': f1
    }


def compute_intensity_metrics(y_true, y_pred):
    """Compute intensity regression metrics"""
    y_true = y_true.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    try:
        spearman, _ = spearmanr(y_true, y_pred)
        if np.isnan(spearman):
            spearman = 0.0
    except:
        spearman = 0.0
    
    return {
        'intensity_rmse': rmse,
        'intensity_mae': mae,
        'intensity_spearman': spearman
    }


# ==================== PRECISION-TARGETED THRESHOLD TUNING ====================

def tune_thresholds_for_precision(y_true_tensor, logits_tensor, 
                                  precision_targets=None, grid=None):
    """
    Tune thresholds for each emotion label to achieve target precision
    while maximizing F1 score
    
    Parameters:
    -----------
    y_true_tensor : Tensor
        True labels (batch_size, num_classes)
    logits_tensor : Tensor
        Predicted logits (batch_size, num_classes)
    precision_targets : dict
        Target precision for each label
    grid : array-like
        Threshold values to try
    """
    if grid is None:
        grid = np.linspace(0.05, 0.95, 50)
    
    if precision_targets is None:
        precision_targets = {
            'humor': 0.70,
            'sarcasm': 0.70,
            'offensive': 0.75,
            'motivational': 0.60
        }
    
    probs = torch.sigmoid(logits_tensor).detach().cpu().numpy()
    y_true = y_true_tensor.detach().cpu().numpy().astype(int)
    C = probs.shape[1]
    
    best_thresholds = []
    
    print(f"\n{'='*70}")
    print("Precision-Targeted Threshold Tuning")
    print(f"{'='*70}")
    
    for c, name in enumerate(LABELS):
        target_precision = precision_targets.get(name, 0.70)
        
        best_thr = 0.5
        best_f1 = -1
        best_prec = 0
        best_rec = 0
        
        for t in grid:
            pred = (probs[:, c] >= t).astype(int)
            
            # Calculate metrics
            prec = precision_score(y_true[:, c], pred, zero_division=0)
            rec = recall_score(y_true[:, c], pred, zero_division=0)
            f1 = f1_score(y_true[:, c], pred, zero_division=0)
            
            # Check if precision meets target
            if prec >= target_precision:
                if f1 > best_f1:
                    best_f1 = f1
                    best_thr = t
                    best_prec = prec
                    best_rec = rec
        
        # If no threshold meets target, use the one with highest F1
        if best_f1 == -1:
            for t in grid:
                pred = (probs[:, c] >= t).astype(int)
                f1 = f1_score(y_true[:, c], pred, zero_division=0)
                prec = precision_score(y_true[:, c], pred, zero_division=0)
                rec = recall_score(y_true[:, c], pred, zero_division=0)
                
                if f1 > best_f1:
                    best_f1 = f1
                    best_thr = t
                    best_prec = prec
                    best_rec = rec
        
        best_thresholds.append(best_thr)
        
        # Calculate AUPRC
        ap = average_precision_score(y_true[:, c], probs[:, c])
        
        status = "✓" if best_prec >= target_precision else "⚠"
        print(f"\n{status} {name.upper()}:")
        print(f"  Target Precision: {target_precision:.2f}")
        print(f"  Best Threshold: {best_thr:.3f}")
        print(f"  Precision: {best_prec:.4f}")
        print(f"  Recall: {best_rec:.4f}")
        print(f"  F1: {best_f1:.4f}")
        print(f"  AUPRC: {ap:.4f}")
    
    return np.array(best_thresholds)


# ==================== Per-Label Confusion Matrix ====================

def eval_multilabel(y_true_tensor, logits_tensor, thresholds=None):
    """Evaluate multi-label classification with per-label confusion matrix"""
    probs = torch.sigmoid(logits_tensor).detach().cpu().numpy()
    y_true = y_true_tensor.detach().cpu().numpy().astype(int)
    C = probs.shape[1]
    
    if thresholds is None:
        thresholds = np.array([0.5] * C)
    
    y_pred = (probs >= thresholds).astype(int)
    
    print(f"\n{'='*70}")
    print("Per-Label Evaluation")
    print(f"{'='*70}")
    
    for i, name in enumerate(LABELS):
        tn, fp, fn, tp = confusion_matrix(
            y_true[:, i], y_pred[:, i], labels=[0, 1]
        ).ravel()
        ap = average_precision_score(y_true[:, i], probs[:, i])
        
        print(f"\n=== {name.upper()} ===")
        print(f"TP={tp} FP={fp} FN={fn} TN={tn} | AUPRC={ap:.4f}")
        print(classification_report(y_true[:, i], y_pred[:, i], digits=4))
    
    return y_pred, probs


# ==================== Test Metrics ====================
try:
    print("\n" + "="*70)
    print("Testing enhanced metrics computation...")
    print("="*70)
    
    # Create dummy predictions
    dummy_sentiment_logits = torch.randn(10, cfg['NUM_SENTIMENT_CLASSES'])
    dummy_sentiment_labels = torch.randint(0, cfg['NUM_SENTIMENT_CLASSES'], (10,))
    
    dummy_emotion_logits = torch.randn(10, cfg['NUM_EMOTION_CLASSES'])
    dummy_emotion_labels = torch.randint(0, 2, (10, cfg['NUM_EMOTION_CLASSES'])).float()
    
    dummy_intensity_pred = torch.rand(10)
    dummy_intensity_true = torch.rand(10)
    
    # Compute metrics
    sentiment_metrics = compute_sentiment_metrics(dummy_sentiment_labels, dummy_sentiment_logits)
    emotion_metrics = compute_emotion_metrics(dummy_emotion_labels, dummy_emotion_logits)
    intensity_metrics = compute_intensity_metrics(dummy_intensity_true, dummy_intensity_pred)
    
    print("\n✓ Sentiment Metrics:")
    for key, value in sentiment_metrics.items():
        print(f"  {key}: {value:.4f}")
    
    print("\n✓ Emotion Metrics:")
    for key, value in emotion_metrics.items():
        print(f"  {key}: {value:.4f}")
    
    print("\n✓ Intensity Metrics:")
    for key, value in intensity_metrics.items():
        print(f"  {key}: {value:.4f}")
    
    print("\n" + "="*70)
    print("Part 5 Complete - Enhanced metrics tested successfully")
    print("="*70)
    print("\nKey Changes:")
    print("✓ Precision-targeted threshold tuning")
    print("✓ Per-label confusion matrix evaluation")
    print("✓ AUPRC tracking for each emotion")
    print("✓ Customizable precision targets")

except Exception as e:
    print(f"\n✗ Error in metrics computation: {e}")
    import traceback
    traceback.print_exc()




Downloading...
From (original): https://drive.google.com/uc?id=1jEJ2nf5CDJknq80ogzU-Uyz7jbBi-1LZ
From (redirected): https://drive.google.com/uc?id=1jEJ2nf5CDJknq80ogzU-Uyz7jbBi-1LZ&confirm=t&uuid=8f99e725-a826-423f-a0b6-88ab0567207a
To: /kaggle/working/trainImages.zip
100%|█████████████████████████████████████████| 187M/187M [00:01<00:00, 149MB/s]
Archive:  trainImages.zip
replace __MACOSX/._trainImages? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Retrieving folder contents
Processing file 12r4t1_kGLPDFilQjq9z-aajTkgfxmKbW format.py
Processing file 1sFYqIKpiUhsnuFDN0rY4wAGKkjiFI2JR memotion3-test.zip
Processing file 1eKS4bGya5SQXP0E9S9iHf0pUUbR0RJJb memotion3-val.zip
Processing file 11TRDmQhPXcwXK3DJIPkXyYBIaL8ard1v memotion3.zip
Processing file 1X_K5hJmYOFO3BlkHJn50UypX0RJ_SYU_ README.txt
Processing file 1Q1I3E00H2L6V2fdV-y5796HKbzas2vxF testImages.zip
Processing file 1jEJ2nf5CDJknq80ogzU-Uyz7jbBi-1LZ trainImages.zip
Processing file 189yDwbXt0fdhz-Zm4bKSlL3-E6M74kKl valImages.zip
Retriev

100%|██████████| 1050/1050 [00:00<00:00, 6230.77it/s]



📁 Copying training images...


100%|██████████| 5950/5950 [00:00<00:00, 6016.46it/s]



✅ Successfully created stratified split datasets!
Training CSV saved to: /kaggle/working/train_split.csv
Validation CSV saved to: /kaggle/working/validation_split.csv

📊 Final Verification:
Training CSV rows: 5950
Validation CSV rows: 1050
Images copied to training directory: 5950 (missing: 0)
Images copied to validation directory: 1050 (missing: 0)
Images in training directory: 5950
Images in validation directory: 1050

✅ Created ZIP files:
📦 Training dataset ZIP: /kaggle/working/training_dataset.zip
📦 Validation dataset ZIP: /kaggle/working/validation_dataset.zip

DATA PREPARATION COMPLETE

Key Improvements:
✓ Stratified multi-label split (not last 1500 rows)
✓ Label normalization (lowercase + strip)
✓ Binary emotion columns for diagnostics
✓ Label priors saved for training
✓ Extension handling for images


...

✓ Encoders frozen
✓ Enhanced model with Cross-Attention initialized

Model Summary:
  Total parameters: 333,225,738
  Trainable parameters: 8,213,514
  Frozen parameters: 325,012,224

Testing forward pass with dummy data...
✓ Forward pass successful!
  Sentiment logits shape: torch.Size([2, 5])
  Emotion logits shape: torch.Size([2, 4])
  Intensity shape: torch.Size([2])

Part 3 Complete - Enhanced Model with ASL Loss initialized

Key Changes:
✓ ASL loss for emotions (γ_neg=6.0, γ_pos=0.0, clip=0.05)
✓ Prior logit adjustment for emotions
✓ Class-balanced CE for sentiment
✓ Label smoothing as fallback
Creating tokenizer and transforms...


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

✓ Tokenizer loaded: google/muril-base-cased
✓ Image transform created (train=True)
✓ Image transform created (train=False)

Creating datasets...
  Detected columns - Image: Unnamed: 0, Text: ocr, Sentiment: overall
✓ Dataset initialized with 5950 samples
  Detected columns - Image: Unnamed: 0, Text: ocr, Sentiment: overall
✓ Dataset initialized with 1050 samples

Creating Weighted Sampler for Motivational Oversampling

Motivational samples: 706 (11.87%)
Non-motivational samples: 5244
Oversampling factor: 5.0x
✓ Weighted sampler created

Creating dataloaders...
✓ Train batches: 371
✓ Val batches: 66

Testing dataloader with real batch...
  Input IDs shape: torch.Size([16, 128])
  Attention mask shape: torch.Size([16, 128])
  Image shape: torch.Size([16, 3, 224, 224])
  Sentiment labels shape: torch.Size([16])
  Emotion labels shape: torch.Size([16, 4])
  Intensity shape: torch.Size([16])

Sample emotion labels (motivational column):
tensor([0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.

In [4]:
# ==================== PART 6: ENHANCED TRAINER WITH EARLIER UNFREEZING ====================
"""
Part 6: Enhanced Trainer with Earlier Backbone Unfreezing
"""

class Trainer:
    """Enhanced Trainer for multi-task meme analysis"""
    
    def __init__(self, model, cfg, train_loader, val_loader, device):
        self.model = model
        self.cfg = cfg
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        
        self.optimizer = self.make_optimizer()
        self.scheduler = self.make_scheduler()
        self.scaler = GradScaler() if cfg['USE_AMP'] else None
        
        self.current_epoch = 0
        self.best_metric = -float('inf')
        self.gradient_accumulation_steps = cfg.get('GRADIENT_ACCUMULATION_STEPS', 1)
        
        # Store validation outputs for diagnostics
        self.val_emotion_logits = None
        self.val_emotion_labels = None
        
        print("✓ Trainer initialized")
    
    def make_optimizer(self):
        """Create optimizer with parameter groups"""
        head_params = []
        backbone_params = []
        
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                if 'text_model' in name or 'image_model' in name:
                    backbone_params.append(param)
                else:
                    head_params.append(param)
        
        param_groups = [
            {'params': head_params, 'lr': self.cfg['LR_HEADS']},
        ]
        
        if backbone_params:
            param_groups.append({'params': backbone_params, 'lr': self.cfg['LR_BACKBONE']})
        
        print(f"  Optimizer: {len(head_params)} head params, {len(backbone_params)} backbone params")
        
        return torch.optim.AdamW(param_groups, weight_decay=self.cfg['WEIGHT_DECAY'])
    
    def make_scheduler(self):
        """Create learning rate scheduler"""
        if self.cfg['SCHEDULER'] == 'cosine':
            return torch.optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer, T_max=self.cfg['EPOCHS']
            )
        return None
    
    def train_epoch(self, epoch):
        """Train for one epoch"""
        self.model.train()
        total_loss = 0
        total_sent_loss = 0
        total_emotion_loss = 0
        total_intensity_loss = 0
        
        pbar = tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.cfg['EPOCHS']} [Train]")
        self.optimizer.zero_grad()
        
        for batch_idx, batch in enumerate(pbar):
            # Move to device
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            image = batch['image'].to(self.device)
            sentiment_label = batch['sentiment_label'].to(self.device)
            emotion_labels = batch['emotion_labels'].to(self.device)
            intensity = batch['intensity'].to(self.device)
            
            batch_device = {
                'sentiment_label': sentiment_label,
                'emotion_labels': emotion_labels,
                'intensity': intensity
            }
            
            # Forward pass with AMP
            if self.cfg['USE_AMP']:
                with autocast():
                    outputs = self.model(input_ids, attention_mask, image)
                    loss, l_sent, l_emotion, l_int = combined_loss(outputs, batch_device, self.cfg)
                    loss = loss / self.gradient_accumulation_steps
                
                self.scaler.scale(loss).backward()
                
                if (batch_idx + 1) % self.gradient_accumulation_steps == 0:
                    if self.cfg['GRADIENT_CLIP'] > 0:
                        self.scaler.unscale_(self.optimizer)
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.cfg['GRADIENT_CLIP'])
                    
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                    self.optimizer.zero_grad()
            else:
                outputs = self.model(input_ids, attention_mask, image)
                loss, l_sent, l_emotion, l_int = combined_loss(outputs, batch_device, self.cfg)
                loss = loss / self.gradient_accumulation_steps
                loss.backward()
                
                if (batch_idx + 1) % self.gradient_accumulation_steps == 0:
                    if self.cfg['GRADIENT_CLIP'] > 0:
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.cfg['GRADIENT_CLIP'])
                    
                    self.optimizer.step()
                    self.optimizer.zero_grad()
            
            total_loss += loss.item() * self.gradient_accumulation_steps
            total_sent_loss += l_sent.item()
            total_emotion_loss += l_emotion.item()
            total_intensity_loss += l_int.item()
            
            pbar.set_postfix({
                'loss': f"{loss.item() * self.gradient_accumulation_steps:.4f}",
                'sent': f"{l_sent.item():.4f}",
                'emo': f"{l_emotion.item():.4f}",
                'int': f"{l_int.item():.4f}"
            })
        
        avg_loss = total_loss / len(self.train_loader)
        print(f"\nTrain Loss: {avg_loss:.4f}")
        print(f"  Sentiment: {total_sent_loss/len(self.train_loader):.4f}")
        print(f"  Emotion (ASL): {total_emotion_loss/len(self.train_loader):.4f}")
        print(f"  Intensity: {total_intensity_loss/len(self.train_loader):.4f}")
        
        return avg_loss
    
    def validate(self, epoch):
        """Validate the model"""
        self.model.eval()
        total_loss = 0
        
        all_sentiment_labels = []
        all_sentiment_logits = []
        all_emotion_labels = []
        all_emotion_logits = []
        all_intensity_true = []
        all_intensity_pred = []
        
        pbar = tqdm(self.val_loader, desc=f"Epoch {epoch+1}/{self.cfg['EPOCHS']} [Val]")
        
        with torch.no_grad():
            for batch in pbar:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                image = batch['image'].to(self.device)
                sentiment_label = batch['sentiment_label'].to(self.device)
                emotion_labels = batch['emotion_labels'].to(self.device)
                intensity = batch['intensity'].to(self.device)
                
                batch_device = {
                    'sentiment_label': sentiment_label,
                    'emotion_labels': emotion_labels,
                    'intensity': intensity
                }
                
                outputs = self.model(input_ids, attention_mask, image)
                loss, _, _, _ = combined_loss(outputs, batch_device, self.cfg)
                
                total_loss += loss.item()
                
                all_sentiment_labels.append(sentiment_label)
                all_sentiment_logits.append(outputs['sentiment_logits'])
                all_emotion_labels.append(emotion_labels)
                all_emotion_logits.append(outputs['emotion_logits'])
                all_intensity_true.append(intensity)
                all_intensity_pred.append(outputs['intensity'])
        
        # Aggregate metrics
        all_sentiment_labels = torch.cat(all_sentiment_labels)
        all_sentiment_logits = torch.cat(all_sentiment_logits)
        all_emotion_labels = torch.cat(all_emotion_labels)
        all_emotion_logits = torch.cat(all_emotion_logits)
        all_intensity_true = torch.cat(all_intensity_true)
        all_intensity_pred = torch.cat(all_intensity_pred)
        
        # Store for diagnostics
        self.val_emotion_logits = all_emotion_logits
        self.val_emotion_labels = all_emotion_labels
        
        sentiment_metrics = compute_sentiment_metrics(all_sentiment_labels, all_sentiment_logits)
        emotion_metrics = compute_emotion_metrics(
            all_emotion_labels, all_emotion_logits, 
            thresholds=self.cfg.get('EMO_THRESHOLDS', [0.5, 0.5, 0.6, 0.6])
        )
        intensity_metrics = compute_intensity_metrics(all_intensity_true, all_intensity_pred)
        
        avg_loss = total_loss / len(self.val_loader)
        
        metrics = {
            'val_loss': avg_loss,
            **sentiment_metrics,
            **emotion_metrics,
            **intensity_metrics
        }
        
        print(f"\nValidation Metrics:")
        print(f"  Loss: {avg_loss:.4f}")
        print(f"  Sentiment F1 (macro): {sentiment_metrics['sentiment_f1']:.4f}")
        print(f"  Emotion F1 (samples): {emotion_metrics['emotion_f1']:.4f}")
        print(f"  Intensity RMSE: {intensity_metrics['intensity_rmse']:.4f}")
        
        return metrics
    
    def run_diagnostics(self, epoch):
        """Run diagnostic checks after validation"""
        if self.val_emotion_logits is None or self.val_emotion_labels is None:
            return
        
        print(f"\n{'='*70}")
        print(f"DIAGNOSTIC: Per-Label Analysis (Epoch {epoch+1})")
        print(f"{'='*70}")
        
        # Per-label confusion matrix with current thresholds
        current_thresholds = np.array(self.cfg.get('EMO_THRESHOLDS', [0.5, 0.5, 0.6, 0.6]))
        _ = eval_multilabel(self.val_emotion_labels, self.val_emotion_logits, 
                           thresholds=current_thresholds)
        
        # Precision-targeted threshold tuning
        print(f"\n{'='*70}")
        print("DIAGNOSTIC: Precision-Targeted Threshold Tuning")
        print(f"{'='*70}")
        
        precision_targets = self.cfg.get('PRECISION_TARGETS', {
            'humor': 0.70,
            'sarcasm': 0.70,
            'offensive': 0.75,
            'motivational': 0.60
        })
        
        best_thr = tune_thresholds_for_precision(
            self.val_emotion_labels, 
            self.val_emotion_logits,
            precision_targets=precision_targets
        )
        
        self.cfg['EMO_THRESHOLDS'] = best_thr.tolist()
        print(f"\n✓ Updated thresholds: {self.cfg['EMO_THRESHOLDS']}")
        
        # Re-run evaluation with tuned thresholds
        print(f"\n{'='*70}")
        print("DIAGNOSTIC: Results with Tuned Thresholds")
        print(f"{'='*70}")
        _ = eval_multilabel(self.val_emotion_labels, self.val_emotion_logits, 
                           thresholds=best_thr)
    
    def fit(self):
        """Train the model"""
        print(f"\n{'='*70}")
        print(f"Starting Training for {self.cfg['EPOCHS']} epochs")
        print(f"{'='*70}\n")
        
        for epoch in range(self.cfg['EPOCHS']):
            self.current_epoch = epoch
            
            # CHANGED: Earlier backbone unfreezing at epoch 2 (was 3)
            if epoch == self.cfg.get('UNFREEZE_BACKBONE_EPOCH', 2):
                print(f"\n{'='*50}")
                print(f"⚡ Unfreezing backbone at epoch {epoch+1} (EARLIER!)")
                print(f"{'='*50}\n")
                self.model.unfreeze_backbone(layers_to_unfreeze=2)
                self.optimizer = self.make_optimizer()
                self.scheduler = self.make_scheduler()
            
            # Train
            train_loss = self.train_epoch(epoch)
            
            # Validate
            val_metrics = self.validate(epoch)
            
            # Run diagnostics (every 2 epochs or last epoch)
            if epoch % 2 == 0 or epoch == self.cfg['EPOCHS'] - 1:
                self.run_diagnostics(epoch)
            
            # Scheduler step
            if self.scheduler:
                self.scheduler.step()
            
            # Composite metric for checkpoint (macro F1 preferred)
            composite_metric = (
                val_metrics['sentiment_f1'] + 
                val_metrics['emotion_f1'] - 
                val_metrics['intensity_rmse']
            )
            
            # Save best checkpoint
            if composite_metric > self.best_metric:
                self.best_metric = composite_metric
                checkpoint_path = os.path.join(self.cfg['CHECKPOINT_PATH'], 'best_model.pt')
                save_checkpoint({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'best_metric': self.best_metric,
                    'metrics': val_metrics,
                    'config': self.cfg
                }, checkpoint_path)
                print(f"✓ Saved best model (composite: {composite_metric:.4f})")
            
            print(f"\n{'='*70}\n")
        
        print("Training completed!")
        return self.best_metric


# ==================== Initialize Trainer ====================
try:
    print("\nInitializing enhanced trainer...")
    trainer = Trainer(model, cfg, train_loader, val_loader, device)
    
    print("\n" + "="*70)
    print("Part 6 Complete - Enhanced Trainer ready")
    print("="*70)
    print("\nKey Changes:")
    print("✓ Backbone unfreezing moved to epoch 2 (was 3)")
    print("✓ Precision-targeted threshold tuning integrated")
    print("✓ ASL loss tracked separately in training")
    print("✓ Macro F1 used for early stopping")
    print("\nRun Part 7 to start training")

except Exception as e:
    print(f"\n✗ Error initializing trainer: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 7: TRAINING WITH DIAGNOSTICS ====================
"""
Part 7: Start Training with Comprehensive Diagnostics
"""

print("\n" + "="*70)
print("Part 7: Training with Enhanced Diagnostics")
print("="*70)

# ==================== DIAGNOSTIC 1: Tiny-batch Overfit & Gradient Sanity ====================
print("\n" + "="*70)
print("DIAGNOSTIC 1: Tiny-batch Overfit & Gradient Sanity Check")
print("="*70)

tiny_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
model.train()
opt = torch.optim.AdamW(model.parameters(), lr=1e-4)

print("\nRunning 200 steps on tiny batch to verify model can overfit...")
for step, batch in enumerate(tiny_loader):
    if step > 200:
        break
    
    # Move batch to device
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    image = batch['image'].to(device)
    
    batch_device = {
        'sentiment_label': batch['sentiment_label'].to(device),
        'emotion_labels': batch['emotion_labels'].to(device),
        'intensity': batch['intensity'].to(device)
    }
    
    outputs = model(input_ids, attention_mask, image)
    total, l_sent, l_emo, l_int = combined_loss(outputs, batch_device, cfg)
    
    opt.zero_grad(set_to_none=True)
    total.backward()
    
    # Gradient NaN/inf check
    bad_grad = False
    for p in model.parameters():
        if p.grad is not None:
            if torch.isnan(p.grad).any() or torch.isinf(p.grad).any():
                bad_grad = True
                break
    
    if bad_grad:
        print("✗ Bad gradient detected (NaN/Inf). Stop & inspect loss/labels.")
        break
    
    opt.step()
    
    if step % 20 == 0:
        print(f"{step:03d}: total={total.item():.4f} | sent={l_sent.item():.4f} | emo(ASL)={l_emo.item():.4f} | int={l_int.item():.4f}")

print("✓ Tiny-batch sanity done. Model can learn from data.")
print("Emotion loss (ASL) should have decreased. If not, check labels/loss function.")

# Reinitialize model and trainer for actual training
print("\nReinitializing model for full training...")
model = EnhancedFusionModel(cfg).to(device)
trainer = Trainer(model, cfg, train_loader, val_loader, device)

# ==================== Start Full Training ====================
try:
    print("\n" + "="*70)
    print("STARTING FULL TRAINING")
    print("="*70)
    
    best_metric = trainer.fit()
    
    print(f"\n{'='*70}")
    print(f"Training Completed!")
    print(f"Best composite metric: {best_metric:.4f}")
    print(f"Best model saved to: {cfg['CHECKPOINT_PATH']}/best_model.pt")
    print(f"{'='*70}")
    
    # Save model card
    model_card = f"""# Enhanced Meme Sentiment and Emotion Analysis Model

## Model Details
- **Text Encoder**: {cfg['TEXT_MODEL']}
- **Image Encoder**: {cfg['IMAGE_MODEL']}
- **Architecture**: Multi-modal fusion with cross-attention
- **Tasks**:
  - Sentiment Classification (5 classes)
  - Emotion Classification (4 binary classes: humor, sarcasm, offensive, motivational)
  - Intensity Regression (continuous 0-1)

## Training Configuration
- Epochs: {cfg['EPOCHS']}
- Batch Size: {cfg['BATCH_SIZE']}
- Gradient Accumulation: {cfg.get('GRADIENT_ACCUMULATION_STEPS', 1)}
- Learning Rate (Heads): {cfg['LR_HEADS']}
- Learning Rate (Backbone): {cfg['LR_BACKBONE']}
- Mixed Precision: {cfg['USE_AMP']}

## Enhanced Imbalance Handling
- **Stratified Split**: Multi-label stratification on offensive, motivational, and sentiment
- **Loss Functions**:
  - Sentiment: Class-Balanced Cross-Entropy (weight: {cfg['LOSS_WEIGHTS']['sentiment']})
  - Emotions: Asymmetric Loss (ASL) with γ_neg={cfg['ASL_GAMMA_NEG']}, clip={cfg['ASL_CLIP']}
  - Intensity: Smooth L1 Loss (weight: {cfg['LOSS_WEIGHTS']['intensity']})
- **Oversampling**: Motivational samples oversampled {cfg.get('MOTIVATIONAL_OVERSAMPLE_FACTOR', 5.0)}x
- **Prior Adjustment**: Emotion logits adjusted by training priors
- **Backbone Unfreezing**: Earlier at epoch {cfg.get('UNFREEZE_BACKBONE_EPOCH', 2)}

## Performance
- Best Composite Metric: {best_metric:.4f}

## Dataset
- Training Samples: {len(train_df)}
- Validation Samples: {len(val_df)}
- Train/val priors matched via stratified split
- Emotions: offensive≈{LABEL_PRIORS['offensive_pos_rate']:.1%}, motivational≈{LABEL_PRIORS['motivational_pos_rate']:.1%}

## Tuned Emotion Thresholds (Precision-Targeted)
- Humor: {cfg['EMO_THRESHOLDS'][0]:.3f}
- Sarcasm: {cfg['EMO_THRESHOLDS'][1]:.3f}
- Offensive: {cfg['EMO_THRESHOLDS'][2]:.3f}
- Motivational: {cfg['EMO_THRESHOLDS'][3]:.3f}

## Key Improvements
✓ Mitigated imbalance using ASL (γ_neg=6) for emotions, class-balanced CE for sentiment
✓ Oversampling for motivational, precision-targeted thresholds
✓ Earlier unfreeze (epoch 2) + higher sentiment task weight reduced neutral collapse
✓ Stratified split ensures train/val prior matching
"""
    
    model_card_path = os.path.join(cfg['CHECKPOINT_PATH'], 'model_card.md')
    with open(model_card_path, 'w') as f:
        f.write(model_card)
    
    print(f"\n✓ Model card saved to: {model_card_path}")
    
    print("\n" + "="*70)
    print("Part 7 Complete - Training finished")
    print("="*70)

except KeyboardInterrupt:
    print("\n\n⚠ Training interrupted by user")
except Exception as e:
    print(f"\n✗ Error during training: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 8: ENHANCED INFERENCE ====================
"""
Part 8: Inference with Tuned Thresholds
"""

class Predictor:
    """Enhanced predictor using tuned thresholds"""
    
    def __init__(self, checkpoint_path: str, device: str = 'cuda'):
        print("Initializing predictor...")
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        
        # Load checkpoint and config
        checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
        self.cfg = checkpoint['config']
        
        # Load model
        try:
            print("  Loading EnhancedFusionModel...")
            self.model = EnhancedFusionModel(self.cfg).to(self.device)
            print("  ✓ Using EnhancedFusionModel")
        except NameError:
            raise RuntimeError("EnhancedFusionModel not found! Run Part 3 first.")
        
        # Load model weights
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        
        print(f"✓ Model loaded from epoch {checkpoint['epoch']} with best metric {checkpoint['best_metric']:.4f}")
        
        # Load tokenizer and image transform
        self.tokenizer = get_tokenizer(self.cfg)
        self.transform = get_image_transform(self.cfg, is_train=False)
        
        # Get tuned thresholds
        self.emotion_thresholds = self.cfg.get('EMO_THRESHOLDS', [0.5, 0.5, 0.6, 0.6])
        print(f"✓ Using tuned thresholds: {self.emotion_thresholds}")
        
        # Create reverse mappings
        self.sentiment_map_rev = {
            0: 'very_positive',
            1: 'positive',
            2: 'neutral',
            3: 'negative',
            4: 'very_negative'
        }
        
        self.emotion_labels = ['humor', 'sarcasm', 'offensive', 'motivational']
        
        print("✓ Predictor initialized successfully")
    
    def predict(self, text: str, image_path: str) -> Dict:
        """Predict with tuned thresholds"""
        # Preprocess Text
        encoding = self.tokenizer(
            text,
            max_length=self.cfg['MAX_LEN'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        
        # Preprocess Image
        try:
            image = Image.open(image_path).convert('RGB')
            image_tensor = self.transform(image).unsqueeze(0).to(self.device)
        except Exception as e:
            print(f"Could not load image: {e}. Using blank image.")
            image_tensor = torch.zeros(1, 3, self.cfg['IMG_SIZE'], self.cfg['IMG_SIZE']).to(self.device)
        
        # Run Inference
        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask, image_tensor)
        
        # Post-process Outputs
        sentiment_pred_idx = torch.argmax(outputs['sentiment_logits'], dim=1).item()
        sentiment_label = self.sentiment_map_rev.get(sentiment_pred_idx, "unknown")
        sentiment_probs = torch.softmax(outputs['sentiment_logits'], dim=1).squeeze().cpu().numpy()
        
        # Use tuned thresholds for emotions
        emotion_probs = torch.sigmoid(outputs['emotion_logits']).squeeze().cpu().numpy()
        emotion_binary = (emotion_probs >= np.array(self.emotion_thresholds)).astype(int)
        
        emotion_preds = {
            label: {
                'probability': f"{prob:.2%}",
                'predicted': bool(binary),
                'threshold': f"{thr:.3f}"
            }
            for label, prob, binary, thr in zip(
                self.emotion_labels, 
                emotion_probs, 
                emotion_binary,
                self.emotion_thresholds
            )
        }
        
        intensity_score = outputs['intensity'].item()
        
        return {
            'sentiment': sentiment_label,
            'sentiment_confidence': f"{sentiment_probs[sentiment_pred_idx]:.2%}",
            'sentiment_probs': {
                self.sentiment_map_rev[i]: f"{p:.2%}" 
                for i, p in enumerate(sentiment_probs)
            },
            'emotions': emotion_preds,
            'intensity_score': f"{intensity_score:.4f}"
        }
    
    def predict_batch(self, text_list, image_path_list):
        """Predict for multiple samples"""
        results = []
        for text, img_path in zip(text_list, image_path_list):
            result = self.predict(text, img_path)
            results.append(result)
        return results


# ==================== Run Inference Example ====================
try:
    CHECKPOINT_DIR = "/kaggle/working/checkpoints"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    BEST_MODEL_PATH = os.path.join(CHECKPOINT_DIR, 'best_model.pt')
    
    if not os.path.exists(BEST_MODEL_PATH):
        print(f"✗ Checkpoint not found at {BEST_MODEL_PATH}")
        print("  Please train the model first (run Part 7)")
        raise FileNotFoundError(f"Model checkpoint not found: {BEST_MODEL_PATH}")
    
    print("\n" + "="*70)
    print("INITIALIZING ENHANCED PREDICTOR")
    print("="*70)
    
    predictor = Predictor(checkpoint_path=BEST_MODEL_PATH, device=DEVICE)
    
    print("\n" + "="*70)
    print("RUNNING SAMPLE PREDICTIONS")
    print("="*70)
    
    # Get column names
    image_col_options = ['image_name', 'image', 'img_name', 'filename', 'Unnamed: 0']
    image_col = next((col for col in image_col_options if col in val_df.columns), val_df.columns[0])
    
    text_col_options = ['text', 'ocr', 'ocr_text', 'caption', 'OCR_extracted_text']
    text_col = next((col for col in text_col_options if col in val_df.columns), None)
    
    # Select 5 random samples
    import random
    random.seed(42)
    sample_indices = random.sample(range(len(val_df)), min(5, len(val_df)))
    
    for idx in sample_indices:
        sample_row = val_df.iloc[idx]
        sample_text = str(sample_row.get(text_col, '')) if text_col else ''
        sample_image_name = str(sample_row.get(image_col, f'{idx}.jpg'))
        
        if not any(sample_image_name.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
            sample_image_name = f"{sample_image_name}.jpg"
        
        sample_image_path = os.path.join(val_img_dir, sample_image_name)
        
        print(f"\n{'='*70}")
        print(f"Sample {idx}:")
        print(f"  Text: '{sample_text[:100]}...'")
        print("="*70)
        
        if not os.path.exists(sample_image_path):
            print(f"⚠️ Image not found: {sample_image_path}")
            continue
        
        # Make prediction
        result = predictor.predict(text=sample_text, image_path=sample_image_path)
        
        print("\nPrediction Result:")
        print(f"  Sentiment: {result['sentiment']} (confidence: {result['sentiment_confidence']})")
        print(f"  Intensity: {result['intensity_score']}")
        print("  Emotions (with tuned thresholds):")
        for emotion, info in result['emotions'].items():
            pred_str = "✓ YES" if info['predicted'] else "✗ NO"
            print(f"    {emotion.capitalize()}: {info['probability']} {pred_str} (threshold: {info['threshold']})")
    
    print("\n" + "="*70)
    print("Part 8 Complete - Enhanced inference pipeline tested")
    print("="*70)

except FileNotFoundError as e:
    print(f"\n✗ {e}")
except Exception as e:
    print(f"\n✗ Error during inference: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 9: DETAILED EVALUATION ====================
"""
Part 9: Comprehensive Evaluation and Error Analysis
"""

from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def run_full_evaluation(predictor, val_loader):
    """Run model on entire validation set"""
    predictor.model.eval()
    all_preds = []
    
    pbar = tqdm(val_loader, desc="Running Full Evaluation")
    
    with torch.no_grad():
        for batch in pbar:
            input_ids = batch['input_ids'].to(predictor.device)
            attention_mask = batch['attention_mask'].to(predictor.device)
            image = batch['image'].to(predictor.device)
            
            outputs = predictor.model(input_ids, attention_mask, image)
            
            sentiment_preds = torch.argmax(outputs['sentiment_logits'], dim=1).cpu().numpy()
            
            # Use tuned thresholds for emotions
            emotion_probs = torch.sigmoid(outputs['emotion_logits']).cpu().numpy()
            emotion_thresholds = np.array(predictor.emotion_thresholds)
            emotion_preds = (emotion_probs >= emotion_thresholds).astype(int)
            
            intensity_preds = outputs['intensity'].cpu().numpy().flatten()
            
            sentiment_true = batch['sentiment_label'].cpu().numpy()
            emotion_true = batch['emotion_labels'].int().cpu().numpy()
            intensity_true = batch['intensity'].cpu().numpy().flatten()
            
            for i in range(len(input_ids)):
                all_preds.append({
                    'sentiment_pred': sentiment_preds[i],
                    'sentiment_true': sentiment_true[i],
                    'emotion_pred': emotion_preds[i],
                    'emotion_true': emotion_true[i],
                    'intensity_pred': intensity_preds[i],
                    'intensity_true': intensity_true[i]
                })
    
    results_df = pd.DataFrame(all_preds)
    results_df['sentiment_pred_label'] = results_df['sentiment_pred'].map(predictor.sentiment_map_rev)
    results_df['sentiment_true_label'] = results_df['sentiment_true'].map(predictor.sentiment_map_rev)
    
    return results_df


try:
    print("\n" + "="*70)
    print("STARTING DETAILED EVALUATION")
    print("="*70)
    
    if 'predictor' not in locals() or predictor is None:
        print("✗ Predictor not found. Please run Part 8 first.")
        raise NameError("Predictor not initialized")
    
    print("\nStarting full evaluation on validation set...")
    eval_results_df = run_full_evaluation(predictor, val_loader)
    
    # Merge with original validation dataframe
    full_analysis_df = pd.concat([val_df.reset_index(drop=True), eval_results_df], axis=1)
    full_analysis_df['sentiment_correct'] = full_analysis_df['sentiment_pred'] == full_analysis_df['sentiment_true']
    full_analysis_df['intensity_error'] = np.abs(
        full_analysis_df['intensity_pred'] - full_analysis_df['intensity_true']
    )
    
    print(f"\n✓ Full evaluation complete. Analysis DataFrame shape: {full_analysis_df.shape}")
    
    # ==================== Performance Summary ====================
    print("\n" + "="*70)
    print("PERFORMANCE SUMMARY")
    print("="*70)
    
    overall_acc = full_analysis_df['sentiment_correct'].mean()
    avg_intensity_error = full_analysis_df['intensity_error'].mean()
    
    print(f"\nOverall Metrics:")
    print(f"  Sentiment Accuracy: {overall_acc:.2%}")
    print(f"  Average Intensity Error: {avg_intensity_error:.4f}")
    
    print(f"\nPer-Class Sentiment Accuracy:")
    for sent_label in full_analysis_df['sentiment_true_label'].unique():
        class_df = full_analysis_df[full_analysis_df['sentiment_true_label'] == sent_label]
        if len(class_df) > 0:
            class_acc = class_df['sentiment_correct'].mean()
            print(f"  {sent_label}: {class_acc:.2%} ({len(class_df)} samples)")
    
    # ==================== Confusion Matrix ====================
    print("\n" + "="*70)
    print("CONFUSION MATRIX")
    print("="*70)
    
    sentiment_labels = sorted(full_analysis_df['sentiment_true_label'].unique())
    cm = confusion_matrix(
        full_analysis_df['sentiment_true_label'],
        full_analysis_df['sentiment_pred_label'],
        labels=sentiment_labels
    )
    
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=sentiment_labels)
    fig, ax = plt.subplots(figsize=(10, 10))
    disp.plot(cmap=plt.cm.Blues, ax=ax, xticks_rotation='vertical', values_format='d')
    plt.title('Sentiment Classification Confusion Matrix', fontsize=14, pad=20)
    plt.tight_layout()
    
    cm_path = '/kaggle/working/confusion_matrix.png'
    plt.savefig(cm_path, dpi=300, bbox_inches='tight')
    print(f"✓ Confusion matrix saved to {cm_path}")
    plt.show()
    
    # ==================== Save Results ====================
    results_csv_path = '/kaggle/working/detailed_evaluation_results.csv'
    full_analysis_df.to_csv(results_csv_path, index=False)
    print(f"\n✓ Detailed results saved to {results_csv_path}")
    
    print("\n" + "="*70)
    print("Part 9 Complete - Detailed evaluation finished")
    print("="*70)

except NameError as e:
    print(f"\n✗ {e}")
    print("  Run Part 8 first")
except Exception as e:
    print(f"\n✗ Error during detailed evaluation: {e}")
    import traceback
    traceback.print_exc()


# ==================== PART 10: FEATURE VISUALIZATION ====================
"""
Part 10: t-SNE Feature Visualization
"""

try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    
    print("\n" + "="*70)
    print("DIAGNOSTIC 5: Feature Visualization (t-SNE for Emotion Separability)")
    print("="*70)
    
    if 'predictor' not in locals() or predictor is None:
        print("✗ Predictor not found. Please run Part 8 first.")
        raise NameError("Predictor not initialized")
    
    model.eval()
    all_feat, all_offensive = [], []
    feature_key = 'text_features'
    
    print(f"\nExtracting {feature_key} from validation set...")
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Extracting features"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            image = batch['image'].to(device)
            
            out = model(input_ids, attention_mask, image)
            feat = out[feature_key]
            
            all_feat.append(feat.detach().cpu().numpy())
            all_offensive.append(batch['emotion_labels'][:, 2].detach().cpu().numpy())  # offensive is index 2
    
    X = np.concatenate(all_feat, 0)
    y = np.concatenate(all_offensive, 0)
    
    print(f"Feature matrix shape: {X.shape}")
    print(f"Offensive label distribution: {np.sum(y==1)} offensive, {np.sum(y==0)} non-offensive")
    
    print("\nRunning t-SNE dimensionality reduction...")
    Z = TSNE(n_components=2, perplexity=30, learning_rate='auto', 
             init='pca', random_state=42).fit_transform(X)
    
    plt.figure(figsize=(10, 8))
    plt.scatter(Z[y==0, 0], Z[y==0, 1], s=6, alpha=0.5, label='non-offensive', c='blue')
    plt.scatter(Z[y==1, 0], Z[y==1, 1], s=12, alpha=0.8, label='offensive', c='red')
    plt.legend()
    plt.title(f"t-SNE of {feature_key} — Offensive Separability", fontsize=14)
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    tsne_path = '/kaggle/working/tsne_offensive_separability.png'
    plt.savefig(tsne_path, dpi=300, bbox_inches='tight')
    print(f"✓ t-SNE plot saved to {tsne_path}")
    plt.show()
    
    print("\n" + "="*70)
    print("Part 10 Complete - All diagnostics finished!")
    print("="*70)
    
    print("\nGenerated Files:")
    print("  1. /kaggle/working/checkpoints/best_model.pt")
    print("  2. /kaggle/working/checkpoints/model_card.md")
    print("  3. /kaggle/working/confusion_matrix.png")
    print("  4. /kaggle/working/detailed_evaluation_results.csv")
    print("  5. /kaggle/working/tsne_offensive_separability.png")
    print("  6. /kaggle/working/label_priors.json")
    
    print("\n" + "="*70)
    print("ALL PARTS COMPLETE! 🎉")
    print("="*70)
    
    print("\n📊 SUMMARY OF ENHANCEMENTS:")
    print("="*70)
    print("✓ Stratified multi-label split (not last 1500 rows)")
    print("✓ ASL loss for emotions (γ_neg=6, clip=0.05)")
    print("✓ Class-balanced CE for sentiment")
    print("✓ WeightedRandomSampler for motivational (5x)")
    print("✓ Precision-targeted threshold tuning")
    print("✓ Earlier backbone unfreezing (epoch 2)")
    print("✓ Prior logit adjustment for emotions")
    print("✓ Label normalization and proper extension handling")
    print("✓ Comprehensive diagnostics and evaluation")
    print("="*70)

except Exception as e:
    print(f"\n✗ Error during t-SNE visualization: {e}")
    import traceback
    traceback.print_exc()


Initializing enhanced trainer...
  Optimizer: 48 head params, 0 backbone params
✓ Trainer initialized

Part 6 Complete - Enhanced Trainer ready

Key Changes:
✓ Backbone unfreezing moved to epoch 2 (was 3)
✓ Precision-targeted threshold tuning integrated
✓ ASL loss tracked separately in training
✓ Macro F1 used for early stopping

Run Part 7 to start training

Part 7: Training with Enhanced Diagnostics

DIAGNOSTIC 1: Tiny-batch Overfit & Gradient Sanity Check

Running 200 steps on tiny batch to verify model can overfit...


/tmp/ipykernel_47/1818283930.py:18: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler() if cfg['USE_AMP'] else None
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


000: total=3.2757 | sent=1.8085 | emo(ASL)=0.4532 | int=0.2195


KeyboardInterrupt: 